## Loading Best Acc list from Txt for single layer

In [1]:
#best_acc_eval_results_by_layer = {}
#best_f1_eval_results_by_layer = {}
import ast

with open('L12_Frozen_acc.txt') as file:
    data = file.read()    
file.close()
    
best_acc_eval_results_by_layer = ast.literal_eval(data)


with open('L12_Frozen_f1.txt') as file:
    data = file.read()    
file.close()


best_f1_eval_results_by_layer = ast.literal_eval(data)


In [36]:
print('Best Acc List:\n', best_acc_eval_results_by_layer)
print()
print('Best F1 List:\n', best_f1_eval_results_by_layer)

Best Acc List:
 {(12, 0): {'acc': 0.8580357142857142, 'f1': 0.7823453075568864}, (12, 0.25): {'acc': 0.8625, 'f1': 0.7960469155284686}, (12, 0.75): {'acc': 0.8544642857142857, 'f1': 0.7955473836394455}, (11, 0): {'acc': 0.8625, 'f1': 0.7983648942079483}, (10, 0): {'acc': 0.8526785714285714, 'f1': 0.7791258718917419}, (9, 0): {'acc': 0.8482142857142857, 'f1': 0.7775968480864099}, (8, 0): {'acc': 0.8383928571428572, 'f1': 0.7648445547046677}, (7, 0): {'acc': 0.8258928571428571, 'f1': 0.7403418274383831}, (6, 0): {'acc': 0.8089285714285714, 'f1': 0.7116461710857843}, (5, 0): {'acc': 0.7857142857142857, 'f1': 0.6710740356981434}, (4, 0): {'acc': 0.7848214285714286, 'f1': 0.6839390708793694}, (3, 0): {'acc': 0.7848214285714286, 'f1': 0.6624438214965042}, (2, 0): {'acc': 0.7758928571428572, 'f1': 0.6818424550257919}, (1, 0): {'acc': 0.7633928571428571, 'f1': 0.6241070849483733}, (0, 0): {'acc': 0.65, 'f1': 0.26262626262626265}, (12, 0.9): {'acc': 0.8375, 'f1': 0.7580973192536766}, (12, 0.5):

### Save Dictionary 

In [37]:
with open('L12_Frozen_acc.txt', 'w') as file:
    file.write(str(best_acc_eval_results_by_layer))
file.close()
    
with open('L12_Frozen_f1.txt', 'w') as file:
    file.write(str(best_f1_eval_results_by_layer))
file.close()

### Device Check

In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.current_device()
torch.cuda.device(0)


## Loading Dataset, Model

In [2]:
# coding=utf-8
import argparse
import logging
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import random

import numpy as np
import pandas as pd
import torch
from transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)
from torch.utils.data import DataLoader

from datasets import load_datasets_and_vocabs
from model import (Aspect_Text_GAT_ours,
                    Pure_Bert, Aspect_Bert_GAT, Aspect_Text_GAT_only)
from trainer import train

logger = logging.getLogger(__name__)

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

def parse_args(args):
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument('--dataset_name', type=str, default='rest',
                        choices=['rest', 'laptop', 'twitter'],
                        help='Choose absa dataset.')
    parser.add_argument('--output_dir', type=str, default='/data1/SHENWZH/ABSA_online/data/output-gcn',
                        help='Directory to store intermedia data, such as vocab, embeddings, tags_vocab.')
    parser.add_argument('--num_classes', type=int, default=3,
                        help='Number of classes of ABSA.')


    parser.add_argument('--cuda_id', type=str, default='3',
                        help='Choose which GPUs to run')
    parser.add_argument('--seed', type=int, default=2019,
                        help='random seed for initialization')

    # Model parameters
    parser.add_argument('--glove_dir', type=str, default='/data1/SHENWZH/wordvec',
                        help='Directory storing glove embeddings')
    parser.add_argument('--bert_model_dir', type=str, default='/data1/SHENWZH/models/bert_base',
                        help='Path to pre-trained Bert model.')
    parser.add_argument('--pure_bert', action='store_true',
                        help='Cat text and aspect, [cls] to predict.')
    
    parser.add_argument('--pure_bert_layer_agg', action='store_true',
                        help='Pure bert layer aggregation enable/not')

    parser.add_argument('--pure_bert_layer_agg_list', type=str, default="12",
                        help='Pure Bert layer number to aggregate')

    parser.add_argument('--pure_bert_linear_layer_count', type=int, default=2,
                        help='Pure Bert final linear layer count')
    
    parser.add_argument('--gat_bert', action='store_true',
                        help='Cat text and aspect, [cls] to predict.')

    parser.add_argument('--highway', action='store_true',
                        help='Use highway embed.')

    parser.add_argument('--num_layers', type=int, default=2,
                        help='Number of layers of bilstm or highway or elmo.')
    
    parser.add_argument('--hidden_layer_no', type=int, default=12,
                        help='NO of hidder layer of bert state output to FC.')
 
    parser.add_argument('--frozen_percent', type=float, default = 0.1,
                        help='Percentage of the frozen layers.')

    parser.add_argument('--hidden_layer_a_no', type=int, default=12,
                        help='NO of hidder layer 1 of bert state output to FC.')
    
    parser.add_argument('--hidden_layer_b_no', type=int, default=11,
                        help='NO of hidder layer 2 of bert state output to FC.')

    parser.add_argument('--add_non_connect',  type= bool, default=True,
                        help='Add a sepcial "non-connect" relation for aspect with no direct connection.')
    parser.add_argument('--multi_hop',  type= bool, default=True,
                        help='Multi hop non connection.')
    parser.add_argument('--max_hop', type = int, default=4,
                        help='max number of hops')


    parser.add_argument('--num_heads', type=int, default=6,
                        help='Number of heads for gat.')
    
    parser.add_argument('--dropout', type=float, default=0,
                        help='Dropout rate for embedding.')


    parser.add_argument('--num_gcn_layers', type=int, default=1,
                        help='Number of GCN layers.')
    parser.add_argument('--gcn_mem_dim', type=int, default=300,
                        help='Dimension of the W in GCN.')
    parser.add_argument('--gcn_dropout', type=float, default=0.2,
                        help='Dropout rate for GCN.')
    # GAT
    parser.add_argument('--gat', action='store_true',
                        help='GAT')
    parser.add_argument('--gat_our', action='store_true',
                        help='GAT_our')
    parser.add_argument('--gat_attention_type', type = str, choices=['linear','dotprod','gcn'], default='dotprod',
                        help='The attention used for gat')

    parser.add_argument('--embedding_type', type=str,default='glove', choices=['glove','bert'])
    parser.add_argument('--embedding_dim', type=int, default=300,
                        help='Dimension of glove embeddings')
    parser.add_argument('--dep_relation_embed_dim', type=int, default=300,
                        help='Dimension for dependency relation embeddings.')

    parser.add_argument('--hidden_size', type=int, default=300,
                        help='Hidden size of bilstm, in early stage.')
    parser.add_argument('--final_hidden_size', type=int, default=300,
                        help='Hidden size of bilstm, in early stage.')
    parser.add_argument('--num_mlps', type=int, default=2,
                        help='Number of mlps in the last of model.')

    # Training parameters
    parser.add_argument("--per_gpu_train_batch_size", default=16, type=int,
                        help="Batch size per GPU/CPU for training.")
    parser.add_argument("--per_gpu_eval_batch_size", default=32, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
    parser.add_argument('--gradient_accumulation_steps', type=int, default=2,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
    parser.add_argument("--learning_rate", default=1e-3, type=float,
                        help="The initial learning rate for Adam.")
    
    parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                        help="Epsilon for Adam optimizer.")

    parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
    parser.add_argument("--num_train_epochs", default=30.0, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps(that update the weights) to perform. Override num_train_epochs.")
    parser.add_argument('--logging_steps', type=int, default=50,
                        help="Log every X updates steps.")
    
    return parser.parse_args(args)


def check_args(args):
    '''
    eliminate confilct situations
    
    '''
    logger.info(vars(args))

/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Single Layer Model 

In [3]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
    
# Parse args

args_str = "--embedding_type bert --output_dir data/output-gcn --dropout 0.3 --hidden_size 200 --learning_rate 5e-5 --bert_model_dir ./test/saved_model --pure_bert --hidden_layer_no 12 --frozen_percent 0"
#args = parse_args(['--gat_our', '--highway', '--num_heads', '7', '--dropout', '0.8', '--output_dir',
#                   'output/r-gat', '--glove_dir', 'glove', '--cuda_id', '0'])
args = parse_args(args_str.split(' '))
check_args(args)

# Setup CUDA, GPU training
os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_id
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args.device = device
logger.info('Device is %s', args.device)

# Set seed
set_seed(args)

# Bert, load pretrained model and tokenizer, check if neccesary to put bert here
if args.embedding_type == 'bert':
    tokenizer = BertTokenizer.from_pretrained(args.bert_model_dir)
    args.tokenizer = tokenizer

# Load datasets and vocabs
train_dataset, test_dataset, word_vocab, dep_tag_vocab, pos_tag_vocab= load_datasets_and_vocabs(args)




07/25/2021 16:41:19 - INFO - __main__ -   {'dataset_name': 'rest', 'output_dir': 'data/output-gcn', 'num_classes': 3, 'cuda_id': '3', 'seed': 2019, 'glove_dir': '/data1/SHENWZH/wordvec', 'bert_model_dir': './test/saved_model', 'pure_bert': True, 'pure_bert_layer_agg': False, 'pure_bert_layer_agg_list': '12', 'pure_bert_linear_layer_count': 2, 'gat_bert': False, 'highway': False, 'num_layers': 2, 'hidden_layer_no': 12, 'frozen_percent': 0.0, 'hidden_layer_a_no': 12, 'hidden_layer_b_no': 11, 'add_non_connect': True, 'multi_hop': True, 'max_hop': 4, 'num_heads': 6, 'dropout': 0.3, 'num_gcn_layers': 1, 'gcn_mem_dim': 300, 'gcn_dropout': 0.2, 'gat': False, 'gat_our': False, 'gat_attention_type': 'dotprod', 'embedding_type': 'bert', 'embedding_dim': 300, 'dep_relation_embed_dim': 300, 'hidden_size': 200, 'final_hidden_size': 300, 'num_mlps': 2, 'per_gpu_train_batch_size': 16, 'per_gpu_eval_batch_size': 32, 'gradient_accumulation_steps': 2, 'learning_rate': 5e-05, 'weight_decay': 0.0, 'adam_e

# Pure Bert - Single Layer 

## Output of Bert, output[12] All Frozen

In [11]:
args.hidden_layer_no = 12
args.frozen_percent = 1

# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, arg.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, arg.frozen_percent)] = best_f1_eval_result

07/23/2021 02:14:52 - INFO - model -   Hidden Layer 12
07/23/2021 02:14:52 - INFO - trainer -   ***** Running training *****
07/23/2021 02:14:52 - INFO - trainer -     Num examples = 3602
07/23/2021 02:14:52 - INFO - trainer -     Num Epochs = 30
07/23/2021 02:14:52 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 02:14:52 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 02:14:52 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 02:14:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:14:55 - INFO - trainer -     Num examples = 1120
07/23/2021 02:14:55 - INFO - trainer -     Batch size = 32
07/23/2021 02:14:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:14:56 - INFO - trainer -     eval loss: 0.9064929451261248
07/23/2021 02:14:56 - INFO - trainer -     acc = 0.6491071428571429
07/23/2021 02:14:56 - INFO - trainer -     f1 = 0.2624075076700957
07/23/202

07/23/2021 02:15:48 - INFO - trainer -     acc = 0.66875
07/23/2021 02:15:48 - INFO - trainer -     f1 = 0.35051252443039344
07/23/2021 02:15:51 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:15:51 - INFO - trainer -     Num examples = 1120
07/23/2021 02:15:51 - INFO - trainer -     Batch size = 32
07/23/2021 02:15:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:15:52 - INFO - trainer -     eval loss: 0.7815773725509644
07/23/2021 02:15:52 - INFO - trainer -     acc = 0.6660714285714285
07/23/2021 02:15:52 - INFO - trainer -     f1 = 0.34528654773689427
07/23/2021 02:15:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:15:54 - INFO - trainer -     Num examples = 1120
07/23/2021 02:15:54 - INFO - trainer -     Batch size = 32
07/23/2021 02:15:55 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:15:55 - INFO - trainer -     eval loss: 0.7860433169773646
07/23/2021 02:15:55 - INFO - trainer -     acc = 0.6767857142857143
07/23

07/23/2021 02:16:48 - INFO - trainer -     eval loss: 0.7194710178034646
07/23/2021 02:16:48 - INFO - trainer -     acc = 0.7044642857142858
07/23/2021 02:16:48 - INFO - trainer -     f1 = 0.46891240565291886
Epoch:  50%|█████     | 15/30 [01:57<01:56,  7.75s/it]07/23/2021 02:16:50 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:16:50 - INFO - trainer -     Num examples = 1120
07/23/2021 02:16:50 - INFO - trainer -     Batch size = 32
07/23/2021 02:16:51 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:16:51 - INFO - trainer -     eval loss: 0.7247870666640145
07/23/2021 02:16:51 - INFO - trainer -     acc = 0.7008928571428571
07/23/2021 02:16:51 - INFO - trainer -     f1 = 0.45426988613989755
07/23/2021 02:16:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:16:53 - INFO - trainer -     Num examples = 1120
07/23/2021 02:16:53 - INFO - trainer -     Batch size = 32
07/23/2021 02:16:55 - INFO - trainer -   ***** Eval results *****
07/23/20

07/23/2021 02:17:47 - INFO - trainer -     Batch size = 32
07/23/2021 02:17:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:17:49 - INFO - trainer -     eval loss: 0.6892816594668797
07/23/2021 02:17:49 - INFO - trainer -     acc = 0.7160714285714286
07/23/2021 02:17:49 - INFO - trainer -     f1 = 0.4912712749682577
07/23/2021 02:17:51 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:17:51 - INFO - trainer -     Num examples = 1120
07/23/2021 02:17:51 - INFO - trainer -     Batch size = 32
07/23/2021 02:17:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:17:52 - INFO - trainer -     eval loss: 0.6935524501970836
07/23/2021 02:17:52 - INFO - trainer -     acc = 0.7089285714285715
07/23/2021 02:17:52 - INFO - trainer -     f1 = 0.4695821032834326
Epoch:  77%|███████▋  | 23/30 [03:02<00:56,  8.09s/it]07/23/2021 02:17:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:17:55 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 02:18:47 - INFO - trainer -     Num examples = 1120
07/23/2021 02:18:47 - INFO - trainer -     Batch size = 32
07/23/2021 02:18:48 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:18:48 - INFO - trainer -     eval loss: 0.6775206297636032
07/23/2021 02:18:48 - INFO - trainer -     acc = 0.7267857142857143
07/23/2021 02:18:48 - INFO - trainer -     f1 = 0.5151736707334873
Epoch: 100%|██████████| 30/30 [03:57<00:00,  7.92s/it]
07/23/2021 02:18:50 - INFO - __main__ -     acc = 0.7366071428571429
07/23/2021 02:18:50 - INFO - __main__ -     f1 = 0.5572514607994633


### Output of Bert, output[12] Frozen 0

In [4]:
args.hidden_layer_no = 12
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 03:00:39 - INFO - model -   Hidden Layer 12 frozen percentage 0
07/23/2021 03:00:41 - INFO - trainer -   ***** Running training *****
07/23/2021 03:00:41 - INFO - trainer -     Num examples = 3602
07/23/2021 03:00:41 - INFO - trainer -     Num Epochs = 30
07/23/2021 03:00:41 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 03:00:41 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 03:00:41 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 03:00:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:00:49 - INFO - trainer -     Num examples = 1120
07/23/2021 03:00:49 - INFO - trainer -     Batch size = 32
07/23/2021 03:00:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:00:50 - INFO - trainer -     eval loss: 0.6072911794696535
07/23/2021 03:00:50 - INFO - trainer -     acc = 0.7607142857142857
07/23/2021 03:00:50 - INFO - trainer -     f1 = 0.545665

07/23/2021 03:03:00 - INFO - trainer -     acc = 0.8580357142857142
07/23/2021 03:03:00 - INFO - trainer -     f1 = 0.7823453075568864
07/23/2021 03:03:08 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:03:08 - INFO - trainer -     Num examples = 1120
07/23/2021 03:03:08 - INFO - trainer -     Batch size = 32
07/23/2021 03:03:09 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:03:09 - INFO - trainer -     eval loss: 0.8051507401280105
07/23/2021 03:03:09 - INFO - trainer -     acc = 0.8410714285714286
07/23/2021 03:03:09 - INFO - trainer -     f1 = 0.7556317379138645
07/23/2021 03:03:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:03:17 - INFO - trainer -     Num examples = 1120
07/23/2021 03:03:17 - INFO - trainer -     Batch size = 32
07/23/2021 03:03:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:03:18 - INFO - trainer -     eval loss: 0.8362530197183202
07/23/2021 03:03:18 - INFO - trainer -     acc = 0.8392857142857

07/23/2021 03:05:27 - INFO - trainer -     eval loss: 1.0138292685467085
07/23/2021 03:05:27 - INFO - trainer -     acc = 0.8517857142857143
07/23/2021 03:05:27 - INFO - trainer -     f1 = 0.7839190006750095
Epoch:  50%|█████     | 15/30 [04:52<04:48, 19.22s/it]07/23/2021 03:05:34 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:05:34 - INFO - trainer -     Num examples = 1120
07/23/2021 03:05:34 - INFO - trainer -     Batch size = 32
07/23/2021 03:05:36 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:05:36 - INFO - trainer -     eval loss: 1.0893574889233735
07/23/2021 03:05:36 - INFO - trainer -     acc = 0.8410714285714286
07/23/2021 03:05:36 - INFO - trainer -     f1 = 0.7543556908053791
07/23/2021 03:05:43 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:05:43 - INFO - trainer -     Num examples = 1120
07/23/2021 03:05:43 - INFO - trainer -     Batch size = 32
07/23/2021 03:05:44 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 03:07:50 - INFO - trainer -     Batch size = 32
07/23/2021 03:07:51 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:07:51 - INFO - trainer -     eval loss: 1.1743714654606134
07/23/2021 03:07:51 - INFO - trainer -     acc = 0.8321428571428572
07/23/2021 03:07:51 - INFO - trainer -     f1 = 0.761935087584944
07/23/2021 03:07:58 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:07:58 - INFO - trainer -     Num examples = 1120
07/23/2021 03:07:58 - INFO - trainer -     Batch size = 32
07/23/2021 03:07:59 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:07:59 - INFO - trainer -     eval loss: 1.2112254739244235
07/23/2021 03:07:59 - INFO - trainer -     acc = 0.8366071428571429
07/23/2021 03:07:59 - INFO - trainer -     f1 = 0.7583271659048078
Epoch:  77%|███████▋  | 23/30 [07:25<02:12, 18.95s/it]07/23/2021 03:08:06 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:08:06 - INFO - trainer -     Num examples = 1120
07/23/2021 

07/23/2021 03:10:14 - INFO - trainer -     Num examples = 1120
07/23/2021 03:10:14 - INFO - trainer -     Batch size = 32
07/23/2021 03:10:15 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:10:15 - INFO - trainer -     eval loss: 1.15045785567823
07/23/2021 03:10:15 - INFO - trainer -     acc = 0.8392857142857143
07/23/2021 03:10:15 - INFO - trainer -     f1 = 0.7704319640260014
Epoch: 100%|██████████| 30/30 [09:39<00:00, 19.31s/it]
07/23/2021 03:10:20 - INFO - __main__ -     acc = 0.8580357142857142
07/23/2021 03:10:20 - INFO - __main__ -     f1 = 0.7823453075568864


NameError: name 'best_acc_eval_results_by_layer' is not defined

In [7]:
best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

In [8]:
best_acc_eval_results_by_layer

{(12, 0): {'acc': 0.8580357142857142, 'f1': 0.7823453075568864}}

In [9]:
best_f1_eval_results_by_layer

{(12, 0): {'acc': 0.8580357142857142, 'f1': 0.794648953589351}}

### Output of Bert, output[12] Frozen 0, Comfirm

In [4]:
args.hidden_layer_no = 12
args.frozen_percent = 0

# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, arg.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, arg.frozen_percent)] = best_f1_eval_result

07/25/2021 16:43:03 - INFO - model -   Hidden Layer 12 frozen percentage 0
07/25/2021 16:43:08 - INFO - trainer -   ***** Running training *****
07/25/2021 16:43:08 - INFO - trainer -     Num examples = 3602
07/25/2021 16:43:08 - INFO - trainer -     Num Epochs = 30
07/25/2021 16:43:08 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/25/2021 16:43:08 - INFO - trainer -     Gradient Accumulation steps = 2
07/25/2021 16:43:08 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/25/2021 16:43:17 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 16:43:17 - INFO - trainer -     Num examples = 1120
07/25/2021 16:43:17 - INFO - trainer -     Batch size = 32
07/25/2021 16:43:18 - INFO - trainer -   ***** Eval results *****
07/25/2021 16:43:18 - INFO - trainer -     eval loss: 0.6072911794696535
07/25/2021 16:43:18 - INFO - trainer -     acc = 0.7607142857142857
07/25/2021 16:43:18 - INFO - trainer -     f1 = 0.545665

07/25/2021 16:45:33 - INFO - trainer -     acc = 0.8580357142857142
07/25/2021 16:45:33 - INFO - trainer -     f1 = 0.7823453075568864
07/25/2021 16:45:41 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 16:45:41 - INFO - trainer -     Num examples = 1120
07/25/2021 16:45:41 - INFO - trainer -     Batch size = 32
07/25/2021 16:45:42 - INFO - trainer -   ***** Eval results *****
07/25/2021 16:45:42 - INFO - trainer -     eval loss: 0.8051507401280105
07/25/2021 16:45:42 - INFO - trainer -     acc = 0.8410714285714286
07/25/2021 16:45:42 - INFO - trainer -     f1 = 0.7556317379138645
07/25/2021 16:45:50 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 16:45:50 - INFO - trainer -     Num examples = 1120
07/25/2021 16:45:50 - INFO - trainer -     Batch size = 32
07/25/2021 16:45:51 - INFO - trainer -   ***** Eval results *****
07/25/2021 16:45:51 - INFO - trainer -     eval loss: 0.8362530197183202
07/25/2021 16:45:51 - INFO - trainer -     acc = 0.8392857142857

07/25/2021 16:48:07 - INFO - trainer -     eval loss: 1.0138292685467085
07/25/2021 16:48:07 - INFO - trainer -     acc = 0.8517857142857143
07/25/2021 16:48:07 - INFO - trainer -     f1 = 0.7839190006750095
Epoch:  50%|█████     | 15/30 [05:05<05:00, 20.03s/it]07/25/2021 16:48:14 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 16:48:14 - INFO - trainer -     Num examples = 1120
07/25/2021 16:48:14 - INFO - trainer -     Batch size = 32
07/25/2021 16:48:15 - INFO - trainer -   ***** Eval results *****
07/25/2021 16:48:15 - INFO - trainer -     eval loss: 1.0893574889233735
07/25/2021 16:48:15 - INFO - trainer -     acc = 0.8410714285714286
07/25/2021 16:48:15 - INFO - trainer -     f1 = 0.7543556908053791
07/25/2021 16:48:23 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 16:48:23 - INFO - trainer -     Num examples = 1120
07/25/2021 16:48:23 - INFO - trainer -     Batch size = 32
07/25/2021 16:48:24 - INFO - trainer -   ***** Eval results *****
07/25/2021

07/25/2021 16:50:37 - INFO - trainer -     Batch size = 32
07/25/2021 16:50:38 - INFO - trainer -   ***** Eval results *****
07/25/2021 16:50:38 - INFO - trainer -     eval loss: 1.1743714654606134
07/25/2021 16:50:38 - INFO - trainer -     acc = 0.8321428571428572
07/25/2021 16:50:38 - INFO - trainer -     f1 = 0.761935087584944
07/25/2021 16:50:46 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 16:50:46 - INFO - trainer -     Num examples = 1120
07/25/2021 16:50:46 - INFO - trainer -     Batch size = 32
07/25/2021 16:50:47 - INFO - trainer -   ***** Eval results *****
07/25/2021 16:50:47 - INFO - trainer -     eval loss: 1.2112254739244235
07/25/2021 16:50:47 - INFO - trainer -     acc = 0.8366071428571429
07/25/2021 16:50:47 - INFO - trainer -     f1 = 0.7583271659048078
Epoch:  77%|███████▋  | 23/30 [07:46<02:19, 19.99s/it]07/25/2021 16:50:55 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 16:50:55 - INFO - trainer -     Num examples = 1120
07/25/2021 

07/25/2021 16:53:09 - INFO - trainer -     Num examples = 1120
07/25/2021 16:53:09 - INFO - trainer -     Batch size = 32
07/25/2021 16:53:10 - INFO - trainer -   ***** Eval results *****
07/25/2021 16:53:10 - INFO - trainer -     eval loss: 1.15045785567823
07/25/2021 16:53:10 - INFO - trainer -     acc = 0.8392857142857143
07/25/2021 16:53:10 - INFO - trainer -     f1 = 0.7704319640260014
Epoch: 100%|██████████| 30/30 [10:08<00:00, 20.27s/it]
07/25/2021 16:53:16 - INFO - __main__ -     acc = 0.8580357142857142
07/25/2021 16:53:16 - INFO - __main__ -     f1 = 0.7823453075568864


NameError: name 'best_acc_eval_results_by_layer' is not defined

In [6]:
best_acc_eval_result

{'acc': 0.8580357142857142, 'f1': 0.7823453075568864}

In [7]:
best_f1_eval_result

{'acc': 0.8580357142857142, 'f1': 0.794648953589351}

### Output of Bert, output[12] Frozen 0.1

In [35]:
args.hidden_layer_no = 12
args.frozen_percent = 0.1
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 22:00:01 - INFO - model -   Hidden Layer 12 frozen percentage 0.1
07/23/2021 22:00:01 - INFO - trainer -   ***** Running training *****
07/23/2021 22:00:01 - INFO - trainer -     Num examples = 3602
07/23/2021 22:00:01 - INFO - trainer -     Num Epochs = 30
07/23/2021 22:00:01 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 22:00:01 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 22:00:01 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 22:00:08 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:00:08 - INFO - trainer -     Num examples = 1120
07/23/2021 22:00:08 - INFO - trainer -     Batch size = 32
07/23/2021 22:00:09 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:00:09 - INFO - trainer -     eval loss: 0.5813058908496584
07/23/2021 22:00:09 - INFO - trainer -     acc = 0.7598214285714285
07/23/2021 22:00:09 - INFO - trainer -     f1 = 0.5596

07/23/2021 22:02:10 - INFO - trainer -     acc = 0.8517857142857143
07/23/2021 22:02:10 - INFO - trainer -     f1 = 0.7812867581131694
07/23/2021 22:02:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:02:17 - INFO - trainer -     Num examples = 1120
07/23/2021 22:02:17 - INFO - trainer -     Batch size = 32
07/23/2021 22:02:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:02:18 - INFO - trainer -     eval loss: 0.7829165961194251
07/23/2021 22:02:18 - INFO - trainer -     acc = 0.8473214285714286
07/23/2021 22:02:18 - INFO - trainer -     f1 = 0.7725238265783242
07/23/2021 22:02:25 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:02:25 - INFO - trainer -     Num examples = 1120
07/23/2021 22:02:25 - INFO - trainer -     Batch size = 32
07/23/2021 22:02:27 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:02:27 - INFO - trainer -     eval loss: 0.7755085055716335
07/23/2021 22:02:27 - INFO - trainer -     acc = 0.8428571428571

07/23/2021 22:04:26 - INFO - trainer -     eval loss: 1.1507861780734467
07/23/2021 22:04:26 - INFO - trainer -     acc = 0.8348214285714286
07/23/2021 22:04:26 - INFO - trainer -     f1 = 0.7434570827113177
Epoch:  50%|█████     | 15/30 [04:31<04:26, 17.79s/it]07/23/2021 22:04:33 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:04:33 - INFO - trainer -     Num examples = 1120
07/23/2021 22:04:33 - INFO - trainer -     Batch size = 32
07/23/2021 22:04:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:04:34 - INFO - trainer -     eval loss: 1.1823919384108323
07/23/2021 22:04:34 - INFO - trainer -     acc = 0.8294642857142858
07/23/2021 22:04:34 - INFO - trainer -     f1 = 0.7394606395191511
07/23/2021 22:04:41 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:04:41 - INFO - trainer -     Num examples = 1120
07/23/2021 22:04:41 - INFO - trainer -     Batch size = 32
07/23/2021 22:04:42 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 22:06:40 - INFO - trainer -     Batch size = 32
07/23/2021 22:06:41 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:06:41 - INFO - trainer -     eval loss: 1.1901561822521867
07/23/2021 22:06:41 - INFO - trainer -     acc = 0.8339285714285715
07/23/2021 22:06:41 - INFO - trainer -     f1 = 0.740573213195832
07/23/2021 22:06:48 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:06:48 - INFO - trainer -     Num examples = 1120
07/23/2021 22:06:48 - INFO - trainer -     Batch size = 32
07/23/2021 22:06:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:06:49 - INFO - trainer -     eval loss: 1.3690346521637236
07/23/2021 22:06:49 - INFO - trainer -     acc = 0.8321428571428572
07/23/2021 22:06:49 - INFO - trainer -     f1 = 0.7345421742718733
Epoch:  77%|███████▋  | 23/30 [06:54<02:04, 17.76s/it]07/23/2021 22:06:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:06:56 - INFO - trainer -     Num examples = 1120
07/23/2021 

07/23/2021 22:08:54 - INFO - trainer -     Num examples = 1120
07/23/2021 22:08:54 - INFO - trainer -     Batch size = 32
07/23/2021 22:08:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:08:56 - INFO - trainer -     eval loss: 1.4187898783281396
07/23/2021 22:08:56 - INFO - trainer -     acc = 0.8348214285714286
07/23/2021 22:08:56 - INFO - trainer -     f1 = 0.7276603962023368
Epoch: 100%|██████████| 30/30 [08:59<00:00, 17.99s/it]
07/23/2021 22:09:01 - INFO - __main__ -     acc = 0.8553571428571428
07/23/2021 22:09:01 - INFO - __main__ -     f1 = 0.7861172628573901


### Output of Bert, output[12] Frozen 0.25

In [10]:
args.hidden_layer_no = 12
args.frozen_percent = 0.25
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 03:15:48 - INFO - model -   Hidden Layer 12 frozen percentage 0.25
07/23/2021 03:15:49 - INFO - trainer -   ***** Running training *****
07/23/2021 03:15:49 - INFO - trainer -     Num examples = 3602
07/23/2021 03:15:49 - INFO - trainer -     Num Epochs = 30
07/23/2021 03:15:49 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 03:15:49 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 03:15:49 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 03:15:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:15:54 - INFO - trainer -     Num examples = 1120
07/23/2021 03:15:54 - INFO - trainer -     Batch size = 32
07/23/2021 03:15:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:15:56 - INFO - trainer -     eval loss: 0.5889721155166626
07/23/2021 03:15:56 - INFO - trainer -     acc = 0.7821428571428571
07/23/2021 03:15:56 - INFO - trainer -     f1 = 0.620

07/23/2021 03:17:43 - INFO - trainer -     acc = 0.8419642857142857
07/23/2021 03:17:43 - INFO - trainer -     f1 = 0.7656895263010371
07/23/2021 03:17:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:17:49 - INFO - trainer -     Num examples = 1120
07/23/2021 03:17:49 - INFO - trainer -     Batch size = 32
07/23/2021 03:17:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:17:50 - INFO - trainer -     eval loss: 0.7680219803138503
07/23/2021 03:17:50 - INFO - trainer -     acc = 0.85
07/23/2021 03:17:50 - INFO - trainer -     f1 = 0.7725526024299278
07/23/2021 03:17:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:17:56 - INFO - trainer -     Num examples = 1120
07/23/2021 03:17:56 - INFO - trainer -     Batch size = 32
07/23/2021 03:17:57 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:17:57 - INFO - trainer -     eval loss: 0.691530282090285
07/23/2021 03:17:57 - INFO - trainer -     acc = 0.8616071428571429
07/23/2021 

07/23/2021 03:19:43 - INFO - trainer -     acc = 0.8553571428571428
07/23/2021 03:19:43 - INFO - trainer -     f1 = 0.7847646527953435
Epoch:  50%|█████     | 15/30 [03:59<03:57, 15.82s/it]07/23/2021 03:19:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:19:49 - INFO - trainer -     Num examples = 1120
07/23/2021 03:19:49 - INFO - trainer -     Batch size = 32
07/23/2021 03:19:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:19:50 - INFO - trainer -     eval loss: 1.158260177055906
07/23/2021 03:19:50 - INFO - trainer -     acc = 0.84375
07/23/2021 03:19:50 - INFO - trainer -     f1 = 0.7532528634132382
07/23/2021 03:19:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:19:56 - INFO - trainer -     Num examples = 1120
07/23/2021 03:19:56 - INFO - trainer -     Batch size = 32
07/23/2021 03:19:57 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:19:57 - INFO - trainer -     eval loss: 1.1301005523161232
07/23/2021 03:19:57 - 

07/23/2021 03:21:43 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:21:43 - INFO - trainer -     eval loss: 1.083670751673136
07/23/2021 03:21:43 - INFO - trainer -     acc = 0.8455357142857143
07/23/2021 03:21:43 - INFO - trainer -     f1 = 0.7648040728444375
07/23/2021 03:21:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:21:49 - INFO - trainer -     Num examples = 1120
07/23/2021 03:21:49 - INFO - trainer -     Batch size = 32
07/23/2021 03:21:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:21:50 - INFO - trainer -     eval loss: 1.1377573055543637
07/23/2021 03:21:50 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 03:21:50 - INFO - trainer -     f1 = 0.7480734054844126
Epoch:  77%|███████▋  | 23/30 [06:07<01:50, 15.83s/it]07/23/2021 03:21:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:21:56 - INFO - trainer -     Num examples = 1120
07/23/2021 03:21:56 - INFO - trainer -     Batch size = 32
07/23/2021 

07/23/2021 03:23:42 - INFO - trainer -     Batch size = 32
07/23/2021 03:23:44 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:23:44 - INFO - trainer -     eval loss: 1.2049756014204052
07/23/2021 03:23:44 - INFO - trainer -     acc = 0.83125
07/23/2021 03:23:44 - INFO - trainer -     f1 = 0.7377088804997779
Epoch: 100%|██████████| 30/30 [07:59<00:00, 15.99s/it]
07/23/2021 03:23:48 - INFO - __main__ -     acc = 0.8625
07/23/2021 03:23:48 - INFO - __main__ -     f1 = 0.7960469155284686


### Output of Bert, output[12] Frozen 0.25 Confirm 

In [4]:
args.hidden_layer_no = 12
args.frozen_percent = 0.25
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 06:38:22 - INFO - model -   Hidden Layer 12 frozen percentage 0.25
07/24/2021 06:38:27 - INFO - trainer -   ***** Running training *****
07/24/2021 06:38:27 - INFO - trainer -     Num examples = 3602
07/24/2021 06:38:27 - INFO - trainer -     Num Epochs = 30
07/24/2021 06:38:27 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 06:38:27 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 06:38:27 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 06:38:34 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 06:38:34 - INFO - trainer -     Num examples = 1120
07/24/2021 06:38:34 - INFO - trainer -     Batch size = 32
07/24/2021 06:38:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 06:38:35 - INFO - trainer -     eval loss: 0.5556343048810959
07/24/2021 06:38:35 - INFO - trainer -     acc = 0.7875
07/24/2021 06:38:35 - INFO - trainer -     f1 = 0.634383752084603

07/24/2021 06:40:24 - INFO - trainer -     acc = 0.8366071428571429
07/24/2021 06:40:24 - INFO - trainer -     f1 = 0.7460672336772384
07/24/2021 06:40:30 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 06:40:30 - INFO - trainer -     Num examples = 1120
07/24/2021 06:40:30 - INFO - trainer -     Batch size = 32
07/24/2021 06:40:31 - INFO - trainer -   ***** Eval results *****
07/24/2021 06:40:31 - INFO - trainer -     eval loss: 0.7779508674949673
07/24/2021 06:40:31 - INFO - trainer -     acc = 0.85
07/24/2021 06:40:31 - INFO - trainer -     f1 = 0.7783696038084669
07/24/2021 06:40:37 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 06:40:37 - INFO - trainer -     Num examples = 1120
07/24/2021 06:40:37 - INFO - trainer -     Batch size = 32
07/24/2021 06:40:38 - INFO - trainer -   ***** Eval results *****
07/24/2021 06:40:38 - INFO - trainer -     eval loss: 0.8730585308712242
07/24/2021 06:40:38 - INFO - trainer -     acc = 0.8357142857142857
07/24/2021

07/24/2021 06:42:25 - INFO - trainer -     eval loss: 0.998059534539269
07/24/2021 06:42:25 - INFO - trainer -     acc = 0.8473214285714286
07/24/2021 06:42:25 - INFO - trainer -     f1 = 0.7645540429987681
Epoch:  50%|█████     | 15/30 [04:03<03:59, 15.95s/it]07/24/2021 06:42:31 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 06:42:31 - INFO - trainer -     Num examples = 1120
07/24/2021 06:42:31 - INFO - trainer -     Batch size = 32
07/24/2021 06:42:32 - INFO - trainer -   ***** Eval results *****
07/24/2021 06:42:32 - INFO - trainer -     eval loss: 0.9662106688554299
07/24/2021 06:42:32 - INFO - trainer -     acc = 0.8535714285714285
07/24/2021 06:42:32 - INFO - trainer -     f1 = 0.7828630894031737
07/24/2021 06:42:38 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 06:42:38 - INFO - trainer -     Num examples = 1120
07/24/2021 06:42:38 - INFO - trainer -     Batch size = 32
07/24/2021 06:42:39 - INFO - trainer -   ***** Eval results *****
07/24/2021 

07/24/2021 06:44:25 - INFO - trainer -     Batch size = 32
07/24/2021 06:44:26 - INFO - trainer -   ***** Eval results *****
07/24/2021 06:44:26 - INFO - trainer -     eval loss: 1.0580049377017922
07/24/2021 06:44:26 - INFO - trainer -     acc = 0.8580357142857142
07/24/2021 06:44:26 - INFO - trainer -     f1 = 0.7865368210941184
07/24/2021 06:44:32 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 06:44:32 - INFO - trainer -     Num examples = 1120
07/24/2021 06:44:32 - INFO - trainer -     Batch size = 32
07/24/2021 06:44:33 - INFO - trainer -   ***** Eval results *****
07/24/2021 06:44:33 - INFO - trainer -     eval loss: 1.0502274840571253
07/24/2021 06:44:33 - INFO - trainer -     acc = 0.8544642857142857
07/24/2021 06:44:33 - INFO - trainer -     f1 = 0.7801613391454869
Epoch:  77%|███████▋  | 23/30 [06:11<01:52, 16.04s/it]07/24/2021 06:44:39 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 06:44:39 - INFO - trainer -     Num examples = 1120
07/24/2021

07/24/2021 06:46:28 - INFO - trainer -     Num examples = 1120
07/24/2021 06:46:28 - INFO - trainer -     Batch size = 32
07/24/2021 06:46:29 - INFO - trainer -   ***** Eval results *****
07/24/2021 06:46:29 - INFO - trainer -     eval loss: 1.0784009541650967
07/24/2021 06:46:29 - INFO - trainer -     acc = 0.8446428571428571
07/24/2021 06:46:29 - INFO - trainer -     f1 = 0.7703721695842575
Epoch: 100%|██████████| 30/30 [08:06<00:00, 16.23s/it]
07/24/2021 06:46:34 - INFO - __main__ -     acc = 0.8633928571428572
07/24/2021 06:46:34 - INFO - __main__ -     f1 = 0.8001387202068994


NameError: name 'best_acc_eval_results_by_layer' is not defined

In [5]:
best_acc_eval_result

{'acc': 0.8633928571428572, 'f1': 0.8001387202068994}

In [6]:
best_f1_eval_result

{'acc': 0.8633928571428572, 'f1': 0.8001387202068994}

### Output of Bert, output[12] Frozen 0.5

In [32]:
args.hidden_layer_no = 12
args.frozen_percent = 0.5
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:50:49 - INFO - model -   Hidden Layer 12 frozen percentage 0.5
07/23/2021 21:50:49 - INFO - trainer -   ***** Running training *****
07/23/2021 21:50:49 - INFO - trainer -     Num examples = 3602
07/23/2021 21:50:49 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:50:49 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:50:49 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:50:49 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:50:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:50:54 - INFO - trainer -     Num examples = 1120
07/23/2021 21:50:54 - INFO - trainer -     Batch size = 32
07/23/2021 21:50:55 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:50:55 - INFO - trainer -     eval loss: 0.6034779544387545
07/23/2021 21:50:55 - INFO - trainer -     acc = 0.7723214285714286
07/23/2021 21:50:55 - INFO - trainer -     f1 = 0.5679

07/23/2021 21:52:26 - INFO - trainer -     acc = 0.8553571428571428
07/23/2021 21:52:26 - INFO - trainer -     f1 = 0.7807095457637937
07/23/2021 21:52:31 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:52:31 - INFO - trainer -     Num examples = 1120
07/23/2021 21:52:31 - INFO - trainer -     Batch size = 32
07/23/2021 21:52:32 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:52:32 - INFO - trainer -     eval loss: 0.7380651230790786
07/23/2021 21:52:32 - INFO - trainer -     acc = 0.8446428571428571
07/23/2021 21:52:32 - INFO - trainer -     f1 = 0.7743843214761094
07/23/2021 21:52:37 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:52:37 - INFO - trainer -     Num examples = 1120
07/23/2021 21:52:37 - INFO - trainer -     Batch size = 32
07/23/2021 21:52:38 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:52:38 - INFO - trainer -     eval loss: 0.7917135020185794
07/23/2021 21:52:38 - INFO - trainer -     acc = 0.8455357142857

07/23/2021 21:54:08 - INFO - trainer -     eval loss: 1.1416109993567392
07/23/2021 21:54:08 - INFO - trainer -     acc = 0.8366071428571429
07/23/2021 21:54:08 - INFO - trainer -     f1 = 0.7432919212288289
Epoch:  50%|█████     | 15/30 [03:22<03:19, 13.28s/it]07/23/2021 21:54:12 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:54:12 - INFO - trainer -     Num examples = 1120
07/23/2021 21:54:12 - INFO - trainer -     Batch size = 32
07/23/2021 21:54:13 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:54:13 - INFO - trainer -     eval loss: 1.1145888219358833
07/23/2021 21:54:13 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 21:54:13 - INFO - trainer -     f1 = 0.7502830854067538
07/23/2021 21:54:18 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:54:18 - INFO - trainer -     Num examples = 1120
07/23/2021 21:54:18 - INFO - trainer -     Batch size = 32
07/23/2021 21:54:19 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 21:55:47 - INFO - trainer -     Batch size = 32
07/23/2021 21:55:48 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:55:48 - INFO - trainer -     eval loss: 1.284543668571028
07/23/2021 21:55:48 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 21:55:48 - INFO - trainer -     f1 = 0.7522419266091473
07/23/2021 21:55:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:55:53 - INFO - trainer -     Num examples = 1120
07/23/2021 21:55:53 - INFO - trainer -     Batch size = 32
07/23/2021 21:55:54 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:55:54 - INFO - trainer -     eval loss: 1.2800051386733164
07/23/2021 21:55:54 - INFO - trainer -     acc = 0.8508928571428571
07/23/2021 21:55:54 - INFO - trainer -     f1 = 0.7678786693006749
Epoch:  77%|███████▋  | 23/30 [05:09<01:32, 13.21s/it]07/23/2021 21:55:59 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:55:59 - INFO - trainer -     Num examples = 1120
07/23/2021 

07/23/2021 21:57:28 - INFO - trainer -     Num examples = 1120
07/23/2021 21:57:28 - INFO - trainer -     Batch size = 32
07/23/2021 21:57:29 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:57:29 - INFO - trainer -     eval loss: 1.1606569862943226
07/23/2021 21:57:29 - INFO - trainer -     acc = 0.8446428571428571
07/23/2021 21:57:29 - INFO - trainer -     f1 = 0.7698761589074562
Epoch: 100%|██████████| 30/30 [06:43<00:00, 13.46s/it]
07/23/2021 21:57:33 - INFO - __main__ -     acc = 0.8553571428571428
07/23/2021 21:57:33 - INFO - __main__ -     f1 = 0.7807095457637937


### Output of Bert, output[12] Frozen 0.75

In [11]:
args.hidden_layer_no = 12
args.frozen_percent = 0.75
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 03:25:59 - INFO - model -   Hidden Layer 12 frozen percentage 0.75
07/23/2021 03:25:59 - INFO - trainer -   ***** Running training *****
07/23/2021 03:25:59 - INFO - trainer -     Num examples = 3602
07/23/2021 03:25:59 - INFO - trainer -     Num Epochs = 30
07/23/2021 03:25:59 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 03:25:59 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 03:25:59 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 03:26:02 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:26:02 - INFO - trainer -     Num examples = 1120
07/23/2021 03:26:02 - INFO - trainer -     Batch size = 32
07/23/2021 03:26:03 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:26:03 - INFO - trainer -     eval loss: 0.7379045597144536
07/23/2021 03:26:03 - INFO - trainer -     acc = 0.7133928571428572
07/23/2021 03:26:03 - INFO - trainer -     f1 = 0.440

07/23/2021 03:27:13 - INFO - trainer -     acc = 0.84375
07/23/2021 03:27:13 - INFO - trainer -     f1 = 0.7608707862767581
07/23/2021 03:27:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:27:17 - INFO - trainer -     Num examples = 1120
07/23/2021 03:27:17 - INFO - trainer -     Batch size = 32
07/23/2021 03:27:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:27:18 - INFO - trainer -     eval loss: 0.5863856578245759
07/23/2021 03:27:18 - INFO - trainer -     acc = 0.8464285714285714
07/23/2021 03:27:18 - INFO - trainer -     f1 = 0.7727342910724446
07/23/2021 03:27:21 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:27:21 - INFO - trainer -     Num examples = 1120
07/23/2021 03:27:21 - INFO - trainer -     Batch size = 32
07/23/2021 03:27:23 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:27:23 - INFO - trainer -     eval loss: 0.5644378356635571
07/23/2021 03:27:23 - INFO - trainer -     acc = 0.8455357142857143
07/23/2

07/23/2021 03:28:32 - INFO - trainer -     eval loss: 0.868643713336704
07/23/2021 03:28:32 - INFO - trainer -     acc = 0.8401785714285714
07/23/2021 03:28:32 - INFO - trainer -     f1 = 0.7577496152989128
Epoch:  50%|█████     | 15/30 [02:36<02:35, 10.35s/it]07/23/2021 03:28:36 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:28:36 - INFO - trainer -     Num examples = 1120
07/23/2021 03:28:36 - INFO - trainer -     Batch size = 32
07/23/2021 03:28:37 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:28:37 - INFO - trainer -     eval loss: 0.8478353490055139
07/23/2021 03:28:37 - INFO - trainer -     acc = 0.8455357142857143
07/23/2021 03:28:37 - INFO - trainer -     f1 = 0.7668853511362906
07/23/2021 03:28:40 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:28:40 - INFO - trainer -     Num examples = 1120
07/23/2021 03:28:40 - INFO - trainer -     Batch size = 32
07/23/2021 03:28:42 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 03:29:50 - INFO - trainer -     Batch size = 32
07/23/2021 03:29:51 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:29:51 - INFO - trainer -     eval loss: 0.9041991234291344
07/23/2021 03:29:51 - INFO - trainer -     acc = 0.8473214285714286
07/23/2021 03:29:51 - INFO - trainer -     f1 = 0.7794765554218372
07/23/2021 03:29:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:29:54 - INFO - trainer -     Num examples = 1120
07/23/2021 03:29:54 - INFO - trainer -     Batch size = 32
07/23/2021 03:29:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:29:56 - INFO - trainer -     eval loss: 1.0004015458747744
07/23/2021 03:29:56 - INFO - trainer -     acc = 0.8383928571428572
07/23/2021 03:29:56 - INFO - trainer -     f1 = 0.7596352277118966
Epoch:  77%|███████▋  | 23/30 [04:00<01:12, 10.32s/it]07/23/2021 03:29:59 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:29:59 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 03:31:08 - INFO - trainer -     Num examples = 1120
07/23/2021 03:31:08 - INFO - trainer -     Batch size = 32
07/23/2021 03:31:09 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:31:09 - INFO - trainer -     eval loss: 1.2338148049411497
07/23/2021 03:31:09 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 03:31:09 - INFO - trainer -     f1 = 0.7490991675969849
Epoch: 100%|██████████| 30/30 [05:13<00:00, 10.44s/it]
07/23/2021 03:31:12 - INFO - __main__ -     acc = 0.8544642857142857
07/23/2021 03:31:12 - INFO - __main__ -     f1 = 0.7955473836394455


In [12]:
best_acc_eval_results_by_layer

{(12, 0): {'acc': 0.8580357142857142, 'f1': 0.7823453075568864},
 (12, 0.25): {'acc': 0.8625, 'f1': 0.7960469155284686},
 (12, 0.75): {'acc': 0.8544642857142857, 'f1': 0.7955473836394455}}

In [13]:
best_f1_eval_results_by_layer

{(12, 0): {'acc': 0.8580357142857142, 'f1': 0.794648953589351},
 (12, 0.25): {'acc': 0.8616071428571429, 'f1': 0.798711316849118},
 (12, 0.75): {'acc': 0.8544642857142857, 'f1': 0.7955473836394455}}

### Output of Bert, output[12] Frozen 0.9

In [31]:
args.hidden_layer_no = 12
args.frozen_percent = 0.9
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:44:29 - INFO - model -   Hidden Layer 12 frozen percentage 0.9
07/23/2021 21:44:29 - INFO - trainer -   ***** Running training *****
07/23/2021 21:44:29 - INFO - trainer -     Num examples = 3602
07/23/2021 21:44:29 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:44:29 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:44:29 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:44:29 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:44:32 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:44:32 - INFO - trainer -     Num examples = 1120
07/23/2021 21:44:32 - INFO - trainer -     Batch size = 32
07/23/2021 21:44:33 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:44:33 - INFO - trainer -     eval loss: 0.8925355298178537
07/23/2021 21:44:33 - INFO - trainer -     acc = 0.6517857142857143
07/23/2021 21:44:33 - INFO - trainer -     f1 = 0.2697

07/23/2021 21:45:33 - INFO - trainer -     acc = 0.8196428571428571
07/23/2021 21:45:33 - INFO - trainer -     f1 = 0.7252133019087599
07/23/2021 21:45:36 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:45:36 - INFO - trainer -     Num examples = 1120
07/23/2021 21:45:36 - INFO - trainer -     Batch size = 32
07/23/2021 21:45:37 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:45:37 - INFO - trainer -     eval loss: 0.49076588430574963
07/23/2021 21:45:37 - INFO - trainer -     acc = 0.8214285714285714
07/23/2021 21:45:37 - INFO - trainer -     f1 = 0.7320889692680764
07/23/2021 21:45:40 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:45:40 - INFO - trainer -     Num examples = 1120
07/23/2021 21:45:40 - INFO - trainer -     Batch size = 32
07/23/2021 21:45:41 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:45:41 - INFO - trainer -     eval loss: 0.4912434143679483
07/23/2021 21:45:41 - INFO - trainer -     acc = 0.823214285714

07/23/2021 21:46:41 - INFO - trainer -     eval loss: 0.5721568103347506
07/23/2021 21:46:41 - INFO - trainer -     acc = 0.8294642857142858
07/23/2021 21:46:41 - INFO - trainer -     f1 = 0.7482478097373416
Epoch:  50%|█████     | 15/30 [02:15<02:14,  8.95s/it]07/23/2021 21:46:44 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:46:44 - INFO - trainer -     Num examples = 1120
07/23/2021 21:46:44 - INFO - trainer -     Batch size = 32
07/23/2021 21:46:45 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:46:45 - INFO - trainer -     eval loss: 0.5646225491272552
07/23/2021 21:46:45 - INFO - trainer -     acc = 0.8285714285714286
07/23/2021 21:46:45 - INFO - trainer -     f1 = 0.7499838947163641
07/23/2021 21:46:48 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:46:48 - INFO - trainer -     Num examples = 1120
07/23/2021 21:46:48 - INFO - trainer -     Batch size = 32
07/23/2021 21:46:49 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 21:47:49 - INFO - trainer -     Batch size = 32
07/23/2021 21:47:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:47:50 - INFO - trainer -     eval loss: 0.8022272431424686
07/23/2021 21:47:50 - INFO - trainer -     acc = 0.8151785714285714
07/23/2021 21:47:50 - INFO - trainer -     f1 = 0.7240806097482345
07/23/2021 21:47:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:47:53 - INFO - trainer -     Num examples = 1120
07/23/2021 21:47:53 - INFO - trainer -     Batch size = 32
07/23/2021 21:47:54 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:47:54 - INFO - trainer -     eval loss: 0.7403491188905069
07/23/2021 21:47:54 - INFO - trainer -     acc = 0.8285714285714286
07/23/2021 21:47:54 - INFO - trainer -     f1 = 0.7414295842844297
Epoch:  77%|███████▋  | 23/30 [03:28<01:02,  8.97s/it]07/23/2021 21:47:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:47:57 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 21:48:57 - INFO - trainer -     Num examples = 1120
07/23/2021 21:48:57 - INFO - trainer -     Batch size = 32
07/23/2021 21:48:59 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:48:59 - INFO - trainer -     eval loss: 0.9016576081514358
07/23/2021 21:48:59 - INFO - trainer -     acc = 0.81875
07/23/2021 21:48:59 - INFO - trainer -     f1 = 0.731313695379031
Epoch: 100%|██████████| 30/30 [04:32<00:00,  9.07s/it]
07/23/2021 21:49:01 - INFO - __main__ -     acc = 0.8375
07/23/2021 21:49:01 - INFO - __main__ -     f1 = 0.7580973192536766


### Output of Bert, output[11] All Frozen

In [7]:
args.hidden_layer_no = 11
args.frozen_percent = 1
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[args.hidden_layer_no] = best_acc_eval_result
    best_f1_eval_results_by_layer[args.hidden_layer_no] = best_f1_eval_result
    
    
        

07/23/2021 02:04:54 - INFO - model -   Hidden Layer 11
07/23/2021 02:04:55 - INFO - trainer -   ***** Running training *****
07/23/2021 02:04:55 - INFO - trainer -     Num examples = 3602
07/23/2021 02:04:55 - INFO - trainer -     Num Epochs = 30
07/23/2021 02:04:55 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 02:04:55 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 02:04:55 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 02:05:14 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:05:14 - INFO - trainer -     Num examples = 1120
07/23/2021 02:05:14 - INFO - trainer -     Batch size = 32
07/23/2021 02:05:16 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:05:16 - INFO - trainer -     eval loss: 0.8898086428642273
07/23/2021 02:05:16 - INFO - trainer -     acc = 0.65
07/23/2021 02:05:16 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 02:05:18 - 

07/23/2021 02:06:08 - INFO - trainer -     f1 = 0.3486217611429545
07/23/2021 02:06:10 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:06:10 - INFO - trainer -     Num examples = 1120
07/23/2021 02:06:10 - INFO - trainer -     Batch size = 32
07/23/2021 02:06:11 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:06:11 - INFO - trainer -     eval loss: 0.7764740066868918
07/23/2021 02:06:11 - INFO - trainer -     acc = 0.6705357142857142
07/23/2021 02:06:11 - INFO - trainer -     f1 = 0.36833616112009854
07/23/2021 02:06:13 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:06:13 - INFO - trainer -     Num examples = 1120
07/23/2021 02:06:13 - INFO - trainer -     Batch size = 32
07/23/2021 02:06:15 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:06:15 - INFO - trainer -     eval loss: 0.774716945205416
07/23/2021 02:06:15 - INFO - trainer -     acc = 0.6723214285714286
07/23/2021 02:06:15 - INFO - trainer -     f1 = 0.36864012338037

07/23/2021 02:07:07 - INFO - trainer -     acc = 0.6991071428571428
07/23/2021 02:07:07 - INFO - trainer -     f1 = 0.4478531441800257
Epoch:  50%|█████     | 15/30 [02:14<01:57,  7.84s/it]07/23/2021 02:07:09 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:07:09 - INFO - trainer -     Num examples = 1120
07/23/2021 02:07:09 - INFO - trainer -     Batch size = 32
07/23/2021 02:07:10 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:07:10 - INFO - trainer -     eval loss: 0.7287754723003932
07/23/2021 02:07:10 - INFO - trainer -     acc = 0.6955357142857143
07/23/2021 02:07:10 - INFO - trainer -     f1 = 0.43555824301414664
07/23/2021 02:07:13 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:07:13 - INFO - trainer -     Num examples = 1120
07/23/2021 02:07:13 - INFO - trainer -     Batch size = 32
07/23/2021 02:07:14 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:07:14 - INFO - trainer -     eval loss: 0.7314496972731181
07/23/202

07/23/2021 02:08:06 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:08:06 - INFO - trainer -     eval loss: 0.6949426472187042
07/23/2021 02:08:06 - INFO - trainer -     acc = 0.7214285714285714
07/23/2021 02:08:06 - INFO - trainer -     f1 = 0.5206787255820795
07/23/2021 02:08:08 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:08:08 - INFO - trainer -     Num examples = 1120
07/23/2021 02:08:08 - INFO - trainer -     Batch size = 32
07/23/2021 02:08:10 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:08:10 - INFO - trainer -     eval loss: 0.7023780235222408
07/23/2021 02:08:10 - INFO - trainer -     acc = 0.7053571428571429
07/23/2021 02:08:10 - INFO - trainer -     f1 = 0.46748953947588423
Epoch:  77%|███████▋  | 23/30 [03:17<00:54,  7.77s/it]07/23/2021 02:08:12 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:08:12 - INFO - trainer -     Num examples = 1120
07/23/2021 02:08:12 - INFO - trainer -     Batch size = 32
07/23/202

07/23/2021 02:09:05 - INFO - trainer -     Batch size = 32
07/23/2021 02:09:06 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:09:06 - INFO - trainer -     eval loss: 0.6873542359897069
07/23/2021 02:09:06 - INFO - trainer -     acc = 0.7223214285714286
07/23/2021 02:09:06 - INFO - trainer -     f1 = 0.5154186100859396
Epoch: 100%|██████████| 30/30 [04:12<00:00,  8.43s/it]


NameError: name 'best_eval_result' is not defined

In [10]:
if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[args.hidden_layer_no] = best_acc_eval_result
    best_f1_eval_results_by_layer[args.hidden_layer_no] = best_f1_eval_result

07/23/2021 02:11:47 - INFO - __main__ -     acc = 0.7285714285714285
07/23/2021 02:11:47 - INFO - __main__ -     f1 = 0.5414583304317144


In [16]:
best_f1_eval_results_by_layer[11]

{'acc': 0.7285714285714285, 'f1': 0.5414583304317144}

### Output of Bert, output[11] Frozen 0

In [7]:
args.hidden_layer_no = 11
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 19:45:43 - INFO - model -   Hidden Layer 11 frozen percentage 0
07/23/2021 19:45:48 - INFO - trainer -   ***** Running training *****
07/23/2021 19:45:48 - INFO - trainer -     Num examples = 3602
07/23/2021 19:45:48 - INFO - trainer -     Num Epochs = 30
07/23/2021 19:45:48 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 19:45:48 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 19:45:48 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 19:45:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:45:57 - INFO - trainer -     Num examples = 1120
07/23/2021 19:45:57 - INFO - trainer -     Batch size = 32
07/23/2021 19:45:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:45:58 - INFO - trainer -     eval loss: 0.6670303591660091
07/23/2021 19:45:58 - INFO - trainer -     acc = 0.7446428571428572
07/23/2021 19:45:58 - INFO - trainer -     f1 = 0.569091

07/23/2021 19:48:06 - INFO - trainer -     acc = 0.8428571428571429
07/23/2021 19:48:06 - INFO - trainer -     f1 = 0.7614536764901411
07/23/2021 19:48:14 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:48:14 - INFO - trainer -     Num examples = 1120
07/23/2021 19:48:14 - INFO - trainer -     Batch size = 32
07/23/2021 19:48:15 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:48:15 - INFO - trainer -     eval loss: 0.7576428955154759
07/23/2021 19:48:15 - INFO - trainer -     acc = 0.8419642857142857
07/23/2021 19:48:15 - INFO - trainer -     f1 = 0.7712913187104421
07/23/2021 19:48:22 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:48:22 - INFO - trainer -     Num examples = 1120
07/23/2021 19:48:22 - INFO - trainer -     Batch size = 32
07/23/2021 19:48:23 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:48:23 - INFO - trainer -     eval loss: 0.8280671452571239
07/23/2021 19:48:23 - INFO - trainer -     acc = 0.8464285714285

07/23/2021 19:50:30 - INFO - trainer -     acc = 0.8392857142857143
07/23/2021 19:50:30 - INFO - trainer -     f1 = 0.771534570497844
Epoch:  50%|█████     | 15/30 [04:48<04:42, 18.86s/it]07/23/2021 19:50:37 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:50:37 - INFO - trainer -     Num examples = 1120
07/23/2021 19:50:37 - INFO - trainer -     Batch size = 32
07/23/2021 19:50:38 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:50:39 - INFO - trainer -     eval loss: 1.0258610266526895
07/23/2021 19:50:39 - INFO - trainer -     acc = 0.8535714285714285
07/23/2021 19:50:39 - INFO - trainer -     f1 = 0.7844456286365387
07/23/2021 19:50:46 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:50:46 - INFO - trainer -     Num examples = 1120
07/23/2021 19:50:46 - INFO - trainer -     Batch size = 32
07/23/2021 19:50:47 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:50:47 - INFO - trainer -     eval loss: 1.1033744797759157
07/23/2021 

07/23/2021 19:52:52 - INFO - trainer -     eval loss: 1.1764049646411356
07/23/2021 19:52:52 - INFO - trainer -     acc = 0.8419642857142857
07/23/2021 19:52:53 - INFO - trainer -     f1 = 0.7676595326778831
07/23/2021 19:53:00 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:53:00 - INFO - trainer -     Num examples = 1120
07/23/2021 19:53:00 - INFO - trainer -     Batch size = 32
07/23/2021 19:53:01 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:53:01 - INFO - trainer -     eval loss: 1.2921959344829832
07/23/2021 19:53:01 - INFO - trainer -     acc = 0.8267857142857142
07/23/2021 19:53:01 - INFO - trainer -     f1 = 0.7467929817001023
Epoch:  77%|███████▋  | 23/30 [07:19<02:11, 18.72s/it]07/23/2021 19:53:08 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:53:08 - INFO - trainer -     Num examples = 1120
07/23/2021 19:53:08 - INFO - trainer -     Batch size = 32
07/23/2021 19:53:09 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 19:55:14 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:55:14 - INFO - trainer -     eval loss: 1.2867140723192798
07/23/2021 19:55:14 - INFO - trainer -     acc = 0.83125
07/23/2021 19:55:14 - INFO - trainer -     f1 = 0.7477192801190586
Epoch: 100%|██████████| 30/30 [09:31<00:00, 19.06s/it]
07/23/2021 19:55:20 - INFO - __main__ -     acc = 0.8625
07/23/2021 19:55:20 - INFO - __main__ -     f1 = 0.7983648942079483


### Output of Bert, output[10] Frozen 0

In [10]:
args.hidden_layer_no = 10
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 19:57:17 - INFO - model -   Hidden Layer 10 frozen percentage 0
07/23/2021 19:57:17 - INFO - trainer -   ***** Running training *****
07/23/2021 19:57:17 - INFO - trainer -     Num examples = 3602
07/23/2021 19:57:17 - INFO - trainer -     Num Epochs = 30
07/23/2021 19:57:17 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 19:57:17 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 19:57:17 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 19:57:24 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:57:24 - INFO - trainer -     Num examples = 1120
07/23/2021 19:57:24 - INFO - trainer -     Batch size = 32
07/23/2021 19:57:25 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:57:25 - INFO - trainer -     eval loss: 0.6171426385641098
07/23/2021 19:57:25 - INFO - trainer -     acc = 0.7598214285714285
07/23/2021 19:57:25 - INFO - trainer -     f1 = 0.544099

07/23/2021 19:59:26 - INFO - trainer -     acc = 0.84375
07/23/2021 19:59:26 - INFO - trainer -     f1 = 0.7674824219218004
07/23/2021 19:59:33 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:59:33 - INFO - trainer -     Num examples = 1120
07/23/2021 19:59:33 - INFO - trainer -     Batch size = 32
07/23/2021 19:59:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:59:34 - INFO - trainer -     eval loss: 0.9029278035940869
07/23/2021 19:59:34 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 19:59:34 - INFO - trainer -     f1 = 0.746209799056099
07/23/2021 19:59:41 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:59:41 - INFO - trainer -     Num examples = 1120
07/23/2021 19:59:41 - INFO - trainer -     Batch size = 32
07/23/2021 19:59:42 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:59:42 - INFO - trainer -     eval loss: 0.8427069824188947
07/23/2021 19:59:42 - INFO - trainer -     acc = 0.8258928571428571
07/23/20

07/23/2021 20:01:42 - INFO - trainer -     eval loss: 1.0574512737744954
07/23/2021 20:01:42 - INFO - trainer -     acc = 0.8473214285714286
07/23/2021 20:01:42 - INFO - trainer -     f1 = 0.7618338693502092
Epoch:  50%|█████     | 15/30 [04:31<04:27, 17.86s/it]07/23/2021 20:01:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:01:49 - INFO - trainer -     Num examples = 1120
07/23/2021 20:01:49 - INFO - trainer -     Batch size = 32
07/23/2021 20:01:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:01:50 - INFO - trainer -     eval loss: 1.127197695094011
07/23/2021 20:01:50 - INFO - trainer -     acc = 0.8392857142857143
07/23/2021 20:01:50 - INFO - trainer -     f1 = 0.7488800030985567
07/23/2021 20:01:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:01:57 - INFO - trainer -     Num examples = 1120
07/23/2021 20:01:57 - INFO - trainer -     Batch size = 32
07/23/2021 20:01:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 20:03:56 - INFO - trainer -     Num examples = 1120
07/23/2021 20:03:56 - INFO - trainer -     Batch size = 32
07/23/2021 20:03:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:03:58 - INFO - trainer -     eval loss: 1.1964952413125762
07/23/2021 20:03:58 - INFO - trainer -     acc = 0.8214285714285714
07/23/2021 20:03:58 - INFO - trainer -     f1 = 0.7437988925491799
07/23/2021 20:04:04 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:04:04 - INFO - trainer -     Num examples = 1120
07/23/2021 20:04:04 - INFO - trainer -     Batch size = 32
07/23/2021 20:04:06 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:04:06 - INFO - trainer -     eval loss: 1.1488033732889953
07/23/2021 20:04:06 - INFO - trainer -     acc = 0.8410714285714286
07/23/2021 20:04:06 - INFO - trainer -     f1 = 0.7631085945063519
Epoch:  77%|███████▋  | 23/30 [06:55<02:04, 17.83s/it]07/23/2021 20:04:12 - INFO - trainer -   ***** Running evaluation *****
07/23/2021

07/23/2021 20:06:12 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:06:12 - INFO - trainer -     Num examples = 1120
07/23/2021 20:06:12 - INFO - trainer -     Batch size = 32
07/23/2021 20:06:13 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:06:13 - INFO - trainer -     eval loss: 1.147089220837058
07/23/2021 20:06:13 - INFO - trainer -     acc = 0.8419642857142857
07/23/2021 20:06:13 - INFO - trainer -     f1 = 0.7634407069156728
Epoch: 100%|██████████| 30/30 [09:01<00:00, 18.04s/it]
07/23/2021 20:06:18 - INFO - __main__ -     acc = 0.8526785714285714
07/23/2021 20:06:18 - INFO - __main__ -     f1 = 0.7791258718917419


### Output of Bert, output[9] Frozen 0

In [11]:
args.hidden_layer_no = 9
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:08:17 - INFO - model -   Hidden Layer 9 frozen percentage 0
07/23/2021 20:08:18 - INFO - trainer -   ***** Running training *****
07/23/2021 20:08:18 - INFO - trainer -     Num examples = 3602
07/23/2021 20:08:18 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:08:18 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:08:18 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:08:18 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:08:24 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:08:24 - INFO - trainer -     Num examples = 1120
07/23/2021 20:08:24 - INFO - trainer -     Batch size = 32
07/23/2021 20:08:25 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:08:25 - INFO - trainer -     eval loss: 0.7272457633699689
07/23/2021 20:08:25 - INFO - trainer -     acc = 0.7125
07/23/2021 20:08:25 - INFO - trainer -     f1 = 0.47492765327411
07/2

07/23/2021 20:10:23 - INFO - trainer -     acc = 0.8366071428571429
07/23/2021 20:10:23 - INFO - trainer -     f1 = 0.7532209364928567
07/23/2021 20:10:29 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:10:29 - INFO - trainer -     Num examples = 1120
07/23/2021 20:10:29 - INFO - trainer -     Batch size = 32
07/23/2021 20:10:30 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:10:30 - INFO - trainer -     eval loss: 0.9014543068728277
07/23/2021 20:10:30 - INFO - trainer -     acc = 0.8303571428571429
07/23/2021 20:10:30 - INFO - trainer -     f1 = 0.7375290243049806
07/23/2021 20:10:37 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:10:37 - INFO - trainer -     Num examples = 1120
07/23/2021 20:10:37 - INFO - trainer -     Batch size = 32
07/23/2021 20:10:38 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:10:38 - INFO - trainer -     eval loss: 0.8527425945071238
07/23/2021 20:10:38 - INFO - trainer -     acc = 0.8419642857142

07/23/2021 20:12:33 - INFO - trainer -     eval loss: 1.1607451354082774
07/23/2021 20:12:33 - INFO - trainer -     acc = 0.8392857142857143
07/23/2021 20:12:33 - INFO - trainer -     f1 = 0.7568442103281919
Epoch:  50%|█████     | 15/30 [04:21<04:17, 17.17s/it]07/23/2021 20:12:39 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:12:39 - INFO - trainer -     Num examples = 1120
07/23/2021 20:12:39 - INFO - trainer -     Batch size = 32
07/23/2021 20:12:41 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:12:41 - INFO - trainer -     eval loss: 1.1901556309705483
07/23/2021 20:12:41 - INFO - trainer -     acc = 0.8401785714285714
07/23/2021 20:12:41 - INFO - trainer -     f1 = 0.7593674723120128
07/23/2021 20:12:47 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:12:47 - INFO - trainer -     Num examples = 1120
07/23/2021 20:12:47 - INFO - trainer -     Batch size = 32
07/23/2021 20:12:48 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 20:14:42 - INFO - trainer -     Batch size = 32
07/23/2021 20:14:43 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:14:43 - INFO - trainer -     eval loss: 1.2768271582144994
07/23/2021 20:14:43 - INFO - trainer -     acc = 0.825
07/23/2021 20:14:43 - INFO - trainer -     f1 = 0.7358721199815194
07/23/2021 20:14:50 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:14:50 - INFO - trainer -     Num examples = 1120
07/23/2021 20:14:50 - INFO - trainer -     Batch size = 32
07/23/2021 20:14:51 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:14:51 - INFO - trainer -     eval loss: 1.275411521029933
07/23/2021 20:14:51 - INFO - trainer -     acc = 0.8241071428571428
07/23/2021 20:14:51 - INFO - trainer -     f1 = 0.7350663732647691
Epoch:  77%|███████▋  | 23/30 [06:39<02:00, 17.14s/it]07/23/2021 20:14:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:14:57 - INFO - trainer -     Num examples = 1120
07/23/2021 20:14:57 - IN

07/23/2021 20:16:52 - INFO - trainer -     Num examples = 1120
07/23/2021 20:16:52 - INFO - trainer -     Batch size = 32
07/23/2021 20:16:53 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:16:53 - INFO - trainer -     eval loss: 1.3359377365027154
07/23/2021 20:16:53 - INFO - trainer -     acc = 0.8151785714285714
07/23/2021 20:16:53 - INFO - trainer -     f1 = 0.7465585783567249
Epoch: 100%|██████████| 30/30 [08:40<00:00, 17.35s/it]
07/23/2021 20:16:58 - INFO - __main__ -     acc = 0.8482142857142857
07/23/2021 20:16:58 - INFO - __main__ -     f1 = 0.7775968480864099


### Output of Bert, output[8] Frozen 0

In [14]:
args.hidden_layer_no = 8
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:20:50 - INFO - model -   Hidden Layer 8 frozen percentage 0
07/23/2021 20:20:51 - INFO - trainer -   ***** Running training *****
07/23/2021 20:20:51 - INFO - trainer -     Num examples = 3602
07/23/2021 20:20:51 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:20:51 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:20:51 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:20:51 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:20:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:20:57 - INFO - trainer -     Num examples = 1120
07/23/2021 20:20:57 - INFO - trainer -     Batch size = 32
07/23/2021 20:20:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:20:58 - INFO - trainer -     eval loss: 0.8833903431892395
07/23/2021 20:20:58 - INFO - trainer -     acc = 0.6553571428571429
07/23/2021 20:20:58 - INFO - trainer -     f1 = 0.3150021

07/23/2021 20:22:47 - INFO - trainer -     acc = 0.81875
07/23/2021 20:22:47 - INFO - trainer -     f1 = 0.7288510698557097
07/23/2021 20:22:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:22:53 - INFO - trainer -     Num examples = 1120
07/23/2021 20:22:53 - INFO - trainer -     Batch size = 32
07/23/2021 20:22:55 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:22:55 - INFO - trainer -     eval loss: 1.0201144171479557
07/23/2021 20:22:55 - INFO - trainer -     acc = 0.8142857142857143
07/23/2021 20:22:55 - INFO - trainer -     f1 = 0.7235627125006984
07/23/2021 20:23:01 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:23:01 - INFO - trainer -     Num examples = 1120
07/23/2021 20:23:01 - INFO - trainer -     Batch size = 32
07/23/2021 20:23:02 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:23:02 - INFO - trainer -     eval loss: 0.9443445054548127
07/23/2021 20:23:02 - INFO - trainer -     acc = 0.8044642857142857
07/23/2

07/23/2021 20:24:50 - INFO - trainer -     eval loss: 1.3706618103597845
07/23/2021 20:24:50 - INFO - trainer -     acc = 0.8089285714285714
07/23/2021 20:24:50 - INFO - trainer -     f1 = 0.701372544318671
Epoch:  50%|█████     | 15/30 [04:04<04:01, 16.07s/it]07/23/2021 20:24:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:24:56 - INFO - trainer -     Num examples = 1120
07/23/2021 20:24:56 - INFO - trainer -     Batch size = 32
07/23/2021 20:24:57 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:24:57 - INFO - trainer -     eval loss: 1.21262905725411
07/23/2021 20:24:57 - INFO - trainer -     acc = 0.8160714285714286
07/23/2021 20:24:57 - INFO - trainer -     f1 = 0.7218003236683895
07/23/2021 20:25:03 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:25:03 - INFO - trainer -     Num examples = 1120
07/23/2021 20:25:03 - INFO - trainer -     Batch size = 32
07/23/2021 20:25:04 - INFO - trainer -   ***** Eval results *****
07/23/2021 20

07/23/2021 20:26:51 - INFO - trainer -     Batch size = 32
07/23/2021 20:26:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:26:52 - INFO - trainer -     eval loss: 1.4046989509037562
07/23/2021 20:26:52 - INFO - trainer -     acc = 0.8214285714285714
07/23/2021 20:26:52 - INFO - trainer -     f1 = 0.7359142948640968
07/23/2021 20:26:58 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:26:58 - INFO - trainer -     Num examples = 1120
07/23/2021 20:26:58 - INFO - trainer -     Batch size = 32
07/23/2021 20:27:00 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:27:00 - INFO - trainer -     eval loss: 1.4264639983153238
07/23/2021 20:27:00 - INFO - trainer -     acc = 0.8223214285714285
07/23/2021 20:27:00 - INFO - trainer -     f1 = 0.7335601920703959
Epoch:  77%|███████▋  | 23/30 [06:14<01:52, 16.10s/it]07/23/2021 20:27:06 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:27:06 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 20:28:53 - INFO - trainer -     Num examples = 1120
07/23/2021 20:28:53 - INFO - trainer -     Batch size = 32
07/23/2021 20:28:54 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:28:54 - INFO - trainer -     eval loss: 1.3370523265355068
07/23/2021 20:28:54 - INFO - trainer -     acc = 0.8232142857142857
07/23/2021 20:28:54 - INFO - trainer -     f1 = 0.73622635274016
Epoch: 100%|██████████| 30/30 [08:08<00:00, 16.28s/it]
07/23/2021 20:28:59 - INFO - __main__ -     acc = 0.8383928571428572
07/23/2021 20:28:59 - INFO - __main__ -     f1 = 0.7648445547046677


### Output of Bert, output[7] Frozen 0

In [18]:
args.hidden_layer_no = 7
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:39:23 - INFO - model -   Hidden Layer 7 frozen percentage 0
07/23/2021 20:39:23 - INFO - trainer -   ***** Running training *****
07/23/2021 20:39:23 - INFO - trainer -     Num examples = 3602
07/23/2021 20:39:23 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:39:23 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:39:23 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:39:23 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:39:29 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:39:29 - INFO - trainer -     Num examples = 1120
07/23/2021 20:39:29 - INFO - trainer -     Batch size = 32
07/23/2021 20:39:30 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:39:30 - INFO - trainer -     eval loss: 0.8479713116373334
07/23/2021 20:39:30 - INFO - trainer -     acc = 0.6660714285714285
07/23/2021 20:39:30 - INFO - trainer -     f1 = 0.3602930

07/23/2021 20:41:13 - INFO - trainer -     acc = 0.7982142857142858
07/23/2021 20:41:13 - INFO - trainer -     f1 = 0.7077369342335583
07/23/2021 20:41:19 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:41:19 - INFO - trainer -     Num examples = 1120
07/23/2021 20:41:19 - INFO - trainer -     Batch size = 32
07/23/2021 20:41:20 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:41:20 - INFO - trainer -     eval loss: 0.9574142373033933
07/23/2021 20:41:20 - INFO - trainer -     acc = 0.8
07/23/2021 20:41:20 - INFO - trainer -     f1 = 0.706666990387833
07/23/2021 20:41:26 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:41:26 - INFO - trainer -     Num examples = 1120
07/23/2021 20:41:26 - INFO - trainer -     Batch size = 32
07/23/2021 20:41:27 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:41:27 - INFO - trainer -     eval loss: 0.8444215386041574
07/23/2021 20:41:27 - INFO - trainer -     acc = 0.8053571428571429
07/23/2021 2

07/23/2021 20:43:09 - INFO - trainer -     eval loss: 1.3268335704824754
07/23/2021 20:43:09 - INFO - trainer -     acc = 0.7991071428571429
07/23/2021 20:43:09 - INFO - trainer -     f1 = 0.6961996468548409
Epoch:  50%|█████     | 15/30 [03:50<03:47, 15.17s/it]07/23/2021 20:43:15 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:43:15 - INFO - trainer -     Num examples = 1120
07/23/2021 20:43:15 - INFO - trainer -     Batch size = 32
07/23/2021 20:43:16 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:43:16 - INFO - trainer -     eval loss: 1.3430471451953054
07/23/2021 20:43:16 - INFO - trainer -     acc = 0.8151785714285714
07/23/2021 20:43:16 - INFO - trainer -     f1 = 0.7189178440843671
07/23/2021 20:43:21 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:43:21 - INFO - trainer -     Num examples = 1120
07/23/2021 20:43:21 - INFO - trainer -     Batch size = 32
07/23/2021 20:43:23 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 20:45:02 - INFO - trainer -     Batch size = 32
07/23/2021 20:45:04 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:45:04 - INFO - trainer -     eval loss: 1.5809124173746179
07/23/2021 20:45:04 - INFO - trainer -     acc = 0.8026785714285715
07/23/2021 20:45:04 - INFO - trainer -     f1 = 0.6846510818117539
07/23/2021 20:45:09 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:45:09 - INFO - trainer -     Num examples = 1120
07/23/2021 20:45:09 - INFO - trainer -     Batch size = 32
07/23/2021 20:45:10 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:45:10 - INFO - trainer -     eval loss: 1.5788362311008053
07/23/2021 20:45:10 - INFO - trainer -     acc = 0.8026785714285715
07/23/2021 20:45:10 - INFO - trainer -     f1 = 0.6867663063257413
Epoch:  77%|███████▋  | 23/30 [05:52<01:45, 15.09s/it]07/23/2021 20:45:16 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:45:16 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 20:46:57 - INFO - trainer -     Num examples = 1120
07/23/2021 20:46:57 - INFO - trainer -     Batch size = 32
07/23/2021 20:46:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:46:58 - INFO - trainer -     eval loss: 1.6074872690818405
07/23/2021 20:46:58 - INFO - trainer -     acc = 0.80625
07/23/2021 20:46:58 - INFO - trainer -     f1 = 0.7083510035240099
Epoch: 100%|██████████| 30/30 [07:39<00:00, 15.31s/it]
07/23/2021 20:47:03 - INFO - __main__ -     acc = 0.8258928571428571
07/23/2021 20:47:03 - INFO - __main__ -     f1 = 0.7403418274383831


### Output of Bert, output[6] Frozen 0

In [20]:
args.hidden_layer_no = 6
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:49:21 - INFO - model -   Hidden Layer 6 frozen percentage 0
07/23/2021 20:49:21 - INFO - trainer -   ***** Running training *****
07/23/2021 20:49:21 - INFO - trainer -     Num examples = 3602
07/23/2021 20:49:21 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:49:21 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:49:21 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:49:21 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:49:26 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:49:26 - INFO - trainer -     Num examples = 1120
07/23/2021 20:49:26 - INFO - trainer -     Batch size = 32
07/23/2021 20:49:27 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:49:27 - INFO - trainer -     eval loss: 0.8704926252365113
07/23/2021 20:49:27 - INFO - trainer -     acc = 0.6553571428571429
07/23/2021 20:49:27 - INFO - trainer -     f1 = 0.2921050

07/23/2021 20:51:05 - INFO - trainer -     acc = 0.7910714285714285
07/23/2021 20:51:05 - INFO - trainer -     f1 = 0.6843914261807607
07/23/2021 20:51:10 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:51:10 - INFO - trainer -     Num examples = 1120
07/23/2021 20:51:10 - INFO - trainer -     Batch size = 32
07/23/2021 20:51:11 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:51:11 - INFO - trainer -     eval loss: 0.9534122432981219
07/23/2021 20:51:11 - INFO - trainer -     acc = 0.7794642857142857
07/23/2021 20:51:11 - INFO - trainer -     f1 = 0.6866759790027444
07/23/2021 20:51:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:51:17 - INFO - trainer -     Num examples = 1120
07/23/2021 20:51:17 - INFO - trainer -     Batch size = 32
07/23/2021 20:51:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:51:18 - INFO - trainer -     eval loss: 0.8575881902660643
07/23/2021 20:51:18 - INFO - trainer -     acc = 0.7883928571428

07/23/2021 20:52:54 - INFO - trainer -     eval loss: 1.4575537345239094
07/23/2021 20:52:54 - INFO - trainer -     acc = 0.7866071428571428
07/23/2021 20:52:54 - INFO - trainer -     f1 = 0.6938291755193164
Epoch:  50%|█████     | 15/30 [03:37<03:33, 14.26s/it]07/23/2021 20:52:59 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:52:59 - INFO - trainer -     Num examples = 1120
07/23/2021 20:52:59 - INFO - trainer -     Batch size = 32
07/23/2021 20:53:00 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:53:00 - INFO - trainer -     eval loss: 1.506428984138516
07/23/2021 20:53:00 - INFO - trainer -     acc = 0.7857142857142857
07/23/2021 20:53:00 - INFO - trainer -     f1 = 0.6765273130267606
07/23/2021 20:53:05 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:53:05 - INFO - trainer -     Num examples = 1120
07/23/2021 20:53:05 - INFO - trainer -     Batch size = 32
07/23/2021 20:53:06 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 20:54:42 - INFO - trainer -     Batch size = 32
07/23/2021 20:54:43 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:54:43 - INFO - trainer -     eval loss: 1.7340395251688148
07/23/2021 20:54:43 - INFO - trainer -     acc = 0.7866071428571428
07/23/2021 20:54:43 - INFO - trainer -     f1 = 0.6638747258283337
07/23/2021 20:54:48 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:54:48 - INFO - trainer -     Num examples = 1120
07/23/2021 20:54:48 - INFO - trainer -     Batch size = 32
07/23/2021 20:54:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:54:49 - INFO - trainer -     eval loss: 1.5915360525250435
07/23/2021 20:54:49 - INFO - trainer -     acc = 0.7955357142857142
07/23/2021 20:54:49 - INFO - trainer -     f1 = 0.6985318888553168
Epoch:  77%|███████▋  | 23/30 [05:33<01:40, 14.34s/it]07/23/2021 20:54:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:54:54 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 20:56:32 - INFO - trainer -     Num examples = 1120
07/23/2021 20:56:32 - INFO - trainer -     Batch size = 32
07/23/2021 20:56:33 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:56:33 - INFO - trainer -     eval loss: 1.9711945048400334
07/23/2021 20:56:33 - INFO - trainer -     acc = 0.7830357142857143
07/23/2021 20:56:33 - INFO - trainer -     f1 = 0.6547557586860464
Epoch: 100%|██████████| 30/30 [07:15<00:00, 14.53s/it]
07/23/2021 20:56:37 - INFO - __main__ -     acc = 0.8089285714285714
07/23/2021 20:56:37 - INFO - __main__ -     f1 = 0.7116461710857843


### Output of Bert, output[5] Frozen 0

In [23]:
args.hidden_layer_no = 5
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:58:11 - INFO - model -   Hidden Layer 5 frozen percentage 0
07/23/2021 20:58:11 - INFO - trainer -   ***** Running training *****
07/23/2021 20:58:11 - INFO - trainer -     Num examples = 3602
07/23/2021 20:58:11 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:58:11 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:58:11 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:58:11 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:58:16 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:58:16 - INFO - trainer -     Num examples = 1120
07/23/2021 20:58:16 - INFO - trainer -     Batch size = 32
07/23/2021 20:58:17 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:58:17 - INFO - trainer -     eval loss: 0.8661828875541687
07/23/2021 20:58:17 - INFO - trainer -     acc = 0.6508928571428572
07/23/2021 20:58:17 - INFO - trainer -     f1 = 0.2725655

07/23/2021 20:59:50 - INFO - trainer -     acc = 0.7741071428571429
07/23/2021 20:59:50 - INFO - trainer -     f1 = 0.6589658896407363
07/23/2021 20:59:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:59:55 - INFO - trainer -     Num examples = 1120
07/23/2021 20:59:55 - INFO - trainer -     Batch size = 32
07/23/2021 20:59:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:59:56 - INFO - trainer -     eval loss: 0.9493770616395133
07/23/2021 20:59:56 - INFO - trainer -     acc = 0.7553571428571428
07/23/2021 20:59:56 - INFO - trainer -     f1 = 0.6577875587693099
07/23/2021 21:00:01 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:00:01 - INFO - trainer -     Num examples = 1120
07/23/2021 21:00:01 - INFO - trainer -     Batch size = 32
07/23/2021 21:00:02 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:00:02 - INFO - trainer -     eval loss: 0.9017819799482822
07/23/2021 21:00:02 - INFO - trainer -     acc = 0.7678571428571

07/23/2021 21:01:33 - INFO - trainer -     eval loss: 1.4827377183628934
07/23/2021 21:01:33 - INFO - trainer -     acc = 0.7821428571428571
07/23/2021 21:01:33 - INFO - trainer -     f1 = 0.6640637298124588
Epoch:  50%|█████     | 15/30 [03:26<03:23, 13.54s/it]07/23/2021 21:01:38 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:01:38 - INFO - trainer -     Num examples = 1120
07/23/2021 21:01:38 - INFO - trainer -     Batch size = 32
07/23/2021 21:01:39 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:01:39 - INFO - trainer -     eval loss: 1.600597586695637
07/23/2021 21:01:39 - INFO - trainer -     acc = 0.7803571428571429
07/23/2021 21:01:39 - INFO - trainer -     f1 = 0.6678333216553033
07/23/2021 21:01:44 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:01:44 - INFO - trainer -     Num examples = 1120
07/23/2021 21:01:44 - INFO - trainer -     Batch size = 32
07/23/2021 21:01:45 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 21:03:15 - INFO - trainer -     Batch size = 32
07/23/2021 21:03:17 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:03:17 - INFO - trainer -     eval loss: 1.8823651184086754
07/23/2021 21:03:17 - INFO - trainer -     acc = 0.7669642857142858
07/23/2021 21:03:17 - INFO - trainer -     f1 = 0.6450620338169695
07/23/2021 21:03:21 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:03:21 - INFO - trainer -     Num examples = 1120
07/23/2021 21:03:21 - INFO - trainer -     Batch size = 32
07/23/2021 21:03:23 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:03:23 - INFO - trainer -     eval loss: 1.9659954393926975
07/23/2021 21:03:23 - INFO - trainer -     acc = 0.7616071428571428
07/23/2021 21:03:23 - INFO - trainer -     f1 = 0.6526665985983239
Epoch:  77%|███████▋  | 23/30 [05:16<01:35, 13.62s/it]07/23/2021 21:03:28 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:03:28 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 21:04:58 - INFO - trainer -     Num examples = 1120
07/23/2021 21:04:58 - INFO - trainer -     Batch size = 32
07/23/2021 21:04:59 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:04:59 - INFO - trainer -     eval loss: 2.1218604662501352
07/23/2021 21:04:59 - INFO - trainer -     acc = 0.7660714285714286
07/23/2021 21:04:59 - INFO - trainer -     f1 = 0.6266677384431217
Epoch: 100%|██████████| 30/30 [06:52<00:00, 13.74s/it]
07/23/2021 21:05:03 - INFO - __main__ -     acc = 0.7857142857142857
07/23/2021 21:05:03 - INFO - __main__ -     f1 = 0.6710740356981434


### Output of Bert, output[4] Frozen 0

In [24]:
args.hidden_layer_no = 4
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:06:15 - INFO - model -   Hidden Layer 4 frozen percentage 0
07/23/2021 21:06:15 - INFO - trainer -   ***** Running training *****
07/23/2021 21:06:15 - INFO - trainer -     Num examples = 3602
07/23/2021 21:06:15 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:06:15 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:06:15 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:06:15 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:06:20 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:06:20 - INFO - trainer -     Num examples = 1120
07/23/2021 21:06:20 - INFO - trainer -     Batch size = 32
07/23/2021 21:06:21 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:06:21 - INFO - trainer -     eval loss: 0.8319082813603538
07/23/2021 21:06:21 - INFO - trainer -     acc = 0.6580357142857143
07/23/2021 21:06:21 - INFO - trainer -     f1 = 0.3089583

07/23/2021 21:07:46 - INFO - trainer -     acc = 0.7625
07/23/2021 21:07:46 - INFO - trainer -     f1 = 0.6357491249210995
07/23/2021 21:07:51 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:07:51 - INFO - trainer -     Num examples = 1120
07/23/2021 21:07:51 - INFO - trainer -     Batch size = 32
07/23/2021 21:07:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:07:52 - INFO - trainer -     eval loss: 0.9001162192651204
07/23/2021 21:07:52 - INFO - trainer -     acc = 0.7651785714285714
07/23/2021 21:07:52 - INFO - trainer -     f1 = 0.6654542183358543
07/23/2021 21:07:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:07:57 - INFO - trainer -     Num examples = 1120
07/23/2021 21:07:57 - INFO - trainer -     Batch size = 32
07/23/2021 21:07:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:07:58 - INFO - trainer -     eval loss: 0.7961768954992294
07/23/2021 21:07:58 - INFO - trainer -     acc = 0.7732142857142857
07/23/20

07/23/2021 21:09:24 - INFO - trainer -     eval loss: 1.5287791238299437
07/23/2021 21:09:24 - INFO - trainer -     acc = 0.7803571428571429
07/23/2021 21:09:24 - INFO - trainer -     f1 = 0.6739658147843796
Epoch:  50%|█████     | 15/30 [03:12<03:09, 12.61s/it]07/23/2021 21:09:29 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:09:29 - INFO - trainer -     Num examples = 1120
07/23/2021 21:09:29 - INFO - trainer -     Batch size = 32
07/23/2021 21:09:30 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:09:30 - INFO - trainer -     eval loss: 1.5118274636034454
07/23/2021 21:09:30 - INFO - trainer -     acc = 0.7758928571428572
07/23/2021 21:09:30 - INFO - trainer -     f1 = 0.6715384175010634
07/23/2021 21:09:34 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:09:34 - INFO - trainer -     Num examples = 1120
07/23/2021 21:09:34 - INFO - trainer -     Batch size = 32
07/23/2021 21:09:35 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 21:10:58 - INFO - trainer -     Batch size = 32
07/23/2021 21:10:59 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:10:59 - INFO - trainer -     eval loss: 1.9058528916112014
07/23/2021 21:10:59 - INFO - trainer -     acc = 0.7535714285714286
07/23/2021 21:10:59 - INFO - trainer -     f1 = 0.6429390255885766
07/23/2021 21:11:03 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:11:03 - INFO - trainer -     Num examples = 1120
07/23/2021 21:11:03 - INFO - trainer -     Batch size = 32
07/23/2021 21:11:05 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:11:05 - INFO - trainer -     eval loss: 1.868526771238872
07/23/2021 21:11:05 - INFO - trainer -     acc = 0.7794642857142857
07/23/2021 21:11:05 - INFO - trainer -     f1 = 0.675703630251255
Epoch:  77%|███████▋  | 23/30 [04:53<01:27, 12.46s/it]07/23/2021 21:11:09 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:11:09 - INFO - trainer -     Num examples = 1120
07/23/2021 2

07/23/2021 21:12:33 - INFO - trainer -     Num examples = 1120
07/23/2021 21:12:33 - INFO - trainer -     Batch size = 32
07/23/2021 21:12:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:12:34 - INFO - trainer -     eval loss: 2.4177176679883683
07/23/2021 21:12:34 - INFO - trainer -     acc = 0.7491071428571429
07/23/2021 21:12:34 - INFO - trainer -     f1 = 0.5897706107003108
Epoch: 100%|██████████| 30/30 [06:21<00:00, 12.73s/it]
07/23/2021 21:12:37 - INFO - __main__ -     acc = 0.7848214285714286
07/23/2021 21:12:37 - INFO - __main__ -     f1 = 0.6839390708793694


### Output of Bert, output[3] Frozen 0

In [25]:
args.hidden_layer_no = 3
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:14:20 - INFO - model -   Hidden Layer 3 frozen percentage 0
07/23/2021 21:14:20 - INFO - trainer -   ***** Running training *****
07/23/2021 21:14:20 - INFO - trainer -     Num examples = 3602
07/23/2021 21:14:20 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:14:20 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:14:20 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:14:20 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:14:24 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:14:24 - INFO - trainer -     Num examples = 1120
07/23/2021 21:14:24 - INFO - trainer -     Batch size = 32
07/23/2021 21:14:25 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:14:25 - INFO - trainer -     eval loss: 0.8657854846545628
07/23/2021 21:14:25 - INFO - trainer -     acc = 0.65
07/23/2021 21:14:25 - INFO - trainer -     f1 = 0.26262626262626265
07/

07/23/2021 21:15:44 - INFO - trainer -     acc = 0.7732142857142857
07/23/2021 21:15:44 - INFO - trainer -     f1 = 0.6541207509951704
07/23/2021 21:15:48 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:15:48 - INFO - trainer -     Num examples = 1120
07/23/2021 21:15:48 - INFO - trainer -     Batch size = 32
07/23/2021 21:15:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:15:49 - INFO - trainer -     eval loss: 0.8196573532053403
07/23/2021 21:15:49 - INFO - trainer -     acc = 0.7508928571428571
07/23/2021 21:15:49 - INFO - trainer -     f1 = 0.6553107338961442
07/23/2021 21:15:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:15:53 - INFO - trainer -     Num examples = 1120
07/23/2021 21:15:53 - INFO - trainer -     Batch size = 32
07/23/2021 21:15:54 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:15:54 - INFO - trainer -     eval loss: 0.752895949780941
07/23/2021 21:15:54 - INFO - trainer -     acc = 0.76071428571428

07/23/2021 21:17:13 - INFO - trainer -     eval loss: 1.2978238808257239
07/23/2021 21:17:13 - INFO - trainer -     acc = 0.7535714285714286
07/23/2021 21:17:13 - INFO - trainer -     f1 = 0.652335819285672
Epoch:  50%|█████     | 15/30 [02:56<02:54, 11.64s/it]07/23/2021 21:17:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:17:17 - INFO - trainer -     Num examples = 1120
07/23/2021 21:17:17 - INFO - trainer -     Batch size = 32
07/23/2021 21:17:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:17:18 - INFO - trainer -     eval loss: 1.4562470740505626
07/23/2021 21:17:18 - INFO - trainer -     acc = 0.7642857142857142
07/23/2021 21:17:18 - INFO - trainer -     f1 = 0.6407805328036542
07/23/2021 21:17:22 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:17:22 - INFO - trainer -     Num examples = 1120
07/23/2021 21:17:22 - INFO - trainer -     Batch size = 32
07/23/2021 21:17:23 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 21:18:40 - INFO - trainer -     Batch size = 32
07/23/2021 21:18:41 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:18:41 - INFO - trainer -     eval loss: 1.8135411396622658
07/23/2021 21:18:41 - INFO - trainer -     acc = 0.7508928571428571
07/23/2021 21:18:41 - INFO - trainer -     f1 = 0.6200951818854632
07/23/2021 21:18:45 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:18:45 - INFO - trainer -     Num examples = 1120
07/23/2021 21:18:45 - INFO - trainer -     Batch size = 32
07/23/2021 21:18:46 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:18:46 - INFO - trainer -     eval loss: 1.8425300160689013
07/23/2021 21:18:46 - INFO - trainer -     acc = 0.7616071428571428
07/23/2021 21:18:46 - INFO - trainer -     f1 = 0.6370177578142552
Epoch:  77%|███████▋  | 23/30 [04:29<01:21, 11.57s/it]07/23/2021 21:18:50 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:18:50 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 21:20:09 - INFO - trainer -     Num examples = 1120
07/23/2021 21:20:09 - INFO - trainer -     Batch size = 32
07/23/2021 21:20:10 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:20:10 - INFO - trainer -     eval loss: 2.1481233924627303
07/23/2021 21:20:10 - INFO - trainer -     acc = 0.7598214285714285
07/23/2021 21:20:10 - INFO - trainer -     f1 = 0.640020203729202
Epoch: 100%|██████████| 30/30 [05:53<00:00, 11.77s/it]
07/23/2021 21:20:13 - INFO - __main__ -     acc = 0.7848214285714286
07/23/2021 21:20:13 - INFO - __main__ -     f1 = 0.6624438214965042


### Output of Bert, output[2] Frozen 0

In [26]:
args.hidden_layer_no = 2
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:22:12 - INFO - model -   Hidden Layer 2 frozen percentage 0
07/23/2021 21:22:12 - INFO - trainer -   ***** Running training *****
07/23/2021 21:22:12 - INFO - trainer -     Num examples = 3602
07/23/2021 21:22:12 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:22:12 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:22:12 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:22:12 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:22:16 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:22:16 - INFO - trainer -     Num examples = 1120
07/23/2021 21:22:16 - INFO - trainer -     Batch size = 32
07/23/2021 21:22:17 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:22:17 - INFO - trainer -     eval loss: 0.8925731267247882
07/23/2021 21:22:17 - INFO - trainer -     acc = 0.65
07/23/2021 21:22:17 - INFO - trainer -     f1 = 0.26262626262626265
07/

07/23/2021 21:23:30 - INFO - trainer -     acc = 0.76875
07/23/2021 21:23:30 - INFO - trainer -     f1 = 0.6592215434452567
07/23/2021 21:23:33 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:23:33 - INFO - trainer -     Num examples = 1120
07/23/2021 21:23:33 - INFO - trainer -     Batch size = 32
07/23/2021 21:23:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:23:34 - INFO - trainer -     eval loss: 0.7929618362869535
07/23/2021 21:23:34 - INFO - trainer -     acc = 0.7419642857142857
07/23/2021 21:23:34 - INFO - trainer -     f1 = 0.6551175683762319
07/23/2021 21:23:38 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:23:38 - INFO - trainer -     Num examples = 1120
07/23/2021 21:23:38 - INFO - trainer -     Batch size = 32
07/23/2021 21:23:39 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:23:39 - INFO - trainer -     eval loss: 0.7167954217110362
07/23/2021 21:23:39 - INFO - trainer -     acc = 0.7598214285714285
07/23/2

07/23/2021 21:24:51 - INFO - trainer -     acc = 0.7464285714285714
07/23/2021 21:24:51 - INFO - trainer -     f1 = 0.6368776405238977
Epoch:  50%|█████     | 15/30 [02:42<02:40, 10.73s/it]07/23/2021 21:24:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:24:55 - INFO - trainer -     Num examples = 1120
07/23/2021 21:24:55 - INFO - trainer -     Batch size = 32
07/23/2021 21:24:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:24:56 - INFO - trainer -     eval loss: 1.1326560307826314
07/23/2021 21:24:56 - INFO - trainer -     acc = 0.7392857142857143
07/23/2021 21:24:56 - INFO - trainer -     f1 = 0.6218349438744709
07/23/2021 21:25:00 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:25:00 - INFO - trainer -     Num examples = 1120
07/23/2021 21:25:00 - INFO - trainer -     Batch size = 32
07/23/2021 21:25:01 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:25:01 - INFO - trainer -     eval loss: 1.2421877913177013
07/23/2021

07/23/2021 21:26:14 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:26:14 - INFO - trainer -     eval loss: 1.744172774042402
07/23/2021 21:26:14 - INFO - trainer -     acc = 0.7303571428571428
07/23/2021 21:26:14 - INFO - trainer -     f1 = 0.601640215825361
07/23/2021 21:26:18 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:26:18 - INFO - trainer -     Num examples = 1120
07/23/2021 21:26:18 - INFO - trainer -     Batch size = 32
07/23/2021 21:26:19 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:26:19 - INFO - trainer -     eval loss: 1.951490506529808
07/23/2021 21:26:19 - INFO - trainer -     acc = 0.7348214285714286
07/23/2021 21:26:19 - INFO - trainer -     f1 = 0.6190599312516891
Epoch:  77%|███████▋  | 23/30 [04:10<01:16, 10.93s/it]07/23/2021 21:26:23 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:26:23 - INFO - trainer -     Num examples = 1120
07/23/2021 21:26:23 - INFO - trainer -     Batch size = 32
07/23/2021 21

07/23/2021 21:27:37 - INFO - trainer -     Batch size = 32
07/23/2021 21:27:38 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:27:38 - INFO - trainer -     eval loss: 2.2927623949944973
07/23/2021 21:27:38 - INFO - trainer -     acc = 0.7392857142857143
07/23/2021 21:27:38 - INFO - trainer -     f1 = 0.606029207680053
Epoch: 100%|██████████| 30/30 [05:28<00:00, 10.95s/it]
07/23/2021 21:27:41 - INFO - __main__ -     acc = 0.7758928571428572
07/23/2021 21:27:41 - INFO - __main__ -     f1 = 0.6818424550257919


### Output of Bert, output[1] Frozen 0

In [27]:
args.hidden_layer_no = 1
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:29:08 - INFO - model -   Hidden Layer 1 frozen percentage 0
07/23/2021 21:29:08 - INFO - trainer -   ***** Running training *****
07/23/2021 21:29:08 - INFO - trainer -     Num examples = 3602
07/23/2021 21:29:08 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:29:08 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:29:08 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:29:08 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:29:11 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:29:11 - INFO - trainer -     Num examples = 1120
07/23/2021 21:29:11 - INFO - trainer -     Batch size = 32
07/23/2021 21:29:12 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:29:12 - INFO - trainer -     eval loss: 0.9156643169266837
07/23/2021 21:29:12 - INFO - trainer -     acc = 0.65
07/23/2021 21:29:12 - INFO - trainer -     f1 = 0.26262626262626265
07/

07/23/2021 21:30:19 - INFO - trainer -     acc = 0.7535714285714286
07/23/2021 21:30:19 - INFO - trainer -     f1 = 0.6108851774543683
07/23/2021 21:30:23 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:30:23 - INFO - trainer -     Num examples = 1120
07/23/2021 21:30:23 - INFO - trainer -     Batch size = 32
07/23/2021 21:30:24 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:30:24 - INFO - trainer -     eval loss: 0.8492424811635698
07/23/2021 21:30:24 - INFO - trainer -     acc = 0.7321428571428571
07/23/2021 21:30:24 - INFO - trainer -     f1 = 0.6225384797293927
07/23/2021 21:30:27 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:30:27 - INFO - trainer -     Num examples = 1120
07/23/2021 21:30:27 - INFO - trainer -     Batch size = 32
07/23/2021 21:30:28 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:30:28 - INFO - trainer -     eval loss: 0.8078548388821738
07/23/2021 21:30:28 - INFO - trainer -     acc = 0.7491071428571

07/23/2021 21:31:35 - INFO - trainer -     eval loss: 1.200575001431363
07/23/2021 21:31:35 - INFO - trainer -     acc = 0.7303571428571428
07/23/2021 21:31:35 - INFO - trainer -     f1 = 0.5844185250359681
Epoch:  50%|█████     | 15/30 [02:30<02:29,  9.98s/it]07/23/2021 21:31:38 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:31:38 - INFO - trainer -     Num examples = 1120
07/23/2021 21:31:38 - INFO - trainer -     Batch size = 32
07/23/2021 21:31:40 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:31:40 - INFO - trainer -     eval loss: 1.265752311689513
07/23/2021 21:31:40 - INFO - trainer -     acc = 0.7383928571428572
07/23/2021 21:31:40 - INFO - trainer -     f1 = 0.6204036849308273
07/23/2021 21:31:43 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:31:43 - INFO - trainer -     Num examples = 1120
07/23/2021 21:31:43 - INFO - trainer -     Batch size = 32
07/23/2021 21:31:44 - INFO - trainer -   ***** Eval results *****
07/23/2021 2

07/23/2021 21:32:51 - INFO - trainer -     Batch size = 32
07/23/2021 21:32:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:32:52 - INFO - trainer -     eval loss: 1.7786988007170814
07/23/2021 21:32:52 - INFO - trainer -     acc = 0.7232142857142857
07/23/2021 21:32:52 - INFO - trainer -     f1 = 0.6004118420078314
07/23/2021 21:32:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:32:55 - INFO - trainer -     Num examples = 1120
07/23/2021 21:32:55 - INFO - trainer -     Batch size = 32
07/23/2021 21:32:57 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:32:57 - INFO - trainer -     eval loss: 1.8755180808050291
07/23/2021 21:32:57 - INFO - trainer -     acc = 0.7196428571428571
07/23/2021 21:32:57 - INFO - trainer -     f1 = 0.5802008200932794
Epoch:  77%|███████▋  | 23/30 [03:51<01:10, 10.07s/it]07/23/2021 21:33:00 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:33:00 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 21:34:07 - INFO - trainer -     Num examples = 1120
07/23/2021 21:34:07 - INFO - trainer -     Batch size = 32
07/23/2021 21:34:08 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:34:08 - INFO - trainer -     eval loss: 2.409923778261457
07/23/2021 21:34:08 - INFO - trainer -     acc = 0.7071428571428572
07/23/2021 21:34:08 - INFO - trainer -     f1 = 0.5813107590456846
Epoch: 100%|██████████| 30/30 [05:02<00:00, 10.09s/it]
07/23/2021 21:34:10 - INFO - __main__ -     acc = 0.7633928571428571
07/23/2021 21:34:10 - INFO - __main__ -     f1 = 0.6241070849483733


### Output of Bert, output[0] Frozen 0

In [28]:
args.hidden_layer_no = 0
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:35:12 - INFO - model -   Hidden Layer 0 frozen percentage 0
07/23/2021 21:35:12 - INFO - trainer -   ***** Running training *****
07/23/2021 21:35:12 - INFO - trainer -     Num examples = 3602
07/23/2021 21:35:12 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:35:12 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:35:12 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:35:12 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:35:15 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:35:15 - INFO - trainer -     Num examples = 1120
07/23/2021 21:35:15 - INFO - trainer -     Batch size = 32
07/23/2021 21:35:16 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:35:16 - INFO - trainer -     eval loss: 0.8963853376252311
07/23/2021 21:35:16 - INFO - trainer -     acc = 0.65
07/23/2021 21:35:16 - INFO - trainer -     f1 = 0.26262626262626265
07/

07/23/2021 21:36:20 - INFO - trainer -     Num examples = 1120
07/23/2021 21:36:20 - INFO - trainer -     Batch size = 32
07/23/2021 21:36:21 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:36:21 - INFO - trainer -     eval loss: 0.896083174433027
07/23/2021 21:36:21 - INFO - trainer -     acc = 0.65
07/23/2021 21:36:21 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 21:36:24 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:36:24 - INFO - trainer -     Num examples = 1120
07/23/2021 21:36:24 - INFO - trainer -     Batch size = 32
07/23/2021 21:36:25 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:36:25 - INFO - trainer -     eval loss: 0.9039764864104135
07/23/2021 21:36:25 - INFO - trainer -     acc = 0.65
07/23/2021 21:36:25 - INFO - trainer -     f1 = 0.26262626262626265
Epoch:  27%|██▋       | 8/30 [01:13<03:23,  9.24s/it]07/23/2021 21:36:28 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:36:28 - INFO - trainer -

07/23/2021 21:37:29 - INFO - trainer -     Batch size = 32
07/23/2021 21:37:30 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:37:30 - INFO - trainer -     eval loss: 0.8988484944616045
07/23/2021 21:37:30 - INFO - trainer -     acc = 0.65
07/23/2021 21:37:30 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 21:37:33 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:37:33 - INFO - trainer -     Num examples = 1120
07/23/2021 21:37:33 - INFO - trainer -     Batch size = 32
07/23/2021 21:37:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:37:34 - INFO - trainer -     eval loss: 0.8950382990496499
07/23/2021 21:37:34 - INFO - trainer -     acc = 0.65
07/23/2021 21:37:34 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 21:37:37 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:37:37 - INFO - trainer -     Num examples = 1120
07/23/2021 21:37:37 - INFO - trainer -     Batch size = 32
07/23/2021 21:37:38 - INFO - tr

07/23/2021 21:38:40 - INFO - trainer -     eval loss: 0.8954925809587752
07/23/2021 21:38:40 - INFO - trainer -     acc = 0.65
07/23/2021 21:38:40 - INFO - trainer -     f1 = 0.26262626262626265
Epoch:  77%|███████▋  | 23/30 [03:30<01:03,  9.12s/it]07/23/2021 21:38:43 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:38:43 - INFO - trainer -     Num examples = 1120
07/23/2021 21:38:43 - INFO - trainer -     Batch size = 32
07/23/2021 21:38:44 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:38:44 - INFO - trainer -     eval loss: 0.8979322229112897
07/23/2021 21:38:44 - INFO - trainer -     acc = 0.65
07/23/2021 21:38:44 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 21:38:47 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:38:47 - INFO - trainer -     Num examples = 1120
07/23/2021 21:38:47 - INFO - trainer -     Batch size = 32
07/23/2021 21:38:48 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:38:48 - INFO - trainer

## Two Layers Model

In [3]:
# best_acc_eval_results_by_2layer = {}
# best_f1_eval_results_by_2layer = {}
import ast

with open('TwoL_Frozen_acc.txt') as file:
    data = file.read()    
file.close()
    
best_acc_eval_results_by_2layer = ast.literal_eval(data)


with open('TwoL_Frozen_f1.txt') as file:
    data = file.read()    
file.close()


best_f1_eval_results_by_2layer = ast.literal_eval(data)


In [9]:
print('Best Acc List:\n', best_acc_eval_results_by_2layer)
print()
print('Best F1 List:\n', best_f1_eval_results_by_2layer)

Best Acc List:
 {(12, 11, 0.0): {'acc': 0.8526785714285714, 'f1': 0.7803230773414049}, (11, 10, 0.0): {'acc': 0.8571428571428571, 'f1': 0.7857456140350877}, (10, 9, 0.0): {'acc': 0.8446428571428571, 'f1': 0.7739931418092337}, (9, 8, 0.0): {'acc': 0.8419642857142857, 'f1': 0.7598078385970185}, (8, 7, 0.0): {'acc': 0.81875, 'f1': 0.7233900935038547}, (12, 11, 0.25): {'acc': 0.8607142857142858, 'f1': 0.7882635730374766}, (11, 10, 0.25): {'acc': 0.8625, 'f1': 0.799830533061129}, (10, 9, 0.25): {'acc': 0.8508928571428571, 'f1': 0.7739272259731999}, (9, 8, 0.25): {'acc': 0.8464285714285714, 'f1': 0.7779506251390128}, (8, 7, 0.25): {'acc': 0.8330357142857143, 'f1': 0.7421664856980942}, (12, 11, 0.5): {'acc': 0.8482142857142857, 'f1': 0.7653221449912606}, (11, 10, 0.5): {'acc': 0.8607142857142858, 'f1': 0.7963273935366959}, (10, 9, 0.5): {'acc': 0.8464285714285714, 'f1': 0.7689448678540254}, (9, 8, 0.5): {'acc': 0.8455357142857143, 'f1': 0.7703085859359854}, (8, 7, 0.5): {'acc': 0.819642857142

In [10]:
with open('TwoL_Frozen_acc.txt', 'w') as file:
    file.write(str(best_acc_eval_results_by_2layer))
file.close()
    
with open('TwoL_Frozen_f1.txt', 'w') as file:
    file.write(str(best_f1_eval_results_by_2layer))
file.close()

In [5]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
    
# Parse args

args_str = "--embedding_type bert --output_dir data/output-gcn --dropout 0.3 --hidden_size 200 --learning_rate 5e-5 --bert_model_dir ./test/saved_model --pure_bert --hidden_layer_a_no 12 --hidden_layer_b_no 11 --frozen_percent 0"
#args = parse_args(['--gat_our', '--highway', '--num_heads', '7', '--dropout', '0.8', '--output_dir',
#                   'output/r-gat', '--glove_dir', 'glove', '--cuda_id', '0'])
args = parse_args(args_str.split(' '))
check_args(args)

# Setup CUDA, GPU training
os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_id
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args.device = device
logger.info('Device is %s', args.device)

# Set seed
set_seed(args)

# Bert, load pretrained model and tokenizer, check if neccesary to put bert here
if args.embedding_type == 'bert':
    tokenizer = BertTokenizer.from_pretrained(args.bert_model_dir)
    args.tokenizer = tokenizer

# Load datasets and vocabs
train_dataset, test_dataset, word_vocab, dep_tag_vocab, pos_tag_vocab= load_datasets_and_vocabs(args)



07/25/2021 17:03:12 - INFO - __main__ -   {'dataset_name': 'rest', 'output_dir': 'data/output-gcn', 'num_classes': 3, 'cuda_id': '3', 'seed': 2019, 'glove_dir': '/data1/SHENWZH/wordvec', 'bert_model_dir': './test/saved_model', 'pure_bert': True, 'pure_bert_layer_agg': False, 'pure_bert_layer_agg_list': '12', 'pure_bert_linear_layer_count': 2, 'gat_bert': False, 'highway': False, 'num_layers': 2, 'hidden_layer_no': 12, 'frozen_percent': 0.0, 'hidden_layer_a_no': 12, 'hidden_layer_b_no': 11, 'add_non_connect': True, 'multi_hop': True, 'max_hop': 4, 'num_heads': 6, 'dropout': 0.3, 'num_gcn_layers': 1, 'gcn_mem_dim': 300, 'gcn_dropout': 0.2, 'gat': False, 'gat_our': False, 'gat_attention_type': 'dotprod', 'embedding_type': 'bert', 'embedding_dim': 300, 'dep_relation_embed_dim': 300, 'hidden_size': 200, 'final_hidden_size': 300, 'num_mlps': 2, 'per_gpu_train_batch_size': 16, 'per_gpu_eval_batch_size': 32, 'gradient_accumulation_steps': 2, 'learning_rate': 5e-05, 'weight_decay': 0.0, 'adam_e

##  Layer 11 + Layer 12 Frozen 2 

In [4]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))


07/23/2021 03:52:08 - INFO - model -   Hidden Layer 11 & 12, FC1
07/23/2021 03:52:10 - INFO - trainer -   ***** Running training *****
07/23/2021 03:52:10 - INFO - trainer -     Num examples = 3602
07/23/2021 03:52:10 - INFO - trainer -     Num Epochs = 30
07/23/2021 03:52:10 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 03:52:10 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 03:52:10 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 03:52:18 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:52:18 - INFO - trainer -     Num examples = 1120
07/23/2021 03:52:18 - INFO - trainer -     Batch size = 32
07/23/2021 03:52:19 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:52:19 - INFO - trainer -     eval loss: 0.5427059139524187
07/23/2021 03:52:19 - INFO - trainer -     acc = 0.7910714285714285
07/23/2021 03:52:19 - INFO - trainer -     f1 = 0.6710150238294069

07/23/2021 03:54:31 - INFO - trainer -     acc = 0.8482142857142857
07/23/2021 03:54:31 - INFO - trainer -     f1 = 0.7717813872675819
07/23/2021 03:54:39 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:54:39 - INFO - trainer -     Num examples = 1120
07/23/2021 03:54:39 - INFO - trainer -     Batch size = 32
07/23/2021 03:54:40 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:54:40 - INFO - trainer -     eval loss: 0.7895552602214073
07/23/2021 03:54:40 - INFO - trainer -     acc = 0.85625
07/23/2021 03:54:40 - INFO - trainer -     f1 = 0.7878743668392203
07/23/2021 03:54:47 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:54:47 - INFO - trainer -     Num examples = 1120
07/23/2021 03:54:47 - INFO - trainer -     Batch size = 32
07/23/2021 03:54:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:54:49 - INFO - trainer -     eval loss: 0.8248617659422702
07/23/2021 03:54:49 - INFO - trainer -     acc = 0.8419642857142857
07/23/2

07/23/2021 03:56:58 - INFO - trainer -     eval loss: 1.1981529227736505
07/23/2021 03:56:58 - INFO - trainer -     acc = 0.8464285714285714
07/23/2021 03:56:58 - INFO - trainer -     f1 = 0.769884345530729
Epoch:  50%|█████     | 15/30 [04:54<04:49, 19.33s/it]07/23/2021 03:57:05 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:57:05 - INFO - trainer -     Num examples = 1120
07/23/2021 03:57:05 - INFO - trainer -     Batch size = 32
07/23/2021 03:57:07 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:57:07 - INFO - trainer -     eval loss: 1.0475693627543348
07/23/2021 03:57:07 - INFO - trainer -     acc = 0.8598214285714286
07/23/2021 03:57:07 - INFO - trainer -     f1 = 0.7957430951537852
07/23/2021 03:57:14 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:57:14 - INFO - trainer -     Num examples = 1120
07/23/2021 03:57:14 - INFO - trainer -     Batch size = 32
07/23/2021 03:57:15 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 03:59:23 - INFO - trainer -     Batch size = 32
07/23/2021 03:59:24 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:59:24 - INFO - trainer -     eval loss: 1.0949547391002332
07/23/2021 03:59:24 - INFO - trainer -     acc = 0.8473214285714286
07/23/2021 03:59:24 - INFO - trainer -     f1 = 0.7777524586809662
07/23/2021 03:59:32 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:59:32 - INFO - trainer -     Num examples = 1120
07/23/2021 03:59:32 - INFO - trainer -     Batch size = 32
07/23/2021 03:59:33 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:59:33 - INFO - trainer -     eval loss: 1.1553801152589065
07/23/2021 03:59:33 - INFO - trainer -     acc = 0.8482142857142857
07/23/2021 03:59:33 - INFO - trainer -     f1 = 0.7746263074148941
Epoch:  77%|███████▋  | 23/30 [07:29<02:15, 19.29s/it]07/23/2021 03:59:40 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:59:40 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 04:01:49 - INFO - trainer -     Num examples = 1120
07/23/2021 04:01:49 - INFO - trainer -     Batch size = 32
07/23/2021 04:01:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 04:01:50 - INFO - trainer -     eval loss: 1.540078211121155
07/23/2021 04:01:50 - INFO - trainer -     acc = 0.8321428571428572
07/23/2021 04:01:50 - INFO - trainer -     f1 = 0.726141641118113
Epoch: 100%|██████████| 30/30 [09:45<00:00, 19.53s/it]
07/23/2021 04:01:56 - INFO - __main__ -     acc = 0.8598214285714286
07/23/2021 04:01:56 - INFO - __main__ -     f1 = 0.7957430951537852


### Layer 12 + Layer 11, FC1, Frozen 0

In [7]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 12 
args.hidden_layer_b_no = 11 
args.frozen_percent = 0

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result
        

07/24/2021 20:19:52 - INFO - model -   Hidden Layer 12 & 11, Frozen rate 0.0
07/24/2021 20:19:52 - INFO - trainer -   ***** Running training *****
07/24/2021 20:19:52 - INFO - trainer -     Num examples = 3602
07/24/2021 20:19:52 - INFO - trainer -     Num Epochs = 30
07/24/2021 20:19:52 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 20:19:52 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 20:19:52 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 20:20:00 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:20:00 - INFO - trainer -     Num examples = 1120
07/24/2021 20:20:00 - INFO - trainer -     Batch size = 32
07/24/2021 20:20:01 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:20:01 - INFO - trainer -     eval loss: 0.5833822344030652
07/24/2021 20:20:01 - INFO - trainer -     acc = 0.7678571428571429
07/24/2021 20:20:01 - INFO - trainer -     f1 = 0.597

07/24/2021 20:22:19 - INFO - trainer -     eval loss: 0.7004998890184132
07/24/2021 20:22:19 - INFO - trainer -     acc = 0.8482142857142857
07/24/2021 20:22:19 - INFO - trainer -     f1 = 0.7761916451045897
07/24/2021 20:22:27 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:22:27 - INFO - trainer -     Num examples = 1120
07/24/2021 20:22:27 - INFO - trainer -     Batch size = 32
07/24/2021 20:22:28 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:22:28 - INFO - trainer -     eval loss: 0.7961379372415
07/24/2021 20:22:28 - INFO - trainer -     acc = 0.8482142857142857
07/24/2021 20:22:28 - INFO - trainer -     f1 = 0.7938899478291584
07/24/2021 20:22:36 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:22:36 - INFO - trainer -     Num examples = 1120
07/24/2021 20:22:36 - INFO - trainer -     Batch size = 32
07/24/2021 20:22:37 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:22:37 - INFO - trainer -     eval loss: 0.63342933499

07/24/2021 20:24:52 - INFO - trainer -     Batch size = 32
07/24/2021 20:24:53 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:24:53 - INFO - trainer -     eval loss: 1.029206395282277
07/24/2021 20:24:53 - INFO - trainer -     acc = 0.8410714285714286
07/24/2021 20:24:53 - INFO - trainer -     f1 = 0.7695888287412229

Epoch:  50%|█████     | 15/30 [05:07<05:04, 20.27s/it]07/24/2021 20:25:01 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:25:01 - INFO - trainer -     Num examples = 1120
07/24/2021 20:25:01 - INFO - trainer -     Batch size = 32
07/24/2021 20:25:02 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:25:02 - INFO - trainer -     eval loss: 1.0659430999940793
07/24/2021 20:25:02 - INFO - trainer -     acc = 0.8464285714285714
07/24/2021 20:25:02 - INFO - trainer -     f1 = 0.7681818181818182
07/24/2021 20:25:10 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:25:10 - INFO - trainer -     Num examples = 1120
07/24/2021


Epoch:  73%|███████▎  | 22/30 [07:30<02:41, 20.17s/it]07/24/2021 20:27:25 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:27:25 - INFO - trainer -     Num examples = 1120
07/24/2021 20:27:25 - INFO - trainer -     Batch size = 32
07/24/2021 20:27:26 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:27:26 - INFO - trainer -     eval loss: 1.06669949064921
07/24/2021 20:27:26 - INFO - trainer -     acc = 0.8553571428571428
07/24/2021 20:27:26 - INFO - trainer -     f1 = 0.7833795684149577
07/24/2021 20:27:34 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:27:34 - INFO - trainer -     Num examples = 1120
07/24/2021 20:27:34 - INFO - trainer -     Batch size = 32
07/24/2021 20:27:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:27:35 - INFO - trainer -     eval loss: 1.235227497602682
07/24/2021 20:27:35 - INFO - trainer -     acc = 0.8491071428571428
07/24/2021 20:27:35 - INFO - trainer -     f1 = 0.7793601651186789

Epoch:  77%

07/24/2021 20:29:50 - INFO - trainer -     eval loss: 1.210763097121214
07/24/2021 20:29:50 - INFO - trainer -     acc = 0.8410714285714286
07/24/2021 20:29:50 - INFO - trainer -     f1 = 0.7590417294237614
07/24/2021 20:29:57 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:29:57 - INFO - trainer -     Num examples = 1120
07/24/2021 20:29:57 - INFO - trainer -     Batch size = 32
07/24/2021 20:29:59 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:29:59 - INFO - trainer -     eval loss: 1.0325155200037572
07/24/2021 20:29:59 - INFO - trainer -     acc = 0.8428571428571429
07/24/2021 20:29:59 - INFO - trainer -     f1 = 0.7777556416241564

Epoch: 100%|██████████| 30/30 [10:12<00:00, 20.41s/it]
07/24/2021 20:30:05 - INFO - __main__ -     acc = 0.8607142857142858
07/24/2021 20:30:05 - INFO - __main__ -     f1 = 0.7951982679582237


### Layer 11 + Layer 10, FC1, Frozen 0

In [13]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 11
args.hidden_layer_b_no = 10
args.frozen_percent = 0

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 20:32:23 - INFO - model -   Hidden Layer 11 & 10, Frozen rate 0.0
07/24/2021 20:32:23 - INFO - trainer -   ***** Running training *****
07/24/2021 20:32:23 - INFO - trainer -     Num examples = 3602
07/24/2021 20:32:23 - INFO - trainer -     Num Epochs = 30
07/24/2021 20:32:23 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 20:32:23 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 20:32:23 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 20:32:31 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:32:31 - INFO - trainer -     Num examples = 1120
07/24/2021 20:32:31 - INFO - trainer -     Batch size = 32
07/24/2021 20:32:32 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:32:32 - INFO - trainer -     eval loss: 0.6195833772420883
07/24/2021 20:32:32 - INFO - trainer -     acc = 0.7553571428571428
07/24/2021 20:32:32 - INFO - trainer -     f1 = 0.596

07/24/2021 20:34:42 - INFO - trainer -     acc = 0.8455357142857143
07/24/2021 20:34:42 - INFO - trainer -     f1 = 0.7678307844748247
07/24/2021 20:34:49 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:34:49 - INFO - trainer -     Num examples = 1120
07/24/2021 20:34:49 - INFO - trainer -     Batch size = 32
07/24/2021 20:34:50 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:34:50 - INFO - trainer -     eval loss: 0.7603394440774407
07/24/2021 20:34:50 - INFO - trainer -     acc = 0.8535714285714285
07/24/2021 20:34:50 - INFO - trainer -     f1 = 0.7827516493229236
07/24/2021 20:34:58 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:34:58 - INFO - trainer -     Num examples = 1120
07/24/2021 20:34:58 - INFO - trainer -     Batch size = 32
07/24/2021 20:34:59 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:34:59 - INFO - trainer -     eval loss: 0.8498809445649386
07/24/2021 20:34:59 - INFO - trainer -     acc = 0.8383928571428

07/24/2021 20:37:07 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:37:07 - INFO - trainer -     eval loss: 1.2351316335193196
07/24/2021 20:37:07 - INFO - trainer -     acc = 0.8455357142857143
07/24/2021 20:37:07 - INFO - trainer -     f1 = 0.7639116385313947

Epoch:  50%|█████     | 15/30 [04:49<04:45, 19.02s/it]07/24/2021 20:37:14 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:37:14 - INFO - trainer -     Num examples = 1120
07/24/2021 20:37:14 - INFO - trainer -     Batch size = 32
07/24/2021 20:37:15 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:37:15 - INFO - trainer -     eval loss: 1.124032882048154
07/24/2021 20:37:15 - INFO - trainer -     acc = 0.8526785714285714
07/24/2021 20:37:15 - INFO - trainer -     f1 = 0.7833729530314678
07/24/2021 20:37:22 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:37:22 - INFO - trainer -     Num examples = 1120
07/24/2021 20:37:22 - INFO - trainer -     Batch size = 32
07/24/2021

07/24/2021 20:39:30 - INFO - trainer -     Num examples = 1120
07/24/2021 20:39:30 - INFO - trainer -     Batch size = 32
07/24/2021 20:39:31 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:39:31 - INFO - trainer -     eval loss: 1.2894093267644555
07/24/2021 20:39:31 - INFO - trainer -     acc = 0.8348214285714286
07/24/2021 20:39:31 - INFO - trainer -     f1 = 0.7589082166257056
07/24/2021 20:39:38 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:39:38 - INFO - trainer -     Num examples = 1120
07/24/2021 20:39:38 - INFO - trainer -     Batch size = 32
07/24/2021 20:39:39 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:39:39 - INFO - trainer -     eval loss: 1.2167197345622949
07/24/2021 20:39:39 - INFO - trainer -     acc = 0.8410714285714286
07/24/2021 20:39:39 - INFO - trainer -     f1 = 0.7706347439219013

Epoch:  77%|███████▋  | 23/30 [07:23<02:13, 19.00s/it]07/24/2021 20:39:47 - INFO - trainer -   ***** Running evaluation *****
07/24/202

07/24/2021 20:41:54 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:41:54 - INFO - trainer -     Num examples = 1120
07/24/2021 20:41:54 - INFO - trainer -     Batch size = 32
07/24/2021 20:41:55 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:41:55 - INFO - trainer -     eval loss: 1.3672272703546955
07/24/2021 20:41:55 - INFO - trainer -     acc = 0.8330357142857143
07/24/2021 20:41:55 - INFO - trainer -     f1 = 0.7473162936080353

Epoch: 100%|██████████| 30/30 [09:37<00:00, 19.26s/it]
07/24/2021 20:42:01 - INFO - __main__ -     acc = 0.8571428571428571
07/24/2021 20:42:01 - INFO - __main__ -     f1 = 0.7857456140350877


### Layer 10 + Layer 9, FC1, Frozen 0

In [14]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 10
args.hidden_layer_b_no = 9
args.frozen_percent = 0

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 20:43:26 - INFO - model -   Hidden Layer 10 & 9, Frozen rate 0.0
07/24/2021 20:43:26 - INFO - trainer -   ***** Running training *****
07/24/2021 20:43:26 - INFO - trainer -     Num examples = 3602
07/24/2021 20:43:26 - INFO - trainer -     Num Epochs = 30
07/24/2021 20:43:26 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 20:43:26 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 20:43:26 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 20:43:33 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:43:33 - INFO - trainer -     Num examples = 1120
07/24/2021 20:43:33 - INFO - trainer -     Batch size = 32
07/24/2021 20:43:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:43:35 - INFO - trainer -     eval loss: 0.7809685383524213
07/24/2021 20:43:35 - INFO - trainer -     acc = 0.6723214285714286
07/24/2021 20:43:35 - INFO - trainer -     f1 = 0.4455

07/24/2021 20:45:43 - INFO - trainer -     acc = 0.825
07/24/2021 20:45:43 - INFO - trainer -     f1 = 0.7307957617510589
07/24/2021 20:45:50 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:45:50 - INFO - trainer -     Num examples = 1120
07/24/2021 20:45:50 - INFO - trainer -     Batch size = 32
07/24/2021 20:45:51 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:45:51 - INFO - trainer -     eval loss: 1.034096600700702
07/24/2021 20:45:51 - INFO - trainer -     acc = 0.8285714285714286
07/24/2021 20:45:51 - INFO - trainer -     f1 = 0.7423356110632828
07/24/2021 20:45:58 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:45:58 - INFO - trainer -     Num examples = 1120
07/24/2021 20:45:58 - INFO - trainer -     Batch size = 32
07/24/2021 20:45:59 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:45:59 - INFO - trainer -     eval loss: 1.057438575256882
07/24/2021 20:45:59 - INFO - trainer -     acc = 0.8267857142857142
07/24/2021 

07/24/2021 20:48:02 - INFO - trainer -     eval loss: 1.3061328735102766
07/24/2021 20:48:02 - INFO - trainer -     acc = 0.8321428571428572
07/24/2021 20:48:02 - INFO - trainer -     f1 = 0.7403801811952517

Epoch:  50%|█████     | 15/30 [04:41<04:35, 18.34s/it]07/24/2021 20:48:09 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:48:09 - INFO - trainer -     Num examples = 1120
07/24/2021 20:48:09 - INFO - trainer -     Batch size = 32
07/24/2021 20:48:10 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:48:10 - INFO - trainer -     eval loss: 1.3555138620730889
07/24/2021 20:48:10 - INFO - trainer -     acc = 0.8339285714285715
07/24/2021 20:48:10 - INFO - trainer -     f1 = 0.7433162822159286
07/24/2021 20:48:17 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:48:17 - INFO - trainer -     Num examples = 1120
07/24/2021 20:48:17 - INFO - trainer -     Batch size = 32
07/24/2021 20:48:18 - INFO - trainer -   ***** Eval results *****
07/24/202

07/24/2021 20:50:19 - INFO - trainer -     Batch size = 32
07/24/2021 20:50:20 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:50:20 - INFO - trainer -     eval loss: 1.4983210743613848
07/24/2021 20:50:20 - INFO - trainer -     acc = 0.8214285714285714
07/24/2021 20:50:20 - INFO - trainer -     f1 = 0.7434306492515875
07/24/2021 20:50:27 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:50:27 - INFO - trainer -     Num examples = 1120
07/24/2021 20:50:27 - INFO - trainer -     Batch size = 32
07/24/2021 20:50:28 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:50:28 - INFO - trainer -     eval loss: 1.4340662718823296
07/24/2021 20:50:28 - INFO - trainer -     acc = 0.8294642857142858
07/24/2021 20:50:28 - INFO - trainer -     f1 = 0.7564305376208784

Epoch:  77%|███████▋  | 23/30 [07:08<02:07, 18.25s/it]07/24/2021 20:50:35 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:50:35 - INFO - trainer -     Num examples = 1120
07/24/202

07/24/2021 20:52:38 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:52:38 - INFO - trainer -     Num examples = 1120
07/24/2021 20:52:38 - INFO - trainer -     Batch size = 32
07/24/2021 20:52:39 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:52:39 - INFO - trainer -     eval loss: 1.281639055537692
07/24/2021 20:52:39 - INFO - trainer -     acc = 0.83125
07/24/2021 20:52:39 - INFO - trainer -     f1 = 0.7467846634396564

Epoch: 100%|██████████| 30/30 [09:18<00:00, 18.62s/it]
07/24/2021 20:52:45 - INFO - __main__ -     acc = 0.8446428571428571
07/24/2021 20:52:45 - INFO - __main__ -     f1 = 0.7739931418092337


### Layer 9 + Layer 8, FC1, Frozen 0

In [16]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 9
args.hidden_layer_b_no = 8
args.frozen_percent = 0

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 20:55:20 - INFO - model -   Hidden Layer 9 & 8, Frozen rate 0.0
07/24/2021 20:55:20 - INFO - trainer -   ***** Running training *****
07/24/2021 20:55:20 - INFO - trainer -     Num examples = 3602
07/24/2021 20:55:20 - INFO - trainer -     Num Epochs = 30
07/24/2021 20:55:20 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 20:55:20 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 20:55:20 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 20:55:27 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:55:27 - INFO - trainer -     Num examples = 1120
07/24/2021 20:55:27 - INFO - trainer -     Batch size = 32
07/24/2021 20:55:28 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:55:28 - INFO - trainer -     eval loss: 0.8252747229167393
07/24/2021 20:55:28 - INFO - trainer -     acc = 0.6544642857142857
07/24/2021 20:55:28 - INFO - trainer -     f1 = 0.43423

07/24/2021 20:57:26 - INFO - trainer -     acc = 0.8223214285714285
07/24/2021 20:57:26 - INFO - trainer -     f1 = 0.7200233790098401
07/24/2021 20:57:33 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:57:33 - INFO - trainer -     Num examples = 1120
07/24/2021 20:57:33 - INFO - trainer -     Batch size = 32
07/24/2021 20:57:34 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:57:34 - INFO - trainer -     eval loss: 0.9603877478678312
07/24/2021 20:57:34 - INFO - trainer -     acc = 0.825
07/24/2021 20:57:34 - INFO - trainer -     f1 = 0.7359957585821055
07/24/2021 20:57:40 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:57:40 - INFO - trainer -     Num examples = 1120
07/24/2021 20:57:40 - INFO - trainer -     Batch size = 32
07/24/2021 20:57:41 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:57:41 - INFO - trainer -     eval loss: 0.9083973680356784
07/24/2021 20:57:41 - INFO - trainer -     acc = 0.8276785714285714
07/24/202

07/24/2021 20:59:37 - INFO - trainer -     eval loss: 1.271101704672245
07/24/2021 20:59:37 - INFO - trainer -     acc = 0.8366071428571429
07/24/2021 20:59:37 - INFO - trainer -     f1 = 0.752734936750282

Epoch:  50%|█████     | 15/30 [04:22<04:18, 17.21s/it]07/24/2021 20:59:43 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:59:43 - INFO - trainer -     Num examples = 1120
07/24/2021 20:59:43 - INFO - trainer -     Batch size = 32
07/24/2021 20:59:44 - INFO - trainer -   ***** Eval results *****
07/24/2021 20:59:44 - INFO - trainer -     eval loss: 1.293226736358234
07/24/2021 20:59:44 - INFO - trainer -     acc = 0.8357142857142857
07/24/2021 20:59:44 - INFO - trainer -     f1 = 0.7595187070553887
07/24/2021 20:59:51 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 20:59:51 - INFO - trainer -     Num examples = 1120
07/24/2021 20:59:51 - INFO - trainer -     Batch size = 32
07/24/2021 20:59:52 - INFO - trainer -   ***** Eval results *****
07/24/2021 2

07/24/2021 21:01:46 - INFO - trainer -     Num examples = 1120
07/24/2021 21:01:46 - INFO - trainer -     Batch size = 32
07/24/2021 21:01:47 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:01:47 - INFO - trainer -     eval loss: 1.176656475769622
07/24/2021 21:01:47 - INFO - trainer -     acc = 0.8223214285714285
07/24/2021 21:01:47 - INFO - trainer -     f1 = 0.7389257430102858
07/24/2021 21:01:54 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:01:54 - INFO - trainer -     Num examples = 1120
07/24/2021 21:01:54 - INFO - trainer -     Batch size = 32
07/24/2021 21:01:55 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:01:55 - INFO - trainer -     eval loss: 1.3573888548782893
07/24/2021 21:01:55 - INFO - trainer -     acc = 0.8107142857142857
07/24/2021 21:01:55 - INFO - trainer -     f1 = 0.7326874024812877

Epoch:  77%|███████▋  | 23/30 [06:41<02:00, 17.27s/it]07/24/2021 21:02:01 - INFO - trainer -   ***** Running evaluation *****
07/24/2021

07/24/2021 21:03:50 - INFO - trainer -     f1 = 0.7307253088791171
07/24/2021 21:03:57 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:03:57 - INFO - trainer -     Num examples = 1120
07/24/2021 21:03:57 - INFO - trainer -     Batch size = 32
07/24/2021 21:03:58 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:03:58 - INFO - trainer -     eval loss: 1.503513613413088
07/24/2021 21:03:58 - INFO - trainer -     acc = 0.8205357142857143
07/24/2021 21:03:58 - INFO - trainer -     f1 = 0.7177890984762471

Epoch: 100%|██████████| 30/30 [08:42<00:00, 17.43s/it]
07/24/2021 21:04:03 - INFO - __main__ -     acc = 0.8419642857142857
07/24/2021 21:04:03 - INFO - __main__ -     f1 = 0.7598078385970185


### Layer 8 + Layer 7, FC1, Frozen 0

In [17]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 8
args.hidden_layer_b_no = 7
args.frozen_percent = 0

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 21:06:26 - INFO - model -   Hidden Layer 8 & 7, Frozen rate 0.0
07/24/2021 21:06:27 - INFO - trainer -   ***** Running training *****
07/24/2021 21:06:27 - INFO - trainer -     Num examples = 3602
07/24/2021 21:06:27 - INFO - trainer -     Num Epochs = 30
07/24/2021 21:06:27 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 21:06:27 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 21:06:27 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 21:06:33 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:06:33 - INFO - trainer -     Num examples = 1120
07/24/2021 21:06:33 - INFO - trainer -     Batch size = 32
07/24/2021 21:06:34 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:06:34 - INFO - trainer -     eval loss: 0.8293109706469944
07/24/2021 21:06:34 - INFO - trainer -     acc = 0.6758928571428572
07/24/2021 21:06:34 - INFO - trainer -     f1 = 0.40265

07/24/2021 21:08:26 - INFO - trainer -     acc = 0.7991071428571429
07/24/2021 21:08:26 - INFO - trainer -     f1 = 0.6908747995944329
07/24/2021 21:08:32 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:08:32 - INFO - trainer -     Num examples = 1120
07/24/2021 21:08:32 - INFO - trainer -     Batch size = 32
07/24/2021 21:08:33 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:08:33 - INFO - trainer -     eval loss: 1.027752183164869
07/24/2021 21:08:33 - INFO - trainer -     acc = 0.8008928571428572
07/24/2021 21:08:33 - INFO - trainer -     f1 = 0.6974794575634472
07/24/2021 21:08:40 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:08:40 - INFO - trainer -     Num examples = 1120
07/24/2021 21:08:40 - INFO - trainer -     Batch size = 32
07/24/2021 21:08:41 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:08:41 - INFO - trainer -     eval loss: 0.919026266677039
07/24/2021 21:08:41 - INFO - trainer -     acc = 0.798214285714285

07/24/2021 21:10:31 - INFO - trainer -     eval loss: 1.3689970222567873
07/24/2021 21:10:31 - INFO - trainer -     acc = 0.8044642857142857
07/24/2021 21:10:31 - INFO - trainer -     f1 = 0.6947871941170872

Epoch:  50%|█████     | 15/30 [04:10<04:06, 16.43s/it]07/24/2021 21:10:37 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:10:37 - INFO - trainer -     Num examples = 1120
07/24/2021 21:10:37 - INFO - trainer -     Batch size = 32
07/24/2021 21:10:38 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:10:38 - INFO - trainer -     eval loss: 1.542168424503013
07/24/2021 21:10:38 - INFO - trainer -     acc = 0.8017857142857143
07/24/2021 21:10:38 - INFO - trainer -     f1 = 0.6937894658418458
07/24/2021 21:10:44 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:10:44 - INFO - trainer -     Num examples = 1120
07/24/2021 21:10:44 - INFO - trainer -     Batch size = 32
07/24/2021 21:10:46 - INFO - trainer -   ***** Eval results *****
07/24/2021

07/24/2021 21:12:35 - INFO - trainer -     Num examples = 1120
07/24/2021 21:12:35 - INFO - trainer -     Batch size = 32
07/24/2021 21:12:36 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:12:36 - INFO - trainer -     eval loss: 1.6404830421720231
07/24/2021 21:12:36 - INFO - trainer -     acc = 0.8044642857142857
07/24/2021 21:12:36 - INFO - trainer -     f1 = 0.6988674579593153
07/24/2021 21:12:42 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:12:42 - INFO - trainer -     Num examples = 1120
07/24/2021 21:12:42 - INFO - trainer -     Batch size = 32
07/24/2021 21:12:43 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:12:43 - INFO - trainer -     eval loss: 1.7327599831989833
07/24/2021 21:12:43 - INFO - trainer -     acc = 0.7964285714285714
07/24/2021 21:12:43 - INFO - trainer -     f1 = 0.6805847546862346

Epoch:  77%|███████▋  | 23/30 [06:22<01:55, 16.43s/it]07/24/2021 21:12:49 - INFO - trainer -   ***** Running evaluation *****
07/24/202

07/24/2021 21:14:34 - INFO - trainer -     f1 = 0.6885423354116981
07/24/2021 21:14:40 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:14:40 - INFO - trainer -     Num examples = 1120
07/24/2021 21:14:40 - INFO - trainer -     Batch size = 32
07/24/2021 21:14:41 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:14:41 - INFO - trainer -     eval loss: 1.8154807490961893
07/24/2021 21:14:41 - INFO - trainer -     acc = 0.80625
07/24/2021 21:14:41 - INFO - trainer -     f1 = 0.7091483929902197

Epoch: 100%|██████████| 30/30 [08:19<00:00, 16.64s/it]
07/24/2021 21:14:46 - INFO - __main__ -     acc = 0.81875
07/24/2021 21:14:46 - INFO - __main__ -     f1 = 0.7233900935038547


### Layer 12 + Layer 11, FC1, Frozen 0.25

In [22]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 12
args.hidden_layer_b_no = 11
args.frozen_percent = 0.25

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 21:30:21 - INFO - model -   Hidden Layer 12 & 11, Frozen rate 0.25
07/24/2021 21:30:21 - INFO - trainer -   ***** Running training *****
07/24/2021 21:30:21 - INFO - trainer -     Num examples = 3602
07/24/2021 21:30:21 - INFO - trainer -     Num Epochs = 30
07/24/2021 21:30:21 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 21:30:21 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 21:30:21 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 21:30:27 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:30:27 - INFO - trainer -     Num examples = 1120
07/24/2021 21:30:27 - INFO - trainer -     Batch size = 32
07/24/2021 21:30:28 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:30:28 - INFO - trainer -     eval loss: 0.6205353756036077
07/24/2021 21:30:28 - INFO - trainer -     acc = 0.7580357142857143
07/24/2021 21:30:28 - INFO - trainer -     f1 = 0.62

07/24/2021 21:32:20 - INFO - trainer -     eval loss: 0.7487886514714254
07/24/2021 21:32:20 - INFO - trainer -     acc = 0.85
07/24/2021 21:32:20 - INFO - trainer -     f1 = 0.7765062318252506
07/24/2021 21:32:26 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:32:26 - INFO - trainer -     Num examples = 1120
07/24/2021 21:32:26 - INFO - trainer -     Batch size = 32
07/24/2021 21:32:27 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:32:27 - INFO - trainer -     eval loss: 1.0598075664252975
07/24/2021 21:32:27 - INFO - trainer -     acc = 0.8348214285714286
07/24/2021 21:32:27 - INFO - trainer -     f1 = 0.7355839355839356
07/24/2021 21:32:34 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:32:34 - INFO - trainer -     Num examples = 1120
07/24/2021 21:32:34 - INFO - trainer -     Batch size = 32
07/24/2021 21:32:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:32:35 - INFO - trainer -     eval loss: 0.7311538728585999
07/24

07/24/2021 21:34:23 - INFO - trainer -     Batch size = 32
07/24/2021 21:34:24 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:34:24 - INFO - trainer -     eval loss: 1.1903151994095034
07/24/2021 21:34:24 - INFO - trainer -     acc = 0.8455357142857143
07/24/2021 21:34:24 - INFO - trainer -     f1 = 0.7661988056952255

Epoch:  50%|█████     | 15/30 [04:08<04:05, 16.33s/it]07/24/2021 21:34:30 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:34:30 - INFO - trainer -     Num examples = 1120
07/24/2021 21:34:30 - INFO - trainer -     Batch size = 32
07/24/2021 21:34:31 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:34:31 - INFO - trainer -     eval loss: 1.1869402865217125
07/24/2021 21:34:31 - INFO - trainer -     acc = 0.8446428571428571
07/24/2021 21:34:31 - INFO - trainer -     f1 = 0.7608570831560385
07/24/2021 21:34:38 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:34:38 - INFO - trainer -     Num examples = 1120
07/24/202

07/24/2021 21:38:25 - INFO - trainer -     eval loss: 1.2865917825302209
07/24/2021 21:38:25 - INFO - trainer -     acc = 0.8410714285714286
07/24/2021 21:38:25 - INFO - trainer -     f1 = 0.7433996377679075
07/24/2021 21:38:31 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:38:31 - INFO - trainer -     Num examples = 1120
07/24/2021 21:38:31 - INFO - trainer -     Batch size = 32
07/24/2021 21:38:32 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:38:32 - INFO - trainer -     eval loss: 1.37248424599259
07/24/2021 21:38:32 - INFO - trainer -     acc = 0.8321428571428572
07/24/2021 21:38:32 - INFO - trainer -     f1 = 0.729316071726836

Epoch: 100%|██████████| 30/30 [08:16<00:00, 16.55s/it]
07/24/2021 21:38:37 - INFO - __main__ -     acc = 0.8607142857142858
07/24/2021 21:38:37 - INFO - __main__ -     f1 = 0.7882635730374766


### Layer 11 + Layer 10, FC1, Frozen 0.25

In [23]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 11
args.hidden_layer_b_no = 10
args.frozen_percent = 0.25

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 21:41:00 - INFO - model -   Hidden Layer 11 & 10, Frozen rate 0.25
07/24/2021 21:41:00 - INFO - trainer -   ***** Running training *****
07/24/2021 21:41:00 - INFO - trainer -     Num examples = 3602
07/24/2021 21:41:00 - INFO - trainer -     Num Epochs = 30
07/24/2021 21:41:00 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 21:41:00 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 21:41:00 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 21:41:06 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:41:06 - INFO - trainer -     Num examples = 1120
07/24/2021 21:41:06 - INFO - trainer -     Batch size = 32
07/24/2021 21:41:07 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:41:07 - INFO - trainer -     eval loss: 0.6159722558089665
07/24/2021 21:41:07 - INFO - trainer -     acc = 0.7446428571428572
07/24/2021 21:41:07 - INFO - trainer -     f1 = 0.59

07/24/2021 21:42:56 - INFO - trainer -     eval loss: 0.7660928201728634
07/24/2021 21:42:56 - INFO - trainer -     acc = 0.8473214285714286
07/24/2021 21:42:56 - INFO - trainer -     f1 = 0.7714839531472933
07/24/2021 21:43:02 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:43:02 - INFO - trainer -     Num examples = 1120
07/24/2021 21:43:02 - INFO - trainer -     Batch size = 32
07/24/2021 21:43:03 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:43:03 - INFO - trainer -     eval loss: 0.8389516860645797
07/24/2021 21:43:03 - INFO - trainer -     acc = 0.8482142857142857
07/24/2021 21:43:03 - INFO - trainer -     f1 = 0.7556405305538635
07/24/2021 21:43:09 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:43:09 - INFO - trainer -     Num examples = 1120
07/24/2021 21:43:09 - INFO - trainer -     Batch size = 32
07/24/2021 21:43:10 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:43:10 - INFO - trainer -     eval loss: 0.69511230

07/24/2021 21:44:57 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:44:57 - INFO - trainer -     eval loss: 1.1158173831805471
07/24/2021 21:44:57 - INFO - trainer -     acc = 0.8446428571428571
07/24/2021 21:44:57 - INFO - trainer -     f1 = 0.7660817290005791

Epoch:  50%|█████     | 15/30 [04:02<04:01, 16.10s/it]07/24/2021 21:45:03 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:45:03 - INFO - trainer -     Num examples = 1120
07/24/2021 21:45:03 - INFO - trainer -     Batch size = 32
07/24/2021 21:45:04 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:45:04 - INFO - trainer -     eval loss: 1.2130050282029385
07/24/2021 21:45:04 - INFO - trainer -     acc = 0.8464285714285714
07/24/2021 21:45:04 - INFO - trainer -     f1 = 0.764132999547368
07/24/2021 21:45:10 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:45:10 - INFO - trainer -     Num examples = 1120
07/24/2021 21:45:10 - INFO - trainer -     Batch size = 32
07/24/2021

07/24/2021 21:48:50 - INFO - trainer -     eval loss: 1.2853618870857255
07/24/2021 21:48:50 - INFO - trainer -     acc = 0.8491071428571428
07/24/2021 21:48:50 - INFO - trainer -     f1 = 0.7690892261784831
07/24/2021 21:48:56 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:48:56 - INFO - trainer -     Num examples = 1120
07/24/2021 21:48:56 - INFO - trainer -     Batch size = 32
07/24/2021 21:48:57 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:48:57 - INFO - trainer -     eval loss: 1.2579119244688108
07/24/2021 21:48:57 - INFO - trainer -     acc = 0.8491071428571428
07/24/2021 21:48:57 - INFO - trainer -     f1 = 0.7798954181732306

Epoch: 100%|██████████| 30/30 [08:01<00:00, 16.04s/it]
07/24/2021 21:49:02 - INFO - __main__ -     acc = 0.8625
07/24/2021 21:49:02 - INFO - __main__ -     f1 = 0.799830533061129


In [ ]:
### Layer 11 + Layer 10, FC1, Frozen 0.25 Confirm

In [26]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 11
args.hidden_layer_b_no = 10
args.frozen_percent = 0.25

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 21:51:35 - INFO - model -   Hidden Layer 11 & 10, Frozen rate 0.25
07/24/2021 21:51:35 - INFO - trainer -   ***** Running training *****
07/24/2021 21:51:35 - INFO - trainer -     Num examples = 3602
07/24/2021 21:51:35 - INFO - trainer -     Num Epochs = 30
07/24/2021 21:51:35 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 21:51:35 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 21:51:35 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 21:51:41 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:51:41 - INFO - trainer -     Num examples = 1120
07/24/2021 21:51:41 - INFO - trainer -     Batch size = 32
07/24/2021 21:51:42 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:51:42 - INFO - trainer -     eval loss: 0.6159722558089665
07/24/2021 21:51:42 - INFO - trainer -     acc = 0.7446428571428572
07/24/2021 21:51:42 - INFO - trainer -     f1 = 0.59

07/24/2021 21:53:29 - INFO - trainer -     eval loss: 0.7660928201728634
07/24/2021 21:53:29 - INFO - trainer -     acc = 0.8473214285714286
07/24/2021 21:53:29 - INFO - trainer -     f1 = 0.7714839531472933
07/24/2021 21:53:35 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:53:35 - INFO - trainer -     Num examples = 1120
07/24/2021 21:53:35 - INFO - trainer -     Batch size = 32
07/24/2021 21:53:36 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:53:36 - INFO - trainer -     eval loss: 0.8389516860645797
07/24/2021 21:53:36 - INFO - trainer -     acc = 0.8482142857142857
07/24/2021 21:53:36 - INFO - trainer -     f1 = 0.7556405305538635
07/24/2021 21:53:42 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:53:42 - INFO - trainer -     Num examples = 1120
07/24/2021 21:53:42 - INFO - trainer -     Batch size = 32
07/24/2021 21:53:44 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:53:44 - INFO - trainer -     eval loss: 0.69511230

07/24/2021 21:55:29 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:55:29 - INFO - trainer -     eval loss: 1.1158173831805471
07/24/2021 21:55:29 - INFO - trainer -     acc = 0.8446428571428571
07/24/2021 21:55:29 - INFO - trainer -     f1 = 0.7660817290005791

Epoch:  50%|█████     | 15/30 [03:59<03:56, 15.74s/it]07/24/2021 21:55:35 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:55:35 - INFO - trainer -     Num examples = 1120
07/24/2021 21:55:35 - INFO - trainer -     Batch size = 32
07/24/2021 21:55:36 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:55:36 - INFO - trainer -     eval loss: 1.2130050282029385
07/24/2021 21:55:36 - INFO - trainer -     acc = 0.8464285714285714
07/24/2021 21:55:36 - INFO - trainer -     f1 = 0.764132999547368
07/24/2021 21:55:42 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:55:42 - INFO - trainer -     Num examples = 1120
07/24/2021 21:55:42 - INFO - trainer -     Batch size = 32
07/24/2021

07/24/2021 21:59:19 - INFO - trainer -     eval loss: 1.2853618870857255
07/24/2021 21:59:19 - INFO - trainer -     acc = 0.8491071428571428
07/24/2021 21:59:19 - INFO - trainer -     f1 = 0.7690892261784831
07/24/2021 21:59:25 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 21:59:25 - INFO - trainer -     Num examples = 1120
07/24/2021 21:59:25 - INFO - trainer -     Batch size = 32
07/24/2021 21:59:26 - INFO - trainer -   ***** Eval results *****
07/24/2021 21:59:26 - INFO - trainer -     eval loss: 1.2579119244688108
07/24/2021 21:59:26 - INFO - trainer -     acc = 0.8491071428571428
07/24/2021 21:59:26 - INFO - trainer -     f1 = 0.7798954181732306

Epoch: 100%|██████████| 30/30 [07:55<00:00, 15.86s/it]
07/24/2021 21:59:30 - INFO - __main__ -     acc = 0.8625
07/24/2021 21:59:30 - INFO - __main__ -     f1 = 0.799830533061129


### Layer 10 + Layer 9 FC1, Frozen 0.25

In [27]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 10
args.hidden_layer_b_no = 9
args.frozen_percent = 0.25

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 22:01:48 - INFO - model -   Hidden Layer 10 & 9, Frozen rate 0.25
07/24/2021 22:01:48 - INFO - trainer -   ***** Running training *****
07/24/2021 22:01:48 - INFO - trainer -     Num examples = 3602
07/24/2021 22:01:48 - INFO - trainer -     Num Epochs = 30
07/24/2021 22:01:48 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 22:01:48 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 22:01:48 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 22:01:53 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:01:53 - INFO - trainer -     Num examples = 1120
07/24/2021 22:01:53 - INFO - trainer -     Batch size = 32
07/24/2021 22:01:54 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:01:54 - INFO - trainer -     eval loss: 0.7752705693244935
07/24/2021 22:01:54 - INFO - trainer -     acc = 0.6991071428571428
07/24/2021 22:01:54 - INFO - trainer -     f1 = 0.481

07/24/2021 22:03:35 - INFO - trainer -     eval loss: 0.8409024349413812
07/24/2021 22:03:35 - INFO - trainer -     acc = 0.8410714285714286
07/24/2021 22:03:35 - INFO - trainer -     f1 = 0.7559501899040405
07/24/2021 22:03:40 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:03:40 - INFO - trainer -     Num examples = 1120
07/24/2021 22:03:40 - INFO - trainer -     Batch size = 32
07/24/2021 22:03:41 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:03:41 - INFO - trainer -     eval loss: 0.7943162121810019
07/24/2021 22:03:41 - INFO - trainer -     acc = 0.8491071428571428
07/24/2021 22:03:41 - INFO - trainer -     f1 = 0.7724254955692834
07/24/2021 22:03:47 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:03:47 - INFO - trainer -     Num examples = 1120
07/24/2021 22:03:47 - INFO - trainer -     Batch size = 32
07/24/2021 22:03:48 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:03:48 - INFO - trainer -     eval loss: 0.88326670

07/24/2021 22:05:26 - INFO - trainer -     Batch size = 32
07/24/2021 22:05:27 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:05:27 - INFO - trainer -     eval loss: 1.1147120811344524
07/24/2021 22:05:27 - INFO - trainer -     acc = 0.8419642857142857
07/24/2021 22:05:27 - INFO - trainer -     f1 = 0.7577567747143322

Epoch:  50%|█████     | 15/30 [03:44<03:42, 14.84s/it]07/24/2021 22:05:33 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:05:33 - INFO - trainer -     Num examples = 1120
07/24/2021 22:05:33 - INFO - trainer -     Batch size = 32
07/24/2021 22:05:34 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:05:34 - INFO - trainer -     eval loss: 1.415403872355403
07/24/2021 22:05:34 - INFO - trainer -     acc = 0.8285714285714286
07/24/2021 22:05:34 - INFO - trainer -     f1 = 0.7328299738499956
07/24/2021 22:05:39 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:05:39 - INFO - trainer -     Num examples = 1120
07/24/2021

07/24/2021 22:09:06 - INFO - trainer -     acc = 0.8348214285714286
07/24/2021 22:09:06 - INFO - trainer -     f1 = 0.7582488304581623
07/24/2021 22:09:11 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:09:11 - INFO - trainer -     Num examples = 1120
07/24/2021 22:09:11 - INFO - trainer -     Batch size = 32
07/24/2021 22:09:12 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:09:12 - INFO - trainer -     eval loss: 1.240138909207183
07/24/2021 22:09:12 - INFO - trainer -     acc = 0.8401785714285714
07/24/2021 22:09:12 - INFO - trainer -     f1 = 0.7647035369572476

Epoch: 100%|██████████| 30/30 [07:28<00:00, 14.96s/it]
07/24/2021 22:09:16 - INFO - __main__ -     acc = 0.8508928571428571
07/24/2021 22:09:16 - INFO - __main__ -     f1 = 0.7739272259731999


### Layer 9 + Layer 8 FC1, Frozen 0.25

In [28]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 9
args.hidden_layer_b_no = 8
args.frozen_percent = 0.25

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 22:11:46 - INFO - model -   Hidden Layer 9 & 8, Frozen rate 0.25
07/24/2021 22:11:46 - INFO - trainer -   ***** Running training *****
07/24/2021 22:11:46 - INFO - trainer -     Num examples = 3602
07/24/2021 22:11:46 - INFO - trainer -     Num Epochs = 30
07/24/2021 22:11:46 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 22:11:46 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 22:11:46 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 22:11:51 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:11:51 - INFO - trainer -     Num examples = 1120
07/24/2021 22:11:51 - INFO - trainer -     Batch size = 32
07/24/2021 22:11:52 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:11:52 - INFO - trainer -     eval loss: 0.8205281206539699
07/24/2021 22:11:52 - INFO - trainer -     acc = 0.6705357142857142
07/24/2021 22:11:52 - INFO - trainer -     f1 = 0.4286

07/24/2021 22:13:27 - INFO - trainer -     acc = 0.8205357142857143
07/24/2021 22:13:27 - INFO - trainer -     f1 = 0.7274363010176254
07/24/2021 22:13:32 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:13:32 - INFO - trainer -     Num examples = 1120
07/24/2021 22:13:32 - INFO - trainer -     Batch size = 32
07/24/2021 22:13:33 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:13:33 - INFO - trainer -     eval loss: 0.9351661034310902
07/24/2021 22:13:33 - INFO - trainer -     acc = 0.8276785714285714
07/24/2021 22:13:33 - INFO - trainer -     f1 = 0.7323190491565996
07/24/2021 22:13:38 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:13:38 - INFO - trainer -     Num examples = 1120
07/24/2021 22:13:38 - INFO - trainer -     Batch size = 32
07/24/2021 22:13:40 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:13:40 - INFO - trainer -     eval loss: 0.9599471822314497
07/24/2021 22:13:40 - INFO - trainer -     acc = 0.8089285714285

07/24/2021 22:15:14 - INFO - trainer -     eval loss: 1.2100263414650856
07/24/2021 22:15:14 - INFO - trainer -     acc = 0.8357142857142857
07/24/2021 22:15:14 - INFO - trainer -     f1 = 0.7452992523751862

Epoch:  50%|█████     | 15/30 [03:31<03:29, 13.95s/it]07/24/2021 22:15:19 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:15:19 - INFO - trainer -     Num examples = 1120
07/24/2021 22:15:19 - INFO - trainer -     Batch size = 32
07/24/2021 22:15:20 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:15:20 - INFO - trainer -     eval loss: 1.4667669558382062
07/24/2021 22:15:20 - INFO - trainer -     acc = 0.8294642857142858
07/24/2021 22:15:20 - INFO - trainer -     f1 = 0.7348090311135561
07/24/2021 22:15:25 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:15:25 - INFO - trainer -     Num examples = 1120
07/24/2021 22:15:25 - INFO - trainer -     Batch size = 32
07/24/2021 22:15:26 - INFO - trainer -   ***** Eval results *****
07/24/202

07/24/2021 22:16:58 - INFO - trainer -     Num examples = 1120
07/24/2021 22:16:58 - INFO - trainer -     Batch size = 32
07/24/2021 22:16:59 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:16:59 - INFO - trainer -     eval loss: 1.603916642409084
07/24/2021 22:16:59 - INFO - trainer -     acc = 0.8267857142857142
07/24/2021 22:16:59 - INFO - trainer -     f1 = 0.7208637789472853
07/24/2021 22:17:04 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:17:04 - INFO - trainer -     Num examples = 1120
07/24/2021 22:17:04 - INFO - trainer -     Batch size = 32
07/24/2021 22:17:05 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:17:05 - INFO - trainer -     eval loss: 1.4838726303473646
07/24/2021 22:17:05 - INFO - trainer -     acc = 0.8196428571428571
07/24/2021 22:17:05 - INFO - trainer -     f1 = 0.7215338407287524

Epoch:  77%|███████▋  | 23/30 [05:24<01:37, 13.89s/it]07/24/2021 22:17:10 - INFO - trainer -   ***** Running evaluation *****
07/24/2021

07/24/2021 22:18:44 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:18:44 - INFO - trainer -     Num examples = 1120
07/24/2021 22:18:44 - INFO - trainer -     Batch size = 32
07/24/2021 22:18:45 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:18:45 - INFO - trainer -     eval loss: 1.833936360930758
07/24/2021 22:18:45 - INFO - trainer -     acc = 0.8178571428571428
07/24/2021 22:18:45 - INFO - trainer -     f1 = 0.7138334538325934

Epoch: 100%|██████████| 30/30 [07:02<00:00, 14.09s/it]
07/24/2021 22:18:49 - INFO - __main__ -     acc = 0.8464285714285714
07/24/2021 22:18:49 - INFO - __main__ -     f1 = 0.7779506251390128


### Layer 8 + Layer 7 FC1, Frozen 0.25

In [29]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 8
args.hidden_layer_b_no = 7
args.frozen_percent = 0.25

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 22:20:54 - INFO - model -   Hidden Layer 8 & 7, Frozen rate 0.25
07/24/2021 22:20:54 - INFO - trainer -   ***** Running training *****
07/24/2021 22:20:54 - INFO - trainer -     Num examples = 3602
07/24/2021 22:20:54 - INFO - trainer -     Num Epochs = 30
07/24/2021 22:20:54 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 22:20:54 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 22:20:54 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 22:20:59 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:20:59 - INFO - trainer -     Num examples = 1120
07/24/2021 22:20:59 - INFO - trainer -     Batch size = 32
07/24/2021 22:21:00 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:21:00 - INFO - trainer -     eval loss: 0.7931332179478237
07/24/2021 22:21:00 - INFO - trainer -     acc = 0.6973214285714285
07/24/2021 22:21:00 - INFO - trainer -     f1 = 0.4712

07/24/2021 22:22:32 - INFO - trainer -     acc = 0.8285714285714286
07/24/2021 22:22:32 - INFO - trainer -     f1 = 0.7407543160328443
07/24/2021 22:22:37 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:22:37 - INFO - trainer -     Num examples = 1120
07/24/2021 22:22:37 - INFO - trainer -     Batch size = 32
07/24/2021 22:22:38 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:22:38 - INFO - trainer -     eval loss: 1.026219939094569
07/24/2021 22:22:38 - INFO - trainer -     acc = 0.8258928571428571
07/24/2021 22:22:38 - INFO - trainer -     f1 = 0.728316041353347
07/24/2021 22:22:43 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:22:43 - INFO - trainer -     Num examples = 1120
07/24/2021 22:22:43 - INFO - trainer -     Batch size = 32
07/24/2021 22:22:44 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:22:44 - INFO - trainer -     eval loss: 0.9250396050379744
07/24/2021 22:22:44 - INFO - trainer -     acc = 0.832142857142857

07/24/2021 22:24:14 - INFO - trainer -     eval loss: 1.48727455107437
07/24/2021 22:24:14 - INFO - trainer -     acc = 0.8285714285714286
07/24/2021 22:24:14 - INFO - trainer -     f1 = 0.732155477966067

Epoch:  50%|█████     | 15/30 [03:24<03:19, 13.32s/it]07/24/2021 22:24:19 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:24:19 - INFO - trainer -     Num examples = 1120
07/24/2021 22:24:19 - INFO - trainer -     Batch size = 32
07/24/2021 22:24:20 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:24:20 - INFO - trainer -     eval loss: 1.5221318063752343
07/24/2021 22:24:20 - INFO - trainer -     acc = 0.8196428571428571
07/24/2021 22:24:20 - INFO - trainer -     f1 = 0.728169282124345
07/24/2021 22:24:25 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:24:25 - INFO - trainer -     Num examples = 1120
07/24/2021 22:24:25 - INFO - trainer -     Batch size = 32
07/24/2021 22:24:26 - INFO - trainer -   ***** Eval results *****
07/24/2021 22

07/24/2021 22:25:53 - INFO - trainer -     Num examples = 1120
07/24/2021 22:25:53 - INFO - trainer -     Batch size = 32
07/24/2021 22:25:54 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:25:54 - INFO - trainer -     eval loss: 1.5264014609781693
07/24/2021 22:25:54 - INFO - trainer -     acc = 0.8125
07/24/2021 22:25:54 - INFO - trainer -     f1 = 0.719866195781246
07/24/2021 22:25:59 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:25:59 - INFO - trainer -     Num examples = 1120
07/24/2021 22:25:59 - INFO - trainer -     Batch size = 32
07/24/2021 22:26:00 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:26:00 - INFO - trainer -     eval loss: 1.4147446608836098
07/24/2021 22:26:00 - INFO - trainer -     acc = 0.81875
07/24/2021 22:26:00 - INFO - trainer -     f1 = 0.7137963572018653

Epoch:  77%|███████▋  | 23/30 [05:10<01:32, 13.20s/it]07/24/2021 22:26:05 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:26:05 - INFO - trai

07/24/2021 22:27:34 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:27:34 - INFO - trainer -     Num examples = 1120
07/24/2021 22:27:34 - INFO - trainer -     Batch size = 32
07/24/2021 22:27:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:27:35 - INFO - trainer -     eval loss: 1.3971981121204278
07/24/2021 22:27:35 - INFO - trainer -     acc = 0.83125
07/24/2021 22:27:35 - INFO - trainer -     f1 = 0.7469202671169262

Epoch: 100%|██████████| 30/30 [06:44<00:00, 13.47s/it]
07/24/2021 22:27:38 - INFO - __main__ -     acc = 0.8330357142857143
07/24/2021 22:27:38 - INFO - __main__ -     f1 = 0.7421664856980942


### Layer 12 + Layer 11 FC1, Frozen 0.5

In [32]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 12
args.hidden_layer_b_no = 11
args.frozen_percent = 0.5

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 22:30:42 - INFO - model -   Hidden Layer 12 & 11, Frozen rate 0.5
07/24/2021 22:30:42 - INFO - trainer -   ***** Running training *****
07/24/2021 22:30:42 - INFO - trainer -     Num examples = 3602
07/24/2021 22:30:42 - INFO - trainer -     Num Epochs = 30
07/24/2021 22:30:42 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 22:30:42 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 22:30:42 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 22:30:47 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:30:47 - INFO - trainer -     Num examples = 1120
07/24/2021 22:30:47 - INFO - trainer -     Batch size = 32
07/24/2021 22:30:48 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:30:48 - INFO - trainer -     eval loss: 0.5833135251488004
07/24/2021 22:30:48 - INFO - trainer -     acc = 0.7758928571428572
07/24/2021 22:30:48 - INFO - trainer -     f1 = 0.617

07/24/2021 22:32:20 - INFO - trainer -     eval loss: 0.8402434687113522
07/24/2021 22:32:20 - INFO - trainer -     acc = 0.8357142857142857
07/24/2021 22:32:20 - INFO - trainer -     f1 = 0.7513259172082978
07/24/2021 22:32:25 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:32:25 - INFO - trainer -     Num examples = 1120
07/24/2021 22:32:25 - INFO - trainer -     Batch size = 32
07/24/2021 22:32:26 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:32:26 - INFO - trainer -     eval loss: 0.9405827657444336
07/24/2021 22:32:26 - INFO - trainer -     acc = 0.8294642857142858
07/24/2021 22:32:26 - INFO - trainer -     f1 = 0.7423314630350829
07/24/2021 22:32:31 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:32:31 - INFO - trainer -     Num examples = 1120
07/24/2021 22:32:31 - INFO - trainer -     Batch size = 32
07/24/2021 22:32:33 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:32:33 - INFO - trainer -     eval loss: 0.80202672

07/24/2021 22:34:03 - INFO - trainer -     Batch size = 32
07/24/2021 22:34:04 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:34:04 - INFO - trainer -     eval loss: 1.0614013450453057
07/24/2021 22:34:04 - INFO - trainer -     acc = 0.8419642857142857
07/24/2021 22:34:04 - INFO - trainer -     f1 = 0.7565851789873492

Epoch:  50%|█████     | 15/30 [03:26<03:23, 13.58s/it]07/24/2021 22:34:09 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:34:09 - INFO - trainer -     Num examples = 1120
07/24/2021 22:34:09 - INFO - trainer -     Batch size = 32
07/24/2021 22:34:10 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:34:10 - INFO - trainer -     eval loss: 1.1345409189151334
07/24/2021 22:34:10 - INFO - trainer -     acc = 0.8428571428571429
07/24/2021 22:34:10 - INFO - trainer -     f1 = 0.7631533609272364
07/24/2021 22:34:15 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:34:15 - INFO - trainer -     Num examples = 1120
07/24/202


Epoch:  73%|███████▎  | 22/30 [05:01<01:48, 13.53s/it]07/24/2021 22:35:45 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:35:45 - INFO - trainer -     Num examples = 1120
07/24/2021 22:35:45 - INFO - trainer -     Batch size = 32
07/24/2021 22:35:46 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:35:46 - INFO - trainer -     eval loss: 1.2690152496418574
07/24/2021 22:35:46 - INFO - trainer -     acc = 0.8419642857142857
07/24/2021 22:35:46 - INFO - trainer -     f1 = 0.7557349026201486
07/24/2021 22:35:51 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:35:51 - INFO - trainer -     Num examples = 1120
07/24/2021 22:35:51 - INFO - trainer -     Batch size = 32
07/24/2021 22:35:52 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:35:52 - INFO - trainer -     eval loss: 1.2102339987335393
07/24/2021 22:35:52 - INFO - trainer -     acc = 0.8366071428571429
07/24/2021 22:35:52 - INFO - trainer -     f1 = 0.7466560387919848

Epoch:  

07/24/2021 22:37:23 - INFO - trainer -     acc = 0.8375
07/24/2021 22:37:23 - INFO - trainer -     f1 = 0.7615506677676146
07/24/2021 22:37:28 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:37:28 - INFO - trainer -     Num examples = 1120
07/24/2021 22:37:28 - INFO - trainer -     Batch size = 32
07/24/2021 22:37:29 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:37:29 - INFO - trainer -     eval loss: 1.3330445527012802
07/24/2021 22:37:29 - INFO - trainer -     acc = 0.8375
07/24/2021 22:37:29 - INFO - trainer -     f1 = 0.7504924469807414

Epoch: 100%|██████████| 30/30 [06:50<00:00, 13.69s/it]
07/24/2021 22:37:33 - INFO - __main__ -     acc = 0.8482142857142857
07/24/2021 22:37:33 - INFO - __main__ -     f1 = 0.7653221449912606


### Layer 11 + Layer 10 FC1, Frozen 0.5

In [33]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 11
args.hidden_layer_b_no = 10
args.frozen_percent = 0.5

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 22:39:53 - INFO - model -   Hidden Layer 11 & 10, Frozen rate 0.5
07/24/2021 22:39:53 - INFO - trainer -   ***** Running training *****
07/24/2021 22:39:53 - INFO - trainer -     Num examples = 3602
07/24/2021 22:39:53 - INFO - trainer -     Num Epochs = 30
07/24/2021 22:39:53 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 22:39:53 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 22:39:53 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 22:39:57 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:39:57 - INFO - trainer -     Num examples = 1120
07/24/2021 22:39:57 - INFO - trainer -     Batch size = 32
07/24/2021 22:39:58 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:39:58 - INFO - trainer -     eval loss: 0.5779272117785045
07/24/2021 22:39:58 - INFO - trainer -     acc = 0.7830357142857143
07/24/2021 22:39:58 - INFO - trainer -     f1 = 0.618

07/24/2021 22:41:24 - INFO - trainer -     eval loss: 0.8520305435439306
07/24/2021 22:41:24 - INFO - trainer -     acc = 0.8401785714285714
07/24/2021 22:41:24 - INFO - trainer -     f1 = 0.7562885780991135
07/24/2021 22:41:29 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:41:29 - INFO - trainer -     Num examples = 1120
07/24/2021 22:41:29 - INFO - trainer -     Batch size = 32
07/24/2021 22:41:30 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:41:30 - INFO - trainer -     eval loss: 0.9368551102483512
07/24/2021 22:41:30 - INFO - trainer -     acc = 0.8366071428571429
07/24/2021 22:41:30 - INFO - trainer -     f1 = 0.7446966486153758
07/24/2021 22:41:35 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:41:35 - INFO - trainer -     Num examples = 1120
07/24/2021 22:41:35 - INFO - trainer -     Batch size = 32
07/24/2021 22:41:36 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:41:36 - INFO - trainer -     eval loss: 0.85523161

07/24/2021 22:43:01 - INFO - trainer -     Batch size = 32
07/24/2021 22:43:02 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:43:02 - INFO - trainer -     eval loss: 1.0067975380682452
07/24/2021 22:43:02 - INFO - trainer -     acc = 0.84375
07/24/2021 22:43:02 - INFO - trainer -     f1 = 0.7591118074374151

Epoch:  50%|█████     | 15/30 [03:13<03:11, 12.73s/it]07/24/2021 22:43:06 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:43:06 - INFO - trainer -     Num examples = 1120
07/24/2021 22:43:06 - INFO - trainer -     Batch size = 32
07/24/2021 22:43:07 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:43:07 - INFO - trainer -     eval loss: 0.8982494753681489
07/24/2021 22:43:07 - INFO - trainer -     acc = 0.8544642857142857
07/24/2021 22:43:07 - INFO - trainer -     f1 = 0.7855151947975063
07/24/2021 22:43:12 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:43:12 - INFO - trainer -     Num examples = 1120
07/24/2021 22:43:12 

07/24/2021 22:46:09 - INFO - trainer -     acc = 0.8517857142857143
07/24/2021 22:46:09 - INFO - trainer -     f1 = 0.7808184996095641
07/24/2021 22:46:14 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:46:14 - INFO - trainer -     Num examples = 1120
07/24/2021 22:46:14 - INFO - trainer -     Batch size = 32
07/24/2021 22:46:15 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:46:15 - INFO - trainer -     eval loss: 1.3025705974097945
07/24/2021 22:46:15 - INFO - trainer -     acc = 0.8508928571428571
07/24/2021 22:46:15 - INFO - trainer -     f1 = 0.7780735065511881

Epoch: 100%|██████████| 30/30 [06:25<00:00, 12.86s/it]
07/24/2021 22:46:18 - INFO - __main__ -     acc = 0.8607142857142858
07/24/2021 22:46:18 - INFO - __main__ -     f1 = 0.7963273935366959


### Layer 10 + Layer 9 FC1, Frozen 0.5

In [35]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 10
args.hidden_layer_b_no = 9
args.frozen_percent = 0.5

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 22:54:17 - INFO - model -   Hidden Layer 10 & 9, Frozen rate 0.5
07/24/2021 22:54:18 - INFO - trainer -   ***** Running training *****
07/24/2021 22:54:18 - INFO - trainer -     Num examples = 3602
07/24/2021 22:54:18 - INFO - trainer -     Num Epochs = 30
07/24/2021 22:54:18 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 22:54:18 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 22:54:18 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 22:54:22 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:54:22 - INFO - trainer -     Num examples = 1120
07/24/2021 22:54:22 - INFO - trainer -     Batch size = 32
07/24/2021 22:54:23 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:54:23 - INFO - trainer -     eval loss: 0.6801941641739436
07/24/2021 22:54:23 - INFO - trainer -     acc = 0.7589285714285714
07/24/2021 22:54:23 - INFO - trainer -     f1 = 0.5569

07/24/2021 22:55:44 - INFO - trainer -     acc = 0.8375
07/24/2021 22:55:44 - INFO - trainer -     f1 = 0.7614502875611379
07/24/2021 22:55:48 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:55:48 - INFO - trainer -     Num examples = 1120
07/24/2021 22:55:48 - INFO - trainer -     Batch size = 32
07/24/2021 22:55:49 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:55:49 - INFO - trainer -     eval loss: 0.8947362729946949
07/24/2021 22:55:49 - INFO - trainer -     acc = 0.8357142857142857
07/24/2021 22:55:49 - INFO - trainer -     f1 = 0.7490679159739183
07/24/2021 22:55:53 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:55:53 - INFO - trainer -     Num examples = 1120
07/24/2021 22:55:53 - INFO - trainer -     Batch size = 32
07/24/2021 22:55:54 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:55:54 - INFO - trainer -     eval loss: 0.7988020195426153
07/24/2021 22:55:54 - INFO - trainer -     acc = 0.8392857142857143
07/24/20

07/24/2021 22:57:14 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:57:14 - INFO - trainer -     eval loss: 1.1624142830110422
07/24/2021 22:57:14 - INFO - trainer -     acc = 0.83125
07/24/2021 22:57:14 - INFO - trainer -     f1 = 0.7271031319440562

Epoch:  50%|█████     | 15/30 [03:00<02:57, 11.85s/it]07/24/2021 22:57:18 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:57:18 - INFO - trainer -     Num examples = 1120
07/24/2021 22:57:18 - INFO - trainer -     Batch size = 32
07/24/2021 22:57:20 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:57:20 - INFO - trainer -     eval loss: 1.0686553055969332
07/24/2021 22:57:20 - INFO - trainer -     acc = 0.8339285714285715
07/24/2021 22:57:20 - INFO - trainer -     f1 = 0.748487179885117
07/24/2021 22:57:24 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:57:24 - INFO - trainer -     Num examples = 1120
07/24/2021 22:57:24 - INFO - trainer -     Batch size = 32
07/24/2021 22:57:25 -

07/24/2021 22:58:44 - INFO - trainer -     Num examples = 1120
07/24/2021 22:58:44 - INFO - trainer -     Batch size = 32
07/24/2021 22:58:45 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:58:45 - INFO - trainer -     eval loss: 1.195826613068805
07/24/2021 22:58:45 - INFO - trainer -     acc = 0.8285714285714286
07/24/2021 22:58:45 - INFO - trainer -     f1 = 0.7484270630688817
07/24/2021 22:58:49 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 22:58:49 - INFO - trainer -     Num examples = 1120
07/24/2021 22:58:49 - INFO - trainer -     Batch size = 32
07/24/2021 22:58:50 - INFO - trainer -   ***** Eval results *****
07/24/2021 22:58:50 - INFO - trainer -     eval loss: 1.3821197467032886
07/24/2021 22:58:50 - INFO - trainer -     acc = 0.8366071428571429
07/24/2021 22:58:50 - INFO - trainer -     f1 = 0.7472857045390467

Epoch:  77%|███████▋  | 23/30 [04:36<01:23, 11.93s/it]07/24/2021 22:58:55 - INFO - trainer -   ***** Running evaluation *****
07/24/2021

07/24/2021 23:00:10 - INFO - trainer -     f1 = 0.73926933574038
07/24/2021 23:00:14 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:00:14 - INFO - trainer -     Num examples = 1120
07/24/2021 23:00:14 - INFO - trainer -     Batch size = 32
07/24/2021 23:00:15 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:00:15 - INFO - trainer -     eval loss: 1.5502438219413306
07/24/2021 23:00:15 - INFO - trainer -     acc = 0.8357142857142857
07/24/2021 23:00:15 - INFO - trainer -     f1 = 0.7512047356711142

Epoch: 100%|██████████| 30/30 [06:00<00:00, 12.02s/it]
07/24/2021 23:00:18 - INFO - __main__ -     acc = 0.8464285714285714
07/24/2021 23:00:18 - INFO - __main__ -     f1 = 0.7689448678540254


### Layer 9 + Layer 8 FC1, Frozen 0.5

In [36]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 9
args.hidden_layer_b_no = 8
args.frozen_percent = 0.5

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 23:00:51 - INFO - model -   Hidden Layer 9 & 8, Frozen rate 0.5
07/24/2021 23:00:51 - INFO - trainer -   ***** Running training *****
07/24/2021 23:00:51 - INFO - trainer -     Num examples = 3602
07/24/2021 23:00:51 - INFO - trainer -     Num Epochs = 30
07/24/2021 23:00:51 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 23:00:51 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 23:00:51 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 23:00:54 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:00:54 - INFO - trainer -     Num examples = 1120
07/24/2021 23:00:54 - INFO - trainer -     Batch size = 32
07/24/2021 23:00:56 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:00:56 - INFO - trainer -     eval loss: 0.821046748331615
07/24/2021 23:00:56 - INFO - trainer -     acc = 0.65625
07/24/2021 23:00:56 - INFO - trainer -     f1 = 0.2960693213189995


07/24/2021 23:02:10 - INFO - trainer -     eval loss: 0.8451005252078175
07/24/2021 23:02:10 - INFO - trainer -     acc = 0.8142857142857143
07/24/2021 23:02:10 - INFO - trainer -     f1 = 0.707341494674655
07/24/2021 23:02:14 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:02:14 - INFO - trainer -     Num examples = 1120
07/24/2021 23:02:14 - INFO - trainer -     Batch size = 32
07/24/2021 23:02:15 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:02:15 - INFO - trainer -     eval loss: 0.7865543184136706
07/24/2021 23:02:15 - INFO - trainer -     acc = 0.8285714285714286
07/24/2021 23:02:15 - INFO - trainer -     f1 = 0.7398422371631828
07/24/2021 23:02:19 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:02:19 - INFO - trainer -     Num examples = 1120
07/24/2021 23:02:19 - INFO - trainer -     Batch size = 32
07/24/2021 23:02:20 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:02:20 - INFO - trainer -     eval loss: 0.771198801

07/24/2021 23:03:33 - INFO - trainer -     Batch size = 32
07/24/2021 23:03:34 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:03:34 - INFO - trainer -     eval loss: 1.0182226576114772
07/24/2021 23:03:34 - INFO - trainer -     acc = 0.83125
07/24/2021 23:03:34 - INFO - trainer -     f1 = 0.7507217486808216

Epoch:  50%|█████     | 15/30 [02:46<02:44, 10.99s/it]07/24/2021 23:03:38 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:03:38 - INFO - trainer -     Num examples = 1120
07/24/2021 23:03:38 - INFO - trainer -     Batch size = 32
07/24/2021 23:03:39 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:03:39 - INFO - trainer -     eval loss: 1.1506645911215205
07/24/2021 23:03:39 - INFO - trainer -     acc = 0.8303571428571429
07/24/2021 23:03:39 - INFO - trainer -     f1 = 0.7316083297292192
07/24/2021 23:03:43 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:03:43 - INFO - trainer -     Num examples = 1120
07/24/2021 23:03:43 

07/24/2021 23:06:16 - INFO - trainer -     acc = 0.8366071428571429
07/24/2021 23:06:16 - INFO - trainer -     f1 = 0.7489205150109819
07/24/2021 23:06:20 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:06:20 - INFO - trainer -     Num examples = 1120
07/24/2021 23:06:20 - INFO - trainer -     Batch size = 32
07/24/2021 23:06:21 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:06:21 - INFO - trainer -     eval loss: 1.4529979512595934
07/24/2021 23:06:21 - INFO - trainer -     acc = 0.8241071428571428
07/24/2021 23:06:21 - INFO - trainer -     f1 = 0.7194205014487146

Epoch: 100%|██████████| 30/30 [05:33<00:00, 11.12s/it]
07/24/2021 23:06:24 - INFO - __main__ -     acc = 0.8455357142857143
07/24/2021 23:06:24 - INFO - __main__ -     f1 = 0.7703085859359854


### Layer 8 + Layer 7 FC1, Frozen 0.5

In [37]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 8
args.hidden_layer_b_no = 7
args.frozen_percent = 0.5

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 23:06:49 - INFO - model -   Hidden Layer 8 & 7, Frozen rate 0.5
07/24/2021 23:06:50 - INFO - trainer -   ***** Running training *****
07/24/2021 23:06:50 - INFO - trainer -     Num examples = 3602
07/24/2021 23:06:50 - INFO - trainer -     Num Epochs = 30
07/24/2021 23:06:50 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 23:06:50 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 23:06:50 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 23:06:53 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:06:53 - INFO - trainer -     Num examples = 1120
07/24/2021 23:06:53 - INFO - trainer -     Batch size = 32
07/24/2021 23:06:54 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:06:54 - INFO - trainer -     eval loss: 0.8616397483008248
07/24/2021 23:06:54 - INFO - trainer -     acc = 0.65
07/24/2021 23:06:54 - INFO - trainer -     f1 = 0.26262626262626265
0

07/24/2021 23:08:03 - INFO - trainer -     acc = 0.7982142857142858
07/24/2021 23:08:03 - INFO - trainer -     f1 = 0.6931435725850433
07/24/2021 23:08:06 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:08:06 - INFO - trainer -     Num examples = 1120
07/24/2021 23:08:06 - INFO - trainer -     Batch size = 32
07/24/2021 23:08:07 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:08:07 - INFO - trainer -     eval loss: 0.6558181234768459
07/24/2021 23:08:07 - INFO - trainer -     acc = 0.8017857142857143
07/24/2021 23:08:07 - INFO - trainer -     f1 = 0.7117484879307914
07/24/2021 23:08:11 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:08:11 - INFO - trainer -     Num examples = 1120
07/24/2021 23:08:11 - INFO - trainer -     Batch size = 32
07/24/2021 23:08:12 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:08:12 - INFO - trainer -     eval loss: 0.6067730703524181
07/24/2021 23:08:12 - INFO - trainer -     acc = 0.80625
07/24/2

07/24/2021 23:09:21 - INFO - trainer -     eval loss: 1.0395956546600376
07/24/2021 23:09:21 - INFO - trainer -     acc = 0.8053571428571429
07/24/2021 23:09:21 - INFO - trainer -     f1 = 0.7026929699745924

Epoch:  50%|█████     | 15/30 [02:34<02:32, 10.19s/it]07/24/2021 23:09:24 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:09:24 - INFO - trainer -     Num examples = 1120
07/24/2021 23:09:24 - INFO - trainer -     Batch size = 32
07/24/2021 23:09:25 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:09:25 - INFO - trainer -     eval loss: 1.1482549310894683
07/24/2021 23:09:25 - INFO - trainer -     acc = 0.8044642857142857
07/24/2021 23:09:25 - INFO - trainer -     f1 = 0.6970119355138951
07/24/2021 23:09:29 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:09:29 - INFO - trainer -     Num examples = 1120
07/24/2021 23:09:29 - INFO - trainer -     Batch size = 32
07/24/2021 23:09:30 - INFO - trainer -   ***** Eval results *****
07/24/202

07/24/2021 23:10:37 - INFO - trainer -     Batch size = 32
07/24/2021 23:10:38 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:10:38 - INFO - trainer -     eval loss: 1.3978160900729044
07/24/2021 23:10:38 - INFO - trainer -     acc = 0.7866071428571428
07/24/2021 23:10:38 - INFO - trainer -     f1 = 0.6812949110798764
07/24/2021 23:10:42 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:10:42 - INFO - trainer -     Num examples = 1120
07/24/2021 23:10:42 - INFO - trainer -     Batch size = 32
07/24/2021 23:10:43 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:10:43 - INFO - trainer -     eval loss: 1.500273464087929
07/24/2021 23:10:43 - INFO - trainer -     acc = 0.7973214285714286
07/24/2021 23:10:43 - INFO - trainer -     f1 = 0.6852411477411477

Epoch:  77%|███████▋  | 23/30 [03:56<01:11, 10.15s/it]07/24/2021 23:10:46 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:10:46 - INFO - trainer -     Num examples = 1120
07/24/2021

07/24/2021 23:11:55 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:11:55 - INFO - trainer -     Num examples = 1120
07/24/2021 23:11:55 - INFO - trainer -     Batch size = 32
07/24/2021 23:11:56 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:11:56 - INFO - trainer -     eval loss: 1.645246412439987
07/24/2021 23:11:56 - INFO - trainer -     acc = 0.8098214285714286
07/24/2021 23:11:56 - INFO - trainer -     f1 = 0.6944086224905212

Epoch: 100%|██████████| 30/30 [05:09<00:00, 10.30s/it]
07/24/2021 23:11:59 - INFO - __main__ -     acc = 0.8196428571428571
07/24/2021 23:11:59 - INFO - __main__ -     f1 = 0.7251713211996168


### Layer 12 + Layer 11 FC1, Frozen 0.75

In [38]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 12
args.hidden_layer_b_no = 11
args.frozen_percent = 0.75

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 23:12:30 - INFO - model -   Hidden Layer 12 & 11, Frozen rate 0.75
07/24/2021 23:12:30 - INFO - trainer -   ***** Running training *****
07/24/2021 23:12:30 - INFO - trainer -     Num examples = 3602
07/24/2021 23:12:30 - INFO - trainer -     Num Epochs = 30
07/24/2021 23:12:30 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 23:12:30 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 23:12:30 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 23:12:33 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:12:33 - INFO - trainer -     Num examples = 1120
07/24/2021 23:12:33 - INFO - trainer -     Batch size = 32
07/24/2021 23:12:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:12:35 - INFO - trainer -     eval loss: 0.6667796518121447
07/24/2021 23:12:35 - INFO - trainer -     acc = 0.7348214285714286
07/24/2021 23:12:35 - INFO - trainer -     f1 = 0.50

07/24/2021 23:13:48 - INFO - trainer -     eval loss: 0.5944566315173039
07/24/2021 23:13:48 - INFO - trainer -     acc = 0.84375
07/24/2021 23:13:48 - INFO - trainer -     f1 = 0.7731899772780478
07/24/2021 23:13:51 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:13:51 - INFO - trainer -     Num examples = 1120
07/24/2021 23:13:51 - INFO - trainer -     Batch size = 32
07/24/2021 23:13:53 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:13:53 - INFO - trainer -     eval loss: 0.6894874089197921
07/24/2021 23:13:53 - INFO - trainer -     acc = 0.8455357142857143
07/24/2021 23:13:53 - INFO - trainer -     f1 = 0.7717474001630618
07/24/2021 23:13:56 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:13:56 - INFO - trainer -     Num examples = 1120
07/24/2021 23:13:56 - INFO - trainer -     Batch size = 32
07/24/2021 23:13:57 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:13:57 - INFO - trainer -     eval loss: 0.6835149324366024
07

07/24/2021 23:15:10 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:15:10 - INFO - trainer -     eval loss: 0.9241787436152143
07/24/2021 23:15:10 - INFO - trainer -     acc = 0.8276785714285714
07/24/2021 23:15:10 - INFO - trainer -     f1 = 0.7493567566272188

Epoch:  50%|█████     | 15/30 [02:43<02:42, 10.83s/it]07/24/2021 23:15:14 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:15:14 - INFO - trainer -     Num examples = 1120
07/24/2021 23:15:14 - INFO - trainer -     Batch size = 32
07/24/2021 23:15:15 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:15:15 - INFO - trainer -     eval loss: 1.0217643081509908
07/24/2021 23:15:15 - INFO - trainer -     acc = 0.8241071428571428
07/24/2021 23:15:15 - INFO - trainer -     f1 = 0.7393730465966003
07/24/2021 23:15:19 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:15:19 - INFO - trainer -     Num examples = 1120
07/24/2021 23:15:19 - INFO - trainer -     Batch size = 32
07/24/202

07/24/2021 23:17:51 - INFO - trainer -     eval loss: 1.3074995767714945
07/24/2021 23:17:51 - INFO - trainer -     acc = 0.8446428571428571
07/24/2021 23:17:51 - INFO - trainer -     f1 = 0.7609980657967305
07/24/2021 23:17:55 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:17:55 - INFO - trainer -     Num examples = 1120
07/24/2021 23:17:55 - INFO - trainer -     Batch size = 32
07/24/2021 23:17:56 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:17:56 - INFO - trainer -     eval loss: 1.2723460447721924
07/24/2021 23:17:56 - INFO - trainer -     acc = 0.8473214285714286
07/24/2021 23:17:56 - INFO - trainer -     f1 = 0.7721698695503844

Epoch: 100%|██████████| 30/30 [05:29<00:00, 10.98s/it]
07/24/2021 23:17:59 - INFO - __main__ -     acc = 0.8508928571428571
07/24/2021 23:17:59 - INFO - __main__ -     f1 = 0.7836285950931897


### Layer 11 + Layer 10 FC1, Frozen 0.75

In [39]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 11
args.hidden_layer_b_no = 10
args.frozen_percent = 0.75

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 23:18:30 - INFO - model -   Hidden Layer 11 & 10, Frozen rate 0.75
07/24/2021 23:18:30 - INFO - trainer -   ***** Running training *****
07/24/2021 23:18:30 - INFO - trainer -     Num examples = 3602
07/24/2021 23:18:30 - INFO - trainer -     Num Epochs = 30
07/24/2021 23:18:30 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 23:18:30 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 23:18:30 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 23:18:34 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:18:34 - INFO - trainer -     Num examples = 1120
07/24/2021 23:18:34 - INFO - trainer -     Batch size = 32
07/24/2021 23:18:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:18:35 - INFO - trainer -     eval loss: 0.7317614189216068
07/24/2021 23:18:35 - INFO - trainer -     acc = 0.6946428571428571
07/24/2021 23:18:35 - INFO - trainer -     f1 = 0.40

07/24/2021 23:19:43 - INFO - trainer -     eval loss: 0.5763177951797843
07/24/2021 23:19:43 - INFO - trainer -     acc = 0.8196428571428571
07/24/2021 23:19:43 - INFO - trainer -     f1 = 0.7341870997324538
07/24/2021 23:19:46 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:19:46 - INFO - trainer -     Num examples = 1120
07/24/2021 23:19:46 - INFO - trainer -     Batch size = 32
07/24/2021 23:19:48 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:19:48 - INFO - trainer -     eval loss: 0.6656088952706861
07/24/2021 23:19:48 - INFO - trainer -     acc = 0.8267857142857142
07/24/2021 23:19:48 - INFO - trainer -     f1 = 0.7358834735980723
07/24/2021 23:19:51 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:19:51 - INFO - trainer -     Num examples = 1120
07/24/2021 23:19:51 - INFO - trainer -     Batch size = 32
07/24/2021 23:19:52 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:19:52 - INFO - trainer -     eval loss: 0.65717013

07/24/2021 23:20:59 - INFO - trainer -     Batch size = 32
07/24/2021 23:21:00 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:21:00 - INFO - trainer -     eval loss: 0.9405543126964143
07/24/2021 23:21:00 - INFO - trainer -     acc = 0.8205357142857143
07/24/2021 23:21:00 - INFO - trainer -     f1 = 0.7385021217772212

Epoch:  50%|█████     | 15/30 [02:32<02:30, 10.04s/it]07/24/2021 23:21:03 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:21:03 - INFO - trainer -     Num examples = 1120
07/24/2021 23:21:03 - INFO - trainer -     Batch size = 32
07/24/2021 23:21:04 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:21:04 - INFO - trainer -     eval loss: 0.9832452282974763
07/24/2021 23:21:04 - INFO - trainer -     acc = 0.8276785714285714
07/24/2021 23:21:04 - INFO - trainer -     f1 = 0.734453283256394
07/24/2021 23:21:08 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:21:08 - INFO - trainer -     Num examples = 1120
07/24/2021


Epoch:  73%|███████▎  | 22/30 [03:43<01:20, 10.05s/it]07/24/2021 23:22:15 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:22:15 - INFO - trainer -     Num examples = 1120
07/24/2021 23:22:15 - INFO - trainer -     Batch size = 32
07/24/2021 23:22:16 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:22:16 - INFO - trainer -     eval loss: 1.2517426084891798
07/24/2021 23:22:16 - INFO - trainer -     acc = 0.8285714285714286
07/24/2021 23:22:16 - INFO - trainer -     f1 = 0.7268376149034337
07/24/2021 23:22:19 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:22:19 - INFO - trainer -     Num examples = 1120
07/24/2021 23:22:19 - INFO - trainer -     Batch size = 32
07/24/2021 23:22:21 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:22:21 - INFO - trainer -     eval loss: 1.3476056665105196
07/24/2021 23:22:21 - INFO - trainer -     acc = 0.8160714285714286
07/24/2021 23:22:21 - INFO - trainer -     f1 = 0.7047151482524322

Epoch:  

07/24/2021 23:23:28 - INFO - trainer -     acc = 0.8339285714285715
07/24/2021 23:23:28 - INFO - trainer -     f1 = 0.7466720213723018
07/24/2021 23:23:31 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:23:31 - INFO - trainer -     Num examples = 1120
07/24/2021 23:23:31 - INFO - trainer -     Batch size = 32
07/24/2021 23:23:32 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:23:32 - INFO - trainer -     eval loss: 1.3453305513934797
07/24/2021 23:23:32 - INFO - trainer -     acc = 0.8375
07/24/2021 23:23:32 - INFO - trainer -     f1 = 0.7526989700970317

Epoch: 100%|██████████| 30/30 [05:04<00:00, 10.15s/it]
07/24/2021 23:23:35 - INFO - __main__ -     acc = 0.8383928571428572
07/24/2021 23:23:35 - INFO - __main__ -     f1 = 0.7541597272016869


### Layer 10 + Layer 9 FC1, Frozen 0.75

In [40]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 10
args.hidden_layer_b_no = 9
args.frozen_percent = 0.75

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 23:23:41 - INFO - model -   Hidden Layer 10 & 9, Frozen rate 0.75
07/24/2021 23:23:41 - INFO - trainer -   ***** Running training *****
07/24/2021 23:23:41 - INFO - trainer -     Num examples = 3602
07/24/2021 23:23:41 - INFO - trainer -     Num Epochs = 30
07/24/2021 23:23:41 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 23:23:41 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 23:23:41 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 23:23:44 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:23:44 - INFO - trainer -     Num examples = 1120
07/24/2021 23:23:44 - INFO - trainer -     Batch size = 32
07/24/2021 23:23:45 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:23:45 - INFO - trainer -     eval loss: 0.8990142822265625
07/24/2021 23:23:45 - INFO - trainer -     acc = 0.65
07/24/2021 23:23:45 - INFO - trainer -     f1 = 0.29272050671409583

07/24/2021 23:24:47 - INFO - trainer -     eval loss: 0.5252488777041435
07/24/2021 23:24:47 - INFO - trainer -     acc = 0.8232142857142857
07/24/2021 23:24:47 - INFO - trainer -     f1 = 0.727957751474604
07/24/2021 23:24:50 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:24:50 - INFO - trainer -     Num examples = 1120
07/24/2021 23:24:50 - INFO - trainer -     Batch size = 32
07/24/2021 23:24:51 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:24:51 - INFO - trainer -     eval loss: 0.5372289126472813
07/24/2021 23:24:51 - INFO - trainer -     acc = 0.8241071428571428
07/24/2021 23:24:51 - INFO - trainer -     f1 = 0.734081933288688
07/24/2021 23:24:54 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:24:54 - INFO - trainer -     Num examples = 1120
07/24/2021 23:24:54 - INFO - trainer -     Batch size = 32
07/24/2021 23:24:55 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:24:55 - INFO - trainer -     eval loss: 0.5548030412

07/24/2021 23:25:56 - INFO - trainer -     Batch size = 32
07/24/2021 23:25:58 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:25:58 - INFO - trainer -     eval loss: 0.8640542699290173
07/24/2021 23:25:58 - INFO - trainer -     acc = 0.8214285714285714
07/24/2021 23:25:58 - INFO - trainer -     f1 = 0.7218832059826205

Epoch:  50%|█████     | 15/30 [02:19<02:18,  9.25s/it]07/24/2021 23:26:01 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:26:01 - INFO - trainer -     Num examples = 1120
07/24/2021 23:26:01 - INFO - trainer -     Batch size = 32
07/24/2021 23:26:02 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:26:02 - INFO - trainer -     eval loss: 0.7872751233833176
07/24/2021 23:26:02 - INFO - trainer -     acc = 0.8089285714285714
07/24/2021 23:26:02 - INFO - trainer -     f1 = 0.7119461468388509
07/24/2021 23:26:05 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:26:05 - INFO - trainer -     Num examples = 1120
07/24/202


Epoch:  73%|███████▎  | 22/30 [03:25<01:14,  9.33s/it]07/24/2021 23:27:07 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:27:07 - INFO - trainer -     Num examples = 1120
07/24/2021 23:27:07 - INFO - trainer -     Batch size = 32
07/24/2021 23:27:08 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:27:08 - INFO - trainer -     eval loss: 1.0314527549913952
07/24/2021 23:27:08 - INFO - trainer -     acc = 0.8026785714285715
07/24/2021 23:27:08 - INFO - trainer -     f1 = 0.7120576915495435
07/24/2021 23:27:11 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:27:11 - INFO - trainer -     Num examples = 1120
07/24/2021 23:27:11 - INFO - trainer -     Batch size = 32
07/24/2021 23:27:13 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:27:13 - INFO - trainer -     eval loss: 1.0514742031693458
07/24/2021 23:27:13 - INFO - trainer -     acc = 0.8125
07/24/2021 23:27:13 - INFO - trainer -     f1 = 0.7164838336919329

Epoch:  77%|███████▋

07/24/2021 23:28:15 - INFO - trainer -     acc = 0.8071428571428572
07/24/2021 23:28:15 - INFO - trainer -     f1 = 0.7061049667643066
07/24/2021 23:28:18 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:28:18 - INFO - trainer -     Num examples = 1120
07/24/2021 23:28:18 - INFO - trainer -     Batch size = 32
07/24/2021 23:28:19 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:28:19 - INFO - trainer -     eval loss: 1.3101934237139565
07/24/2021 23:28:19 - INFO - trainer -     acc = 0.8080357142857143
07/24/2021 23:28:19 - INFO - trainer -     f1 = 0.7070095162724592

Epoch: 100%|██████████| 30/30 [04:40<00:00,  9.34s/it]
07/24/2021 23:28:21 - INFO - __main__ -     acc = 0.8303571428571429
07/24/2021 23:28:21 - INFO - __main__ -     f1 = 0.7424125253207438


### Layer 9 + Layer 8 FC1, Frozen 0.75

In [41]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 9
args.hidden_layer_b_no = 8
args.frozen_percent = 0.75

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 23:28:27 - INFO - model -   Hidden Layer 9 & 8, Frozen rate 0.75
07/24/2021 23:28:27 - INFO - trainer -   ***** Running training *****
07/24/2021 23:28:27 - INFO - trainer -     Num examples = 3602
07/24/2021 23:28:27 - INFO - trainer -     Num Epochs = 30
07/24/2021 23:28:27 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 23:28:27 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 23:28:27 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 23:28:30 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:28:30 - INFO - trainer -     Num examples = 1120
07/24/2021 23:28:30 - INFO - trainer -     Batch size = 32
07/24/2021 23:28:31 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:28:31 - INFO - trainer -     eval loss: 0.9162766116006034
07/24/2021 23:28:31 - INFO - trainer -     acc = 0.65
07/24/2021 23:28:31 - INFO - trainer -     f1 = 0.26262626262626265


07/24/2021 23:29:28 - INFO - trainer -     eval loss: 0.7998607626983097
07/24/2021 23:29:28 - INFO - trainer -     acc = 0.6732142857142858
07/24/2021 23:29:28 - INFO - trainer -     f1 = 0.36845687716098324
07/24/2021 23:29:30 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:29:30 - INFO - trainer -     Num examples = 1120
07/24/2021 23:29:30 - INFO - trainer -     Batch size = 32
07/24/2021 23:29:31 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:29:31 - INFO - trainer -     eval loss: 0.8059197493961879
07/24/2021 23:29:31 - INFO - trainer -     acc = 0.6830357142857143
07/24/2021 23:29:31 - INFO - trainer -     f1 = 0.43120496061672525
07/24/2021 23:29:34 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:29:34 - INFO - trainer -     Num examples = 1120
07/24/2021 23:29:34 - INFO - trainer -     Batch size = 32
07/24/2021 23:29:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:29:35 - INFO - trainer -     eval loss: 0.791896

07/24/2021 23:30:30 - INFO - trainer -     Batch size = 32
07/24/2021 23:30:31 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:30:31 - INFO - trainer -     eval loss: 0.7564848806176867
07/24/2021 23:30:31 - INFO - trainer -     acc = 0.6821428571428572
07/24/2021 23:30:31 - INFO - trainer -     f1 = 0.40449394154773904

Epoch:  50%|█████     | 15/30 [02:06<02:04,  8.30s/it]07/24/2021 23:30:34 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:30:34 - INFO - trainer -     Num examples = 1120
07/24/2021 23:30:34 - INFO - trainer -     Batch size = 32
07/24/2021 23:30:35 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:30:35 - INFO - trainer -     eval loss: 0.7620581013815744
07/24/2021 23:30:35 - INFO - trainer -     acc = 0.6875
07/24/2021 23:30:35 - INFO - trainer -     f1 = 0.4319311642160962
07/24/2021 23:30:37 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:30:37 - INFO - trainer -     Num examples = 1120
07/24/2021 23:30:37 

07/24/2021 23:32:35 - INFO - trainer -     eval loss: 0.7379787687744414
07/24/2021 23:32:35 - INFO - trainer -     acc = 0.6892857142857143
07/24/2021 23:32:35 - INFO - trainer -     f1 = 0.44726485174754416
07/24/2021 23:32:38 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:32:38 - INFO - trainer -     Num examples = 1120
07/24/2021 23:32:38 - INFO - trainer -     Batch size = 32
07/24/2021 23:32:39 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:32:39 - INFO - trainer -     eval loss: 0.730727595942361
07/24/2021 23:32:39 - INFO - trainer -     acc = 0.7035714285714286
07/24/2021 23:32:39 - INFO - trainer -     f1 = 0.4790996161164907

Epoch: 100%|██████████| 30/30 [04:13<00:00,  8.46s/it]
07/24/2021 23:32:41 - INFO - __main__ -     acc = 0.7080357142857143
07/24/2021 23:32:41 - INFO - __main__ -     f1 = 0.5026585811580803


### Layer 8 + Layer 7 FC1, Frozen 0.75

In [42]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 8
args.hidden_layer_b_no = 7
args.frozen_percent = 0.75

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/24/2021 23:32:48 - INFO - model -   Hidden Layer 8 & 7, Frozen rate 0.75
07/24/2021 23:32:48 - INFO - trainer -   ***** Running training *****
07/24/2021 23:32:48 - INFO - trainer -     Num examples = 3602
07/24/2021 23:32:48 - INFO - trainer -     Num Epochs = 30
07/24/2021 23:32:48 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/24/2021 23:32:48 - INFO - trainer -     Gradient Accumulation steps = 2
07/24/2021 23:32:48 - INFO - trainer -     Total optimization steps = 3390

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/24/2021 23:32:50 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:32:50 - INFO - trainer -     Num examples = 1120
07/24/2021 23:32:50 - INFO - trainer -     Batch size = 32
07/24/2021 23:32:52 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:32:52 - INFO - trainer -     eval loss: 0.915480967930385
07/24/2021 23:32:52 - INFO - trainer -     acc = 0.65
07/24/2021 23:32:52 - INFO - trainer -     f1 = 0.26262626262626265
0

07/24/2021 23:33:47 - INFO - trainer -     acc = 0.6696428571428571
07/24/2021 23:33:47 - INFO - trainer -     f1 = 0.3562471705833168
07/24/2021 23:33:50 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:33:50 - INFO - trainer -     Num examples = 1120
07/24/2021 23:33:50 - INFO - trainer -     Batch size = 32
07/24/2021 23:33:51 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:33:51 - INFO - trainer -     eval loss: 0.7991017069135393
07/24/2021 23:33:51 - INFO - trainer -     acc = 0.6883928571428571
07/24/2021 23:33:51 - INFO - trainer -     f1 = 0.4367644320732915
07/24/2021 23:33:53 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:33:53 - INFO - trainer -     Num examples = 1120
07/24/2021 23:33:53 - INFO - trainer -     Batch size = 32
07/24/2021 23:33:55 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:33:55 - INFO - trainer -     eval loss: 0.7847471330847059
07/24/2021 23:33:55 - INFO - trainer -     acc = 0.6669642857142

07/24/2021 23:34:50 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:34:50 - INFO - trainer -     eval loss: 0.7455326003687722
07/24/2021 23:34:50 - INFO - trainer -     acc = 0.6883928571428571
07/24/2021 23:34:50 - INFO - trainer -     f1 = 0.41950359773255613

Epoch:  50%|█████     | 15/30 [02:04<02:03,  8.23s/it]07/24/2021 23:34:53 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:34:53 - INFO - trainer -     Num examples = 1120
07/24/2021 23:34:53 - INFO - trainer -     Batch size = 32
07/24/2021 23:34:54 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:34:54 - INFO - trainer -     eval loss: 0.7483209209782736
07/24/2021 23:34:54 - INFO - trainer -     acc = 0.6955357142857143
07/24/2021 23:34:54 - INFO - trainer -     f1 = 0.458432800324998
07/24/2021 23:34:57 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:34:57 - INFO - trainer -     Num examples = 1120
07/24/2021 23:34:57 - INFO - trainer -     Batch size = 32
07/24/202

07/24/2021 23:36:53 - INFO - trainer -     acc = 0.6946428571428571
07/24/2021 23:36:53 - INFO - trainer -     f1 = 0.45823628171602365
07/24/2021 23:36:55 - INFO - trainer -   ***** Running evaluation *****
07/24/2021 23:36:55 - INFO - trainer -     Num examples = 1120
07/24/2021 23:36:55 - INFO - trainer -     Batch size = 32
07/24/2021 23:36:56 - INFO - trainer -   ***** Eval results *****
07/24/2021 23:36:56 - INFO - trainer -     eval loss: 0.716807347536087
07/24/2021 23:36:56 - INFO - trainer -     acc = 0.7035714285714286
07/24/2021 23:36:56 - INFO - trainer -     f1 = 0.4835242559890969

Epoch: 100%|██████████| 30/30 [04:10<00:00,  8.35s/it]
07/24/2021 23:36:58 - INFO - __main__ -     acc = 0.7107142857142857
07/24/2021 23:36:58 - INFO - __main__ -     f1 = 0.5126850039490402


### Layer 12 + Layer 10 FC1, Frozen 0

In [6]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 12
args.hidden_layer_b_no = 10
args.frozen_percent = 0

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/25/2021 17:03:24 - INFO - model -   Hidden Layer 12 & 10, Frozen rate 0
07/25/2021 17:03:26 - INFO - trainer -   ***** Running training *****
07/25/2021 17:03:26 - INFO - trainer -     Num examples = 3602
07/25/2021 17:03:26 - INFO - trainer -     Num Epochs = 30
07/25/2021 17:03:26 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/25/2021 17:03:26 - INFO - trainer -     Gradient Accumulation steps = 2
07/25/2021 17:03:26 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/25/2021 17:03:34 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:03:34 - INFO - trainer -     Num examples = 1120
07/25/2021 17:03:34 - INFO - trainer -     Batch size = 32
07/25/2021 17:03:35 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:03:35 - INFO - trainer -     eval loss: 0.5740665503910609
07/25/2021 17:03:35 - INFO - trainer -     acc = 0.7669642857142858
07/25/2021 17:03:35 - INFO - trainer -     f1 = 0.663219

07/25/2021 17:05:51 - INFO - trainer -     acc = 0.8357142857142857
07/25/2021 17:05:51 - INFO - trainer -     f1 = 0.7459476798866719
07/25/2021 17:05:59 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:05:59 - INFO - trainer -     Num examples = 1120
07/25/2021 17:05:59 - INFO - trainer -     Batch size = 32
07/25/2021 17:06:00 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:06:00 - INFO - trainer -     eval loss: 0.7800166217776547
07/25/2021 17:06:00 - INFO - trainer -     acc = 0.8446428571428571
07/25/2021 17:06:00 - INFO - trainer -     f1 = 0.7695124044982403
07/25/2021 17:06:08 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:06:08 - INFO - trainer -     Num examples = 1120
07/25/2021 17:06:08 - INFO - trainer -     Batch size = 32
07/25/2021 17:06:09 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:06:09 - INFO - trainer -     eval loss: 0.9481594916940335
07/25/2021 17:06:09 - INFO - trainer -     acc = 0.8348214285714

07/25/2021 17:08:24 - INFO - trainer -     eval loss: 1.0596918020039863
07/25/2021 17:08:24 - INFO - trainer -     acc = 0.85
07/25/2021 17:08:24 - INFO - trainer -     f1 = 0.7785431229517892
Epoch:  50%|█████     | 15/30 [05:04<05:00, 20.02s/it]07/25/2021 17:08:31 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:08:31 - INFO - trainer -     Num examples = 1120
07/25/2021 17:08:31 - INFO - trainer -     Batch size = 32
07/25/2021 17:08:32 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:08:32 - INFO - trainer -     eval loss: 1.1226529174214062
07/25/2021 17:08:32 - INFO - trainer -     acc = 0.8526785714285714
07/25/2021 17:08:32 - INFO - trainer -     f1 = 0.7783913362873891
07/25/2021 17:08:40 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:08:40 - INFO - trainer -     Num examples = 1120
07/25/2021 17:08:40 - INFO - trainer -     Batch size = 32
07/25/2021 17:08:41 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:08:41 - IN

07/25/2021 17:10:53 - INFO - trainer -     Batch size = 32
07/25/2021 17:10:54 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:10:54 - INFO - trainer -     eval loss: 1.2547459171772062
07/25/2021 17:10:54 - INFO - trainer -     acc = 0.84375
07/25/2021 17:10:54 - INFO - trainer -     f1 = 0.7627501314205699
07/25/2021 17:11:02 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:11:02 - INFO - trainer -     Num examples = 1120
07/25/2021 17:11:02 - INFO - trainer -     Batch size = 32
07/25/2021 17:11:03 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:11:03 - INFO - trainer -     eval loss: 1.2077302853297234
07/25/2021 17:11:03 - INFO - trainer -     acc = 0.8357142857142857
07/25/2021 17:11:03 - INFO - trainer -     f1 = 0.7445107391722429
Epoch:  77%|███████▋  | 23/30 [07:45<02:20, 20.09s/it]07/25/2021 17:11:12 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:11:12 - INFO - trainer -     Num examples = 1120
07/25/2021 17:11:12 -

07/25/2021 17:13:26 - INFO - trainer -     Num examples = 1120
07/25/2021 17:13:26 - INFO - trainer -     Batch size = 32
07/25/2021 17:13:27 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:13:27 - INFO - trainer -     eval loss: 1.333614505670474
07/25/2021 17:13:27 - INFO - trainer -     acc = 0.8357142857142857
07/25/2021 17:13:27 - INFO - trainer -     f1 = 0.747818073552347
Epoch: 100%|██████████| 30/30 [10:07<00:00, 20.24s/it]
07/25/2021 17:13:33 - INFO - __main__ -     acc = 0.8553571428571428
07/25/2021 17:13:33 - INFO - __main__ -     f1 = 0.7867476210655476


### Layer 11 + Layer 9 FC1, Frozen 0

In [7]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 11
args.hidden_layer_b_no = 9
args.frozen_percent = 0

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/25/2021 17:15:29 - INFO - model -   Hidden Layer 11 & 9, Frozen rate 0
07/25/2021 17:15:29 - INFO - trainer -   ***** Running training *****
07/25/2021 17:15:29 - INFO - trainer -     Num examples = 3602
07/25/2021 17:15:29 - INFO - trainer -     Num Epochs = 30
07/25/2021 17:15:29 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/25/2021 17:15:29 - INFO - trainer -     Gradient Accumulation steps = 2
07/25/2021 17:15:29 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/25/2021 17:15:37 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:15:37 - INFO - trainer -     Num examples = 1120
07/25/2021 17:15:37 - INFO - trainer -     Batch size = 32
07/25/2021 17:15:38 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:15:38 - INFO - trainer -     eval loss: 0.5823655941656658
07/25/2021 17:15:38 - INFO - trainer -     acc = 0.7589285714285714
07/25/2021 17:15:38 - INFO - trainer -     f1 = 0.6019723

07/25/2021 17:17:47 - INFO - trainer -     acc = 0.8491071428571428
07/25/2021 17:17:47 - INFO - trainer -     f1 = 0.7794311945187461
07/25/2021 17:17:55 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:17:55 - INFO - trainer -     Num examples = 1120
07/25/2021 17:17:55 - INFO - trainer -     Batch size = 32
07/25/2021 17:17:56 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:17:56 - INFO - trainer -     eval loss: 0.8844967043892081
07/25/2021 17:17:56 - INFO - trainer -     acc = 0.8366071428571429
07/25/2021 17:17:56 - INFO - trainer -     f1 = 0.7558126542460237
07/25/2021 17:18:03 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:18:03 - INFO - trainer -     Num examples = 1120
07/25/2021 17:18:03 - INFO - trainer -     Batch size = 32
07/25/2021 17:18:04 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:18:04 - INFO - trainer -     eval loss: 0.7616656521634598
07/25/2021 17:18:04 - INFO - trainer -     acc = 0.8526785714285

07/25/2021 17:20:12 - INFO - trainer -     eval loss: 1.0396709539761235
07/25/2021 17:20:12 - INFO - trainer -     acc = 0.8455357142857143
07/25/2021 17:20:12 - INFO - trainer -     f1 = 0.7761660921319695
Epoch:  50%|█████     | 15/30 [04:49<04:45, 19.00s/it]07/25/2021 17:20:19 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:20:19 - INFO - trainer -     Num examples = 1120
07/25/2021 17:20:19 - INFO - trainer -     Batch size = 32
07/25/2021 17:20:20 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:20:20 - INFO - trainer -     eval loss: 1.3345708275430037
07/25/2021 17:20:20 - INFO - trainer -     acc = 0.8285714285714286
07/25/2021 17:20:20 - INFO - trainer -     f1 = 0.7208105824193867
07/25/2021 17:20:28 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:20:28 - INFO - trainer -     Num examples = 1120
07/25/2021 17:20:28 - INFO - trainer -     Batch size = 32
07/25/2021 17:20:29 - INFO - trainer -   ***** Eval results *****
07/25/2021

07/25/2021 17:22:34 - INFO - trainer -     Batch size = 32
07/25/2021 17:22:35 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:22:35 - INFO - trainer -     eval loss: 1.2384326501088383
07/25/2021 17:22:35 - INFO - trainer -     acc = 0.8419642857142857
07/25/2021 17:22:35 - INFO - trainer -     f1 = 0.7638349389422295
07/25/2021 17:22:42 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:22:42 - INFO - trainer -     Num examples = 1120
07/25/2021 17:22:42 - INFO - trainer -     Batch size = 32
07/25/2021 17:22:43 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:22:43 - INFO - trainer -     eval loss: 1.3663837600542008
07/25/2021 17:22:43 - INFO - trainer -     acc = 0.8491071428571428
07/25/2021 17:22:43 - INFO - trainer -     f1 = 0.7735710061552247
Epoch:  77%|███████▋  | 23/30 [07:21<02:12, 18.86s/it]07/25/2021 17:22:51 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:22:51 - INFO - trainer -     Num examples = 1120
07/25/2021

07/25/2021 17:25:00 - INFO - trainer -     Num examples = 1120
07/25/2021 17:25:00 - INFO - trainer -     Batch size = 32
07/25/2021 17:25:02 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:25:02 - INFO - trainer -     eval loss: 1.4939184922757704
07/25/2021 17:25:02 - INFO - trainer -     acc = 0.8366071428571429
07/25/2021 17:25:02 - INFO - trainer -     f1 = 0.7397435686700166
Epoch: 100%|██████████| 30/30 [09:37<00:00, 19.26s/it]
07/25/2021 17:25:07 - INFO - __main__ -     acc = 0.8580357142857142
07/25/2021 17:25:07 - INFO - __main__ -     f1 = 0.7948079066504997


### ### Layer 10 + Layer 8 FC1, Frozen 0

In [8]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])

args.hidden_layer_a_no = 10
args.hidden_layer_b_no = 8
args.frozen_percent = 0

if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        best_acc_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_acc_eval_result
        best_f1_eval_results_by_2layer[(args.hidden_layer_a_no, args.hidden_layer_b_no, args.frozen_percent)] = best_f1_eval_result

07/25/2021 17:26:56 - INFO - model -   Hidden Layer 10 & 8, Frozen rate 0
07/25/2021 17:26:56 - INFO - trainer -   ***** Running training *****
07/25/2021 17:26:56 - INFO - trainer -     Num examples = 3602
07/25/2021 17:26:56 - INFO - trainer -     Num Epochs = 30
07/25/2021 17:26:56 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/25/2021 17:26:56 - INFO - trainer -     Gradient Accumulation steps = 2
07/25/2021 17:26:56 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/25/2021 17:27:03 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:27:03 - INFO - trainer -     Num examples = 1120
07/25/2021 17:27:03 - INFO - trainer -     Batch size = 32
07/25/2021 17:27:04 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:27:04 - INFO - trainer -     eval loss: 0.6163517317601612
07/25/2021 17:27:04 - INFO - trainer -     acc = 0.7651785714285714
07/25/2021 17:27:04 - INFO - trainer -     f1 = 0.5968786

07/25/2021 17:29:07 - INFO - trainer -     acc = 0.8339285714285715
07/25/2021 17:29:07 - INFO - trainer -     f1 = 0.7382029149724371
07/25/2021 17:29:14 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:29:14 - INFO - trainer -     Num examples = 1120
07/25/2021 17:29:14 - INFO - trainer -     Batch size = 32
07/25/2021 17:29:15 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:29:15 - INFO - trainer -     eval loss: 0.8685822151335222
07/25/2021 17:29:15 - INFO - trainer -     acc = 0.8383928571428572
07/25/2021 17:29:15 - INFO - trainer -     f1 = 0.7570508171578171
07/25/2021 17:29:22 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:29:22 - INFO - trainer -     Num examples = 1120
07/25/2021 17:29:22 - INFO - trainer -     Batch size = 32
07/25/2021 17:29:24 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:29:24 - INFO - trainer -     eval loss: 0.9981313866105795
07/25/2021 17:29:24 - INFO - trainer -     acc = 0.8258928571428

07/25/2021 17:31:25 - INFO - trainer -     eval loss: 1.1574899566243402
07/25/2021 17:31:25 - INFO - trainer -     acc = 0.8410714285714286
07/25/2021 17:31:25 - INFO - trainer -     f1 = 0.7608228958331784
Epoch:  50%|█████     | 15/30 [04:35<04:32, 18.16s/it]07/25/2021 17:31:32 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:31:32 - INFO - trainer -     Num examples = 1120
07/25/2021 17:31:32 - INFO - trainer -     Batch size = 32
07/25/2021 17:31:33 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:31:33 - INFO - trainer -     eval loss: 1.1814913662848994
07/25/2021 17:31:33 - INFO - trainer -     acc = 0.8473214285714286
07/25/2021 17:31:33 - INFO - trainer -     f1 = 0.76071121618882
07/25/2021 17:31:40 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:31:40 - INFO - trainer -     Num examples = 1120
07/25/2021 17:31:40 - INFO - trainer -     Batch size = 32
07/25/2021 17:31:41 - INFO - trainer -   ***** Eval results *****
07/25/2021 1

07/25/2021 17:33:41 - INFO - trainer -     Batch size = 32
07/25/2021 17:33:42 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:33:42 - INFO - trainer -     eval loss: 1.257109990290233
07/25/2021 17:33:42 - INFO - trainer -     acc = 0.8303571428571429
07/25/2021 17:33:42 - INFO - trainer -     f1 = 0.7513277221221147
07/25/2021 17:33:49 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:33:49 - INFO - trainer -     Num examples = 1120
07/25/2021 17:33:49 - INFO - trainer -     Batch size = 32
07/25/2021 17:33:50 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:33:50 - INFO - trainer -     eval loss: 1.2404053391223506
07/25/2021 17:33:50 - INFO - trainer -     acc = 0.8419642857142857
07/25/2021 17:33:50 - INFO - trainer -     f1 = 0.7601332339894431
Epoch:  77%|███████▋  | 23/30 [07:00<02:06, 18.06s/it]07/25/2021 17:33:57 - INFO - trainer -   ***** Running evaluation *****
07/25/2021 17:33:57 - INFO - trainer -     Num examples = 1120
07/25/2021 

07/25/2021 17:35:58 - INFO - trainer -     Num examples = 1120
07/25/2021 17:35:58 - INFO - trainer -     Batch size = 32
07/25/2021 17:35:59 - INFO - trainer -   ***** Eval results *****
07/25/2021 17:35:59 - INFO - trainer -     eval loss: 1.5868061026857634
07/25/2021 17:35:59 - INFO - trainer -     acc = 0.8303571428571429
07/25/2021 17:35:59 - INFO - trainer -     f1 = 0.7345413375262151
Epoch: 100%|██████████| 30/30 [09:08<00:00, 18.28s/it]
07/25/2021 17:36:04 - INFO - __main__ -     acc = 0.8625
07/25/2021 17:36:04 - INFO - __main__ -     f1 = 0.7969309261884127
